# Initializations

In [4]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:98% !important; }</style>"))
%load_ext autoreload  
%autoreload 2
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
%matplotlib inline

In [5]:
import os
import sys
import argparse
import csv
import json
import time
import types, copy, pprint
from datetime import datetime
for p in ['./src','../..']:
    if p not in sys.path:
        print(f"insert {p}")
        sys.path.insert(0, p)
print(sys.path)
import pickle
import pathlib
import copy
# import io

import numpy as np
import pandas as pd
import plotly.express as px
import plotly.io as pio
# import s3fs
from datetime import datetime
import scipy
print(scipy.__version__)

# from utils_cellpainting import (read_profiles_from_local, cat_columns,  get_cols_like,
                                 # disp_stats, check_values, )
# from KevinsRoutines.utils.utils_cellpainting import (disp_metadata_file, read_profiles_from_local, cat_columns,  get_cols_like,
#                                  disp_stats, check_values, )

# from KevinsRoutines.utils.utils_general  import  list_namespace 
# from scipy.stats import median_absolute_deviation
from pycytominer import normalize, feature_select

from pprint import PrettyPrinter
pp = PrettyPrinter(indent=4)
pd.options.display.width = 132
# torch.set_printoptions(precision=None, threshold=None, edgeitems=None, linewidth=150, profile=None, sci_mode=None)
np.set_printoptions(edgeitems=3, infstr='inf', linewidth=150, nanstr='nan')

pio.renderers.default = "png"  # Set to "svg" or "png" for static plots or "notebook_connected" for interactive plots
pd.set_option('display.width', 140)

insert ./src
insert ../..
['../..', './src', '/home/kevin/WSL-shared/cellpainting/cj-datasets', '/home/kevin/miniforge3/envs/cp311/lib/python311.zip', '/home/kevin/miniforge3/envs/cp311/lib/python3.11', '/home/kevin/miniforge3/envs/cp311/lib/python3.11/lib-dynload', '', '/home/kevin/miniforge3/envs/cp311/lib/python3.11/site-packages', '/home/kevin/miniforge3/envs/cp311/lib/python3.11/site-packages/huggingface_hub-0.20.3-py3.8.egg']
1.11.4


In [6]:
# time.strftime(' %x%X')
# datetime.now().strftime('%X.%f')
# time.strftime('%X %x %Z')
print(datetime.now().strftime('%D-%X.%f'))
print(datetime.now().strftime('%Y-%M-%d %H:%m:%S.%f' ))

import logging 
logLevel = os.environ.get('LOG_LEVEL', 'INFO').upper()
FORMAT = '%(asctime)s - %(levelname)s: - %(message)s'
logging.basicConfig(level="INFO", format= FORMAT)
logging.getLogger("imported_module").setLevel(logging.CRITICAL)

10/24/24-15:37:35.886302
2024-37-24 15:10:35.886538


In [7]:
import dask
import dask.array as da
import dask.dataframe as dd
from dask import delayed
from dask.distributed import Client
from dask.distributed import LocalCluster
from dask_cuda import LocalCUDACluster
import dask_ml.model_selection as dcv
 

/home/kevin/miniforge3/envs/cp311/lib/python3.11/site-packages/dask/dataframe/_pyarrow_compat.py:17: FutureWarning:

Minimal version of pyarrow will soon be increased to 14.0.1. You are using 11.0.0. Please consider upgrading.



In [8]:
from KevinsRoutines.utils.utils_cellpainting import (disp_metadata_file, read_profiles_from_local, cat_columns,  get_cols_like,
                                 disp_stats, check_values, check_df_for_nans )



In [9]:
from utils.jcp_datasets import project_datasets, display_project_datasets
# ds = project_datasets(output_path = './output_10162024/')
ds = project_datasets(output_path = './output_11102023/')
display_project_datasets(ds)

 prefix                                      : 
 prefix_lc                                   : 
 metadata path                               : ./metadata/
 input path                                  : ./input/
 output path                                 : ./output_11102023/

 Input Datasets : 
 ---------------- 
 Compound Metadata Input File                : ./metadata/compound.csv
 Plate Metadata Input File                   : ./metadata/plate_new.csv
 Well Metadata Input File                    : ./metadata/well.csv

 Compound Metadata : 
 ------------------- 
 Metadata + Smiles                           : ./output_11102023/compound_metadata_smiles.csv
          + TPSA                             : ./output_11102023/compound_metadata_tpsa.csv
          + TPSA Cleaned (TPSA <> 0)         : ./output_11102023/compound_metadata_tpsa_clean.csv
          + Base64 Pharmacophores            : ./output_11102023/compound_metadads.ta_pcfp.csv

 Compound/TPSA File:
 -------------------
 compou

# Load Metadata Files

<!-- This notebook loads a small number of plates with precomputed features and the metadata information. -->

- Order Extended Metadata Data set by Source, Batch, Plate

The following files contain the metadata information for the entire dataset.
The schema is [here](metadata/README.md).

## Profiles Metadata file

In [117]:
print(f" Profile Metadata file:  {ds.profileMetadataFile}")

with open(ds.profileMetadataFile, 'rb') as f: 
     pickle_data = pickle.load(f)

 Profile Metadata file:  ./metadata/profile_metadata.pkl


In [118]:
print(f" Profile metadata dict  keys : {pickle_data.keys()}")

MetadataColumns =  pickle_data['metadata_columns']['MetadataNames']
MetadataNames   =  pickle_data['metadata_columns']['MetadataNames']
ParquetColumns = pickle_data['parquet_columns']
MetadataCategoricalColumns = pickle_data['metadata_columns']['MetadataCategoricalColumns']
MetadataFloatColumns       = pickle_data['metadata_columns']['MetadataFloatColumns']
MetadataIntegerColumns     = pickle_data['metadata_columns']['MetadataIntegerColumns']
WRITE_COMPOUND_PROFILE_COLUMNS = MetadataColumns + sorted(list(pickle_data['selected_columns']['selected']))
disp_metadata_file(pickle_data, WRITE_COMPOUND_PROFILE_COLUMNS)

 Profile metadata dict  keys : dict_keys(['all_profile_columns', 'metadata_columns', 'selected_columns', 'parquet_columns'])
--------------------------------------------------------------------------------
 all_profile_columns & COMPOUND_PROFILE_COLUMNS
--------------------------------------------------------------------------------
  Len all_profile_columns        : 3672
  Len COMPOUND_PROFILE_COLUMNS   : 1482

  all_profile_columns[:10]       : ['Metadata_Source', 'Metadata_Batch', 'Metadata_Plate', 'Metadata_Well', 'Metadata_JCP2022', 'Metadata_Hash', 'Metadata_Bin', 'Metadata_TPSA', 'Metadata_lnTPSA', 'Metadata_log10TPSA']
  COMPOUND_PROFILE_COLUMNS[:10]  : ['Metadata_Source', 'Metadata_Batch', 'Metadata_Plate', 'Metadata_Well', 'Metadata_JCP2022', 'Metadata_Hash', 'Metadata_Bin', 'Metadata_TPSA', 'Metadata_lnTPSA', 'Metadata_log10TPSA']

  all_profile_columns[10:15]     : ['Metadata_Permiation', 'Cells_AreaShape_Area', 'Cells_AreaShape_BoundingBoxArea', 'Cells_AreaShape_BoundingBo

### Parquet Profile Columns Metadata

   Parquet files column metadata

In [218]:
SelectedColumns = pickle_data['selected_columns']
print(f" Parquet_columns keys : {ParquetColumns.keys()}")
print(f" Selected_columns keys: {SelectedColumns.keys()}")

parquetAllColumns = ParquetColumns['Cells'] | ParquetColumns['Cytoplasm'] | ParquetColumns['Nuclei'] | ParquetColumns['Image'] | ParquetColumns['Metadata']
parquetNumericColumns  =  ParquetColumns['Cells'] | ParquetColumns['Cytoplasm'] | ParquetColumns['Nuclei']
total_dropped = 0 
for k in ['dropped_correlation', 'dropped_variance', 'dropped_nans', 'dropped_image_features', 'dropped_metadata']:
    total_dropped += len(SelectedColumns[k])
    
print(total_dropped)
print(f"\n")
print(f" Parquet      - Metadata columns:                                             {len(ParquetColumns['Metadata']):7d}")
print(f" File         - Image feature columns:                                        {len(ParquetColumns['Image']):7d}")
print(f" Pre-Drop     - Numeric columns (Cells + Cytoplasm + Nuclei):                 {len(ParquetColumns['Cells']):7d}")
print(f"              - Numeric columns (Cells + Cytoplasm + Nuclei):                 {len(ParquetColumns['Cytoplasm']):7d}")
print(f"              - Numeric columns (Cells + Cytoplasm + Nuclei):                 {len(ParquetColumns['Nuclei']):7d}")
print(f"                                               total columns:                 {len(parquetAllColumns):7d}")
print()
print(f" Dropped      - Correlation:                                                  {len(SelectedColumns['dropped_correlation']):7d}")
print(f" Features     - Variance:                                                     {len(SelectedColumns['dropped_variance']):7d}")
print(f"              - NaN values:                                                   {len(SelectedColumns['dropped_nans']):7d}")
print(f"              - Image Feature Columns:                                        {len(SelectedColumns['dropped_image_features']):7d}")
print(f"              - Metadata Columns:                                             {len(SelectedColumns['dropped_metadata']):7d}\n")
print(f"              - TOTAL Dropped Columns:                                        {total_dropped:7d}")

# print()
# print(f" Selected     - Metadata columns:                                             {len(ParquetColumns['Metadata']):7d}")
# print(f" Features     - Image feature columns:                                        {len(ParquetColumns['Image']):7d}")
# print(f"              - Numeric columns (Cells + Cytoplasm + Nuclei):                 {len(ParquetColumns['Cells']):7d}")
# print(f"              - Numeric columns (Cells + Cytoplasm + Nuclei):                 {len(ParquetColumns['Cytoplasm']):7d}")
# print(f"              - Numeric columns (Cells + Cytoplasm + Nuclei):                 {len(ParquetColumns['Nuclei']):7d}\n")
# print(f"              Parquet file total columns:                                     {len(parquetAllColumns):7d}")
# print()
# print(f"              - Selected numeric columns (Cells + Cytoplasm + Nuclei):        {len(parquetAllColumns) - total_dropped:7d}")
# print(f"              - Selected Numeric columns:                                     {len(SelectedColumns['selected']):7d}")
# print()
# print(f" Selected     - Metadata columns:                                             {len(ParquetColumns['Metadata']):7d}")
# print(f" Columns      - Image feature columns:                                        {len(ParquetColumns['Image']):7d}")
# print(f"              - Numeric columns (Cells + Cytoplasm + Nuclei):                 {len(ParquetColumns['Cells']):7d}")
# print(f"              - Numeric columns (Cells + Cytoplasm + Nuclei):                 {len(ParquetColumns['Cytoplasm']):7d}")
# print(f"              - Numeric columns (Cells + Cytoplasm + Nuclei):                 {len(ParquetColumns['Nuclei']):7d}\n")
# print(f"              Parquet file total columns:                                     {len(parquetAllColumns):7d}")

##-----------------------------------------------------------------------
## Remove numeric columns that contains bad values
##-----------------------------------------------------------------------
print()
for key in ParquetColumns.keys():
    before = len(ParquetColumns[key])
    for x in pickle_data['selected_columns']['dropped_nans']: 
        if x in ParquetColumns[key] :
            del ParquetColumns[key][x]
            # print(f" \t FOUND in ParquetColumns['{key}'], deleted -- {x:35s}")
        # else:
            # print(f"\t NOT FOUND in ParquetColumns['{key}'], skipped -- {x:35s}")
            # pass
    # print(f"ParquetColumns['{key}'] : \t ttl columns before / after dropping Nans Columns : {before:5d}/{len(ParquetColumns[key]):5d}")
 
parquetAllColumns = ParquetColumns['Cells'] | ParquetColumns['Cytoplasm'] | ParquetColumns['Nuclei'] | ParquetColumns['Image'] | ParquetColumns['Metadata']
parquetNumericColumns  =  ParquetColumns['Cells'] | ParquetColumns['Cytoplasm'] | ParquetColumns['Nuclei']

READ_PARQUET_COLUMNS = list(ParquetColumns['Metadata'] | parquetNumericColumns)
READ_SELECTED_PARQUET_COLUMNS_ONLY = list(ParquetColumns['Metadata'].keys()) +  sorted(SelectedColumns['selected'])
JOIN_COLUMNS = MetadataColumns + list(parquetNumericColumns)

print()
print(f" Parquet      - Metadata columns:                                             {len(ParquetColumns['Metadata']):7d}")
print(f" File         - Image feature columns:                                        {len(ParquetColumns['Image']):7d}")
print(f" Post-Drop    - Numeric columns (Cells + Cytoplasm + Nuclei):                 {len(ParquetColumns['Cells']):7d}")
print(f"              - Numeric columns (Cells + Cytoplasm + Nuclei):                 {len(ParquetColumns['Cytoplasm']):7d}")
print(f"              - Numeric columns (Cells + Cytoplasm + Nuclei):                 {len(ParquetColumns['Nuclei']):7d}")
print(f"                                               total columns:                 {len(parquetAllColumns):7d}")
print()
print(f"Selected      - Selected Numeric columns:                                     {len(SelectedColumns['selected']):7d}")
print(f"Columns      ")
print()
print(f" PARQUET COLUMNS : ")
print("-"*60)
print(f"   Metadata Columns : {len(ParquetColumns['Metadata']):4d}    : {ParquetColumns['Metadata']}")
print(f"   Numeric Columns  : {len(parquetNumericColumns):4d}")
# print(f"   Numeric columns :")
print(f"   ------------------")
col_dict = cat_columns(parquetNumericColumns)
print()
print(f"   Total (READ_PARQUET_COLUMNS)     : {len(READ_PARQUET_COLUMNS):4d}  : {READ_PARQUET_COLUMNS[:5]} ...")
print('\n')

print(f" JOIN_COLUMNS : join between parquet and compound extended metadata ")
print("-"*60)
print(f"   Metadata Columns              : {len(MetadataColumns):5d}")
print(f"   profile Numneric Columns      : {len(parquetNumericColumns):5d}")
print(f"   all Columns                   : {len(JOIN_COLUMNS):5d}")   
print('\n')

 Parquet_columns keys : dict_keys(['Metadata', 'Cells', 'Cytoplasm', 'Nuclei', 'Image'])
 Selected_columns keys: dict_keys(['selected', 'dropped_correlation', 'dropped_variance', 'dropped_nans', 'dropped_image_features', 'dropped_metadata'])
3294


 Parquet      - Metadata columns:                                                   3
 File         - Image feature columns:                                           1089
 Pre-Drop     - Numeric columns (Cells + Cytoplasm + Nuclei):                    1249
              - Numeric columns (Cells + Cytoplasm + Nuclei):                    1240
              - Numeric columns (Cells + Cytoplasm + Nuclei):                    1172
                                               total columns:                    4753

 Dropped      - Correlation:                                                     2190
 Features     - Variance:                                                           0
              - NaN values:                                   

In [133]:
##-----------------------------------------------------------------------
## Conversion dictionary for reading CSV files 
##-----------------------------------------------------------------------
profileConversionDict = dict()
## Add numeric features 
## Add Metadata float columns to conversionDict:
## Add Integer metadata columns to conversion Dict
## Add Categorical metadata columns to conversion Dict
profileConversionDict |= {x:y  for x,y in MetadataFloatColumns.items()}
profileConversionDict |= {x:y  for x,y in MetadataIntegerColumns.items()}
profileConversionDict |= {x:y  for x,y in MetadataCategoricalColumns.items()}
profileConversionDict |= {x: np.float32 for x in parquetNumericColumns.keys()} 
print(f" conversion dict length (after addition of Float,Integer,Categorical and Numeric columns): {len(profileConversionDict)}")
## one column is read with no conversion

 conversion dict length (after addition of Float,Integer,Categorical and Numeric columns): 3671


In [ ]:
file_reader_1 =  pd.read_csv('./output_11102023/3_sample_profiles/3sample_profiles_1471_HashOrder.csv', 
                        # names = COMPOUND_PROFILE_COLUMNS,
                        header = 0,
                        skiprows = 0,
                        # nrows = 1,
                        # dtype = Xy_columns_dtype,
                        usecols = ['Metadata_JCP2022', 'Metadata_Hash', 'Metadata_Bin',],
                        # iterator = True,
                        # chunksize = 1000,
                        )

file_reader_1.value_counts()

In [1]:
df_fp = pd.read_pickle(compoundPharmacophoreFile)
df_fp.info()
df_fp.head(10)
df_fp.tail(10)

file_reader_2.value_counts()

NameError: name 'pd' is not defined

# Step 5 - Apply feature selection

In this step, we apply a series of feature selection steps to **level 4a profiles** (normalized well-level profiles) to form **level 4b profiles**.

### Read existing feature_selection csv file

In [59]:
# Apply feature selection
profilesFile = CompoundProfiles2SampleFileCSV +'.'+ type_bz2
featureSelectionFile = './output_11102023//normalized_feature_select.csv'
# featureSelectionFile = pathlib.Path(f"{output_dir}/{plate_id}_normalized_feature_select.csv.gz")
print(f" Profiles file       :  {profilesFile}")
print(f" Features select file:  {featureSelectionFile}")

 Profiles file       :  ./output_11102023/compound_profiles_2samples.csv.bz2
 Features select file:  ./output_11102023//normalized_feature_select.csv


**Pick columns we want to analyse**

In [60]:
for grp in cell_fld_grps:
# for grp in cytoplasm_fld_grps:
# for grp in nuclei_fld_grps:
    tmpColumns = get_cols_like(numeric_columns,grp )
    print(f" {grp:40s}      {len(tmpColumns)}")


 Cells_AreaShape                               53
 Cells_Children                                1
 Cells_Correlation                             80
 Cells_Granularity                             80
 Cells_Intensity                               75
 Cells_Neighbors                               14
 Cells_Parent                                  1
 Cells_RadialDistribution                      171
 Cells_Texture                                 780
 Cells_Texture_AngularSecondMoment             60
 Cells_Texture_Contrast                        60
 Cells_Texture_Correlation                     60
 Cells_Texture_DifferenceEntropy               60
 Cells_Texture_DifferenceVariance              60
 Cells_Texture_Entropy                         60
 Cells_Texture_InfoMeas1                       60
 Cells_Texture_InfoMeas2                       60
 Cells_Texture_InverseDifferenceMoment         60
 Cells_Texture_SumAverage                      60
 Cells_Texture_SumEntropy                      60


## `feature_select()` documentation

`feature_select()` Parameters

-   **profiles :** `pandas.core.frame.DataFrame` or file\
        DataFrame or file of profiles.
-   **features :** list\
        A list of strings corresponding to feature measurement column names in the `profiles` DataFrame.\
        All features listed must be found in `profiles`.\
        Defaults to `"infer"`. If `"infer"`, then assume cell painting features are those prefixed with `"Cells"`, `"Nuclei"`, or `"Cytoplasm"`.
-    **image_features:** bool, default False\
        Whether the profiles contain image features.
-    **samples :** list or str, default "all"\
        Samples to provide operation on.
-    **operation:** list of str or str, default "variance_threshold"\
        Operations to perform on the input profiles.\
     **Supported operations**
     -  **variance_threshold:** Exclude features that have low variance (low information content)
        - **freq_cut :** float, default 0.05 - Ratio (2nd most common feature val / most common). Must range between 0 and 1.
        - **unique_cut :** float, default 0.01 - Ratio (num unique features / num samples). Must range between 0 and 1.
     -  **correlation_threshold:** Exclude features that have correlations above a certain threshold
        - **threshold :** float, default 0.9 -  Must be between (0, 1) to exclude feature
        - **method :**  `pearson`, `spearman`, `kendall`, default `pearson` - indicating which correlation metric to use to test cutoff
     -  **blocklist:** Exclude features
     -  **drop_outliers:** Exclude a feature if its min or max absolute value is greater than the threshold.
     -  **noise_removal:** Remove noisy features, as defined by features with excessive standard deviation within the same perturbation group.
        - **noise_removal_perturb_groups :**
        - **noise_removal_stdev_cutoff :**
     -  **drop_na_columns:**  Get features that have more NA values than cutoff defined
        - **outlier_cutoff :**
-    **output_file :** str, optional\
        If provided, will write feature selected profiles to file. If not specified, will return the feature selected profiles as output.
        We recommend that this output file be suffixed with "_normalized_variable_selected.csv".
-    **output_type :** str, optional\
        If provided, will write feature selected profiles as a specified file type (either CSV or parquet).
        If not specified and output_file is provided, then the file will be outputed as CSV as default.
-    **na_cutoff :** float, default 0.05\
        Proportion of missing values in a column to tolerate before removing.
-    **corr_threshold :** float, default 0.9\
        Value between (0, 1) to exclude features above if any two features are correlated above this threshold.
-    **corr_method :** str, default `pearson`\
        Correlation type to compute. Allowed methods are `spearman`, `kendall` and `pearson`.
-    **freq_cut :** float, default 0.05\
        Ratio (2nd most common feature val / most common). Must range between 0 and 1.
        Remove features lower than freq_cut. A low freq_cut will remove features
        that have large difference between the most common feature and second most
        common feature. (e.g. this will remove a feature: [1, 1, 1, 1, 0.01, 0.01, ...])
-    **unique_cut:** float, default 0.01\
        Ratio (num unique features / num samples). Must range between 0 and 1.
        Remove features less than unique cut. A low unique_cut will remove features
        that have very few different measurements compared to the number of samples.
-    **compression_options :** str or dict, optional\
        Contains compression options as input to
        pd.DataFrame.to_csv(compression=compression_options). pandas version >= 1.2.
-    **float_format :** str, optional\
        Decimal precision to use in writing output file as input to
        pd.DataFrame.to_csv(float_format=float_format). For example, use "%.3g" for 3
        decimal precision.
-    **blocklist_file :** str, optional\
        File location of datafrmame with with features to exclude. Note that if "blocklist" in operation then will remove standard blocklist
-    **outlier_cutoff :** float, default 500\
        The threshold at which the maximum or minimum value of a feature across a full experiment is excluded. Note that this procedure is typically applied after normalization.
-    **noise_removal_perturb_groups:** str or list of str, optional\
        Perturbation groups corresponding to rows in profiles or the the name of the metadata column containing this information.
-    **noise_removal_stdev_cutoff:** float,optional\
        Maximum mean feature standard deviation to be kept for noise removal, grouped by the identity of the perturbation from perturb_list. The data must already be normalized so that this cutoff can apply to all columns.

**Returns**

-  **selected_df :** `pandas.core.frame.DataFrame`, optional \
    The feature selected profile DataFrame.
    If output_file=None, then return the DataFrame.
    If you specify output_file, then write to file and do not return data.

### Define different cell groups , used for feature selection

In [ ]:
image_fld_grps = {'_'.join(x.split('_')[:2]) for x in ParquetColumns['Image'] }
for grp in image_fld_grps:
    imageColumns = get_cols_like(ParquetColumns['Image'],grp )
    print(f" {grp:40s}      {len(imageColumns)}")
print('\n')

cell_fld_grps = ['Cells_AreaShape', 'Cells_Children', 'Cells_Correlation', 
                 ## 'Cells_Correlation_Correlation', 'Cells_Correlation_K', 'Cells_Correlation_Manders','Cells_Correlation_Overlap', 'Cells_Correlation_RWC',
                 ## 'Cells_RadialDistribution_FracAtD', 'Cells_RadialDistribution_MeanFrac', 'Cells_RadialDistribution_RadialCV',
                 'Cells_Granularity', 'Cells_Intensity', 'Cells_Neighbors', 'Cells_Parent',
                 'Cells_RadialDistribution', 
                 'Cells_Texture', 
                 'Cells_Texture_AngularSecondMoment','Cells_Texture_Contrast', 'Cells_Texture_Correlation',
                 'Cells_Texture_DifferenceEntropy', 'Cells_Texture_DifferenceVariance', 'Cells_Texture_Entropy',
                 'Cells_Texture_InfoMeas1' , 'Cells_Texture_InfoMeas2' , 'Cells_Texture_InverseDifferenceMoment',
                 'Cells_Texture_SumAverage', 'Cells_Texture_SumEntropy', 'Cells_Texture_SumVariance',  'Cells_Texture_Variance']

for grp in cell_fld_grps:
    cellColumns = get_cols_like(parquetNumericColumns,grp )
    print(f" {grp:40s}      {len(cellColumns)}")
print('\n')    

cytoplasm_fld_grps = ['Cytoplasm_Texture',
                      'Cytoplasm_Number',
                      'Cytoplasm_Intensity',
                      'Cytoplasm_RadialDistribution',
                      'Cytoplasm_Granularity',
                      'Cytoplasm_Correlation',
                      'Cytoplasm_AreaShape']

for grp in cytoplasm_fld_grps:
    cellColumns = get_cols_like(parquetNumericColumns,grp )
    print(f" {grp:40s}      {len(cellColumns)}")
print('\n')

nuclei_fld_grps = ['Nuclei_Location',
                   'Nuclei_Neighbors',
                   'Nuclei_ObjectSkeleton',
                   'Nuclei_Parent',
                   'Nuclei_Number',
                   'Nuclei_Intensity',
                   'Nuclei_Children',
                   'Nuclei_RadialDistribution',
                   'Nuclei_Granularity',
                   'Nuclei_Correlation',
                   'Nuclei_AreaShape',
                   'Nuclei_Texture']
 
for grp in nuclei_fld_grps:
    cellColumns = get_cols_like(parquetNumericColumns,grp )
    print(f" {grp:40s}      {len(cellColumns)}")


## Column group setup 

**Pick columns we want to analyse**

In [ ]:
featureColumns = {}

# featureColumns_1 = get_cols_like(numeric_columns, ['Cells_AreaShape', 'Cells_Correlation', 'Cells_Granularity',
#                                                   'Cells_Intensity', 'Cells_Children', 'Cells_Neighbors', 'Cells_Parent'])
# featureColumns_2 = get_cols_like(numeric_columns, ['Cells_RadialDistribution', 'Cells_RadialDistribution_FracAtD', 
#                                                    'Cells_RadialDistribution_MeanFrac', 'Cells_RadialDistribution_RadialCV'])

# featureColumns_3 = get_cols_like(numeric_columns, ['Cells_Texture_AngularSecondMoment', 'Cells_Texture_Contrast', 'Cells_Texture_Correlation',
#                                                    'Cells_Texture_DifferenceEntropy', 'Cells_Texture_DifferenceVariance', 'Cells_Texture_Entropy'])
# featureColumns_4 = get_cols_like(numeric_columns, ['Cells_Texture_InfoMeas1' , 'Cells_Texture_InfoMeas2' , 'Cells_Texture_InverseDifferenceMoment',
#                                                     'Cells_Texture_SumAverage', 'Cells_Texture_SumEntropy', 'Cells_Texture_SumVariance',  'Cells_Texture_Variance' ])

featureColumns['Cells_Other'] = get_cols_like(numeric_columns, ['Cells_AreaShape', 'Cells_Children', 'Cells_Correlation', 'Cells_Granularity',
                                                  'Cells_Intensity',  'Cells_Neighbors', 'Cells_Parent',
                                                  'Cells_RadialDistribution']) 
 
featureColumns['Cells_Texture'] = get_cols_like(numeric_columns, ['Cells_Texture'])


featureColumns['Cytoplasm_Other'] = get_cols_like(numeric_columns, ['Cytoplasm_Number',  'Cytoplasm_Intensity', 'Cytoplasm_RadialDistribution',
                                                  'Cytoplasm_Granularity',  'Cytoplasm_Correlation', 'Cytoplasm_AreaShape'])

featureColumns['Cytoplasm_Texture'] = get_cols_like(numeric_columns, ['Cytoplasm_Texture'])

featureColumns['Nuclei_Other'] = get_cols_like(numeric_columns, ['Nuclei_Location'   , 'Nuclei_Neighbors'  , 'Nuclei_ObjectSkeleton', 'Nuclei_Parent',
                                                   'Nuclei_Number'     , 'Nuclei_Intensity'  , 'Nuclei_Children'      , 'Nuclei_RadialDistribution',
                                                   'Nuclei_Granularity', 'Nuclei_Correlation', 'Nuclei_AreaShape'     ,])

featureColumns['Nuclei_Other'] = get_cols_like(numeric_columns, ['Nuclei_Texture'])

for i, (k,v) in enumerate(featureColumns.items()):
    print(f" {i:2d}:  {k:25s}    count: {len(v)}")


In [ ]:
for k in ['Metadata_permiation', 'Metadata_Source', 'Metadata_Batch', 'Metadata_Plate', 'Metadata_Well']:
    print(k, conversionDict[k])


In [ ]:
extMetadataNames
readColumns =  extMetadataNames+ featureColumns['Cells_Texture']
print(len(featureColumns['Cells_Texture']))
len(readColumns)

for i in readColumns:
    if i not in conversionDict:
        print(f" {i:25s} not in conversionDict")

## Read profiles file

In [ ]:
x = 4
profilesFile = CompoundProfiles2SampleFile + f"_{x:03d}" + CSV
print(f" Profiles file       :  {profilesFile}")
print(f" Features select file:  {selectedFeaturesFile}")
print(f"-"*100)

In [ ]:
df_profiles =  pd.read_csv(profilesFile, 
                       low_memory = True, 
                       # chunksize= 15000,  
                       # skiprows=30000, 
                       # iterator = True, 
                       names=JOIN_COLUMNS,
                       dtype = conversionDict,
                       usecols = readColumns)

In [ ]:
df_profiles.shape
print(f"\nSize of loaded dataframe: {df_profiles.memory_usage(deep=True).sum():,d}\n")
df_profiles.info()
df_profiles.head(3)
df_profiles_columns = df_profiles.columns.to_list()
len(readColumns), len(df_profiles_columns)

## Run feature selection

In [ ]:
feature_select_opts = [
    # "variance_threshold",
    "correlation_threshold",
    # "drop_na_columns",
    # "blocklist",
    # "drop_outliers"
]
compression_options = {'method':' bz2'}

In [ ]:
df_feature_selection = feature_select(
                               profiles=df_profiles,
                               features="infer",
                               samples="all",
                               corr_threshold=0.95,
                               freq_cut = 0.05,
                               unique_cut = 0.01,
                               operation=feature_select_opts,
                               # output_file=featureSelectionFile,
                               # compression_options=compression_options,
                              )

In [ ]:
df_feature_selection.info()
df_feature_selection.head(10)

selected_cols = df_feature_selection.columns.to_list()
dropped_cols  = set(readColumns) - set(selected_cols)

print(f" Read columns:     {len(readColumns)}")
print(f" Selected columns: {len(selected_cols)}")
print(f" Dropped columns:  {len(dropped_cols)}")

### Print dropped and selected columns names in tabular fashion to use below

In [ ]:
def print_list_tabular(input_cols, list_name = 'list', num_cols = 4):
    _input_cols = copy.copy(input_cols)
    len_cols = len(_input_cols)
    
    ## Remove Metadata_* columns 
    i = 0
    while i < len_cols:
        if _input_cols[i].startswith("Metadata_"):
            _input_cols.pop(i)
            len_cols -= 1
        else:
            i += 1
            
    tmp_cols = ["\""+x+"\"" for x in _input_cols ]
    
    print(f"\n## Number of columns in list: {len_cols} \n")
    print(f"{list_name} = {{ \\ \n")
    
    for i in range(0, len_cols, num_cols):
        for j in range(i,i+num_cols,1):
            print("   ", end ="")
            if j < len_cols:
                print(f" {tmp_cols[j]:60s} " , end = ',')
            else:
                break
        print()
    print(" }")

In [ ]:
print_list_tabular(selected_cols, 'selected_cols_1')

In [ ]:
# print_list_tabular(dropped_cols, 'correlation_dropped_cols_1')

In [ ]:
# print_list_tabular(dropped_cols, 'variance_dropped_cols_1')

In [ ]:
disp_stats(df_profiles, selected_cols[55:65])

In [ ]:
disp_stats(df_profiles, list(dropped_cols)[55:65])

In [ ]:
# n = df_profiles.shape[0]
# print(n)
# for drop_col in dropped_columns:
#     n_unique = df_profiles[drop_col].nunique()
#     v_counts = df_profiles[drop_col].value_counts()
#     print(f"     {drop_col:50s}:    {n_unique:15d}     unique ratio: {n_unique/n:.3f}")
#     print(v_counts.head(10))

## Feature Selection drops :
- `Cells` features dropped : 1080,  Retained: 98 
- `Cytoplasm` features dropped : 1007, Retained: 154
- `Nuclei` feature dropped : 1049, Retained: 138


### Cells - group 1

In [267]:
# { 
#  "Cells_Children_Cytoplasm_Count"                             , "Cells_Neighbors_SecondClosestObjectNumber_5"                , "Cells_Neighbors_SecondClosestObjectNumber_Adjacent"         , "Cells_Parent_CellsIncludingEdges"                           ,
#  "Cells_RadialDistribution_FracAtD_mito_tubeness_Overflow"    , "Cells_RadialDistribution_MeanFrac_mito_tubeness_Overflow"   , "Cells_RadialDistribution_RadialCV_mito_tubeness_Overflow" }

selected_cols_1 = \
{"Cells_AreaShape_BoundingBoxMaximum_X"                       , "Cells_AreaShape_Center_X"                                   , "Cells_AreaShape_Center_Y"                                   , "Cells_AreaShape_Compactness"                                ,
 "Cells_AreaShape_Eccentricity"                               , "Cells_AreaShape_EulerNumber"                                , "Cells_AreaShape_Extent"                                     , "Cells_AreaShape_MajorAxisLength"                            ,
 "Cells_AreaShape_MedianRadius"                               , "Cells_AreaShape_MinorAxisLength"                            , "Cells_AreaShape_Orientation"                                , "Cells_AreaShape_Perimeter"                                  ,
 "Cells_AreaShape_Solidity"                                   , "Cells_AreaShape_Zernike_0_0"                                , "Cells_AreaShape_Zernike_1_1"                                , "Cells_AreaShape_Zernike_2_0"                                ,
 "Cells_AreaShape_Zernike_2_2"                                , "Cells_AreaShape_Zernike_3_1"                                , "Cells_AreaShape_Zernike_3_3"                                , "Cells_AreaShape_Zernike_4_0"                                ,
 "Cells_AreaShape_Zernike_4_2"                                , "Cells_AreaShape_Zernike_4_4"                                , "Cells_AreaShape_Zernike_5_1"                                , "Cells_AreaShape_Zernike_5_3"                                ,
 "Cells_AreaShape_Zernike_5_5"                                , "Cells_AreaShape_Zernike_6_0"                                , "Cells_AreaShape_Zernike_6_2"                                , "Cells_AreaShape_Zernike_6_4"                                ,
 "Cells_AreaShape_Zernike_6_6"                                , "Cells_AreaShape_Zernike_7_1"                                , "Cells_AreaShape_Zernike_7_3"                                , "Cells_AreaShape_Zernike_7_5"                                ,
 "Cells_AreaShape_Zernike_7_7"                                , "Cells_AreaShape_Zernike_8_0"                                , "Cells_AreaShape_Zernike_8_2"                                , "Cells_AreaShape_Zernike_8_4"                                ,
 "Cells_AreaShape_Zernike_8_6"                                , "Cells_AreaShape_Zernike_8_8"                                , "Cells_AreaShape_Zernike_9_1"                                , "Cells_AreaShape_Zernike_9_3"                                ,
 "Cells_AreaShape_Zernike_9_5"                                , "Cells_AreaShape_Zernike_9_7"                                , "Cells_AreaShape_Zernike_9_9"                                , "Cells_Children_Cytoplasm_Count"                             ,
 "Cells_Correlation_Correlation_AGP_DNA"                      , "Cells_Correlation_Correlation_AGP_ER"                       , "Cells_Correlation_Correlation_AGP_Mito"                     , "Cells_Correlation_Correlation_DNA_ER"                       ,
 "Cells_Correlation_Correlation_DNA_Mito"                     , "Cells_Correlation_Correlation_DNA_RNA"                      , "Cells_Correlation_Correlation_ER_Mito"                      , "Cells_Correlation_Correlation_ER_RNA"                       ,
 "Cells_Correlation_Correlation_Mito_RNA"                     , "Cells_Correlation_K_AGP_DNA"                                , "Cells_Correlation_K_AGP_ER"                                 , "Cells_Correlation_K_AGP_Mito"                               ,
 "Cells_Correlation_K_AGP_RNA"                                , "Cells_Correlation_K_DNA_AGP"                                , "Cells_Correlation_K_DNA_ER"                                 , "Cells_Correlation_K_DNA_Mito"                               ,
 "Cells_Correlation_K_DNA_RNA"                                , "Cells_Correlation_K_ER_AGP"                                 , "Cells_Correlation_K_ER_DNA"                                 , "Cells_Correlation_K_ER_Mito"                                ,
 "Cells_Correlation_K_ER_RNA"                                 , "Cells_Correlation_K_Mito_AGP"                               , "Cells_Correlation_K_Mito_DNA"                               , "Cells_Correlation_K_Mito_ER"                                ,
 "Cells_Correlation_K_Mito_RNA"                               , "Cells_Correlation_K_RNA_AGP"                                , "Cells_Correlation_K_RNA_DNA"                                , "Cells_Correlation_K_RNA_ER"                                 ,
 "Cells_Correlation_K_RNA_Mito"                               , "Cells_Correlation_Manders_AGP_DNA"                          , "Cells_Correlation_Manders_AGP_ER"                           , "Cells_Correlation_Manders_AGP_Mito"                         ,
 "Cells_Correlation_Manders_AGP_RNA"                          , "Cells_Correlation_Manders_DNA_AGP"                          , "Cells_Correlation_Manders_DNA_ER"                           , "Cells_Correlation_Manders_DNA_Mito"                         ,
 "Cells_Correlation_Manders_DNA_RNA"                          , "Cells_Correlation_Manders_ER_AGP"                           , "Cells_Correlation_Manders_ER_DNA"                           , "Cells_Correlation_Manders_ER_Mito"                          ,
 "Cells_Correlation_Manders_ER_RNA"                           , "Cells_Correlation_Manders_Mito_AGP"                         , "Cells_Correlation_Manders_Mito_DNA"                         , "Cells_Correlation_Manders_Mito_ER"                          ,
 "Cells_Correlation_Manders_Mito_RNA"                         , "Cells_Correlation_Manders_RNA_AGP"                          , "Cells_Correlation_Manders_RNA_DNA"                          , "Cells_Correlation_Manders_RNA_ER"                           ,
 "Cells_Correlation_Manders_RNA_Mito"                         , "Cells_Correlation_Overlap_AGP_DNA"                          , "Cells_Correlation_Overlap_AGP_ER"                           , "Cells_Correlation_Overlap_AGP_Mito"                         ,
 "Cells_Correlation_Overlap_AGP_RNA"                          , "Cells_Correlation_Overlap_DNA_ER"                           , "Cells_Correlation_Overlap_DNA_Mito"                         , "Cells_Correlation_Overlap_DNA_RNA"                          ,
 "Cells_Correlation_Overlap_ER_Mito"                          , "Cells_Correlation_Overlap_ER_RNA"                           , "Cells_Correlation_Overlap_Mito_RNA"                         , "Cells_Correlation_RWC_AGP_DNA"                              ,
 "Cells_Correlation_RWC_AGP_ER"                               , "Cells_Correlation_RWC_AGP_Mito"                             , "Cells_Correlation_RWC_AGP_RNA"                              , "Cells_Correlation_RWC_DNA_AGP"                              ,
 "Cells_Correlation_RWC_DNA_ER"                               , "Cells_Correlation_RWC_DNA_Mito"                             , "Cells_Correlation_RWC_DNA_RNA"                              , "Cells_Correlation_RWC_ER_AGP"                               ,
 "Cells_Correlation_RWC_ER_DNA"                               , "Cells_Correlation_RWC_ER_Mito"                              , "Cells_Correlation_RWC_ER_RNA"                               , "Cells_Correlation_RWC_Mito_AGP"                             ,
 "Cells_Correlation_RWC_Mito_DNA"                             , "Cells_Correlation_RWC_Mito_ER"                              , "Cells_Correlation_RWC_Mito_RNA"                             , "Cells_Correlation_RWC_RNA_AGP"                              ,
 "Cells_Correlation_RWC_RNA_DNA"                              , "Cells_Correlation_RWC_RNA_ER"                               , "Cells_Correlation_RWC_RNA_Mito"                             , "Cells_Granularity_10_AGP"                                   ,
 "Cells_Granularity_10_DNA"                                   , "Cells_Granularity_10_ER"                                    , "Cells_Granularity_10_Mito"                                  , "Cells_Granularity_10_RNA"                                   ,
 "Cells_Granularity_11_AGP"                                   , "Cells_Granularity_11_DNA"                                   , "Cells_Granularity_11_ER"                                    , "Cells_Granularity_11_Mito"                                  ,
 "Cells_Granularity_11_RNA"                                   , "Cells_Granularity_12_AGP"                                   , "Cells_Granularity_12_DNA"                                   , "Cells_Granularity_12_ER"                                    ,
 "Cells_Granularity_12_Mito"                                  , "Cells_Granularity_12_RNA"                                   , "Cells_Granularity_13_AGP"                                   , "Cells_Granularity_13_DNA"                                   ,
 "Cells_Granularity_13_ER"                                    , "Cells_Granularity_13_Mito"                                  , "Cells_Granularity_13_RNA"                                   , "Cells_Granularity_14_AGP"                                   ,
 "Cells_Granularity_14_DNA"                                   , "Cells_Granularity_14_ER"                                    , "Cells_Granularity_14_Mito"                                  , "Cells_Granularity_14_RNA"                                   ,
 "Cells_Granularity_15_AGP"                                   , "Cells_Granularity_15_DNA"                                   , "Cells_Granularity_15_ER"                                    , "Cells_Granularity_15_Mito"                                  ,
 "Cells_Granularity_15_RNA"                                   , "Cells_Granularity_16_AGP"                                   , "Cells_Granularity_16_DNA"                                   , "Cells_Granularity_16_ER"                                    ,
 "Cells_Granularity_16_Mito"                                  , "Cells_Granularity_16_RNA"                                   , "Cells_Granularity_1_AGP"                                    , "Cells_Granularity_1_DNA"                                    ,
 "Cells_Granularity_1_ER"                                     , "Cells_Granularity_1_Mito"                                   , "Cells_Granularity_2_AGP"                                    , "Cells_Granularity_2_DNA"                                    ,
 "Cells_Granularity_2_ER"                                     , "Cells_Granularity_2_Mito"                                   , "Cells_Granularity_2_RNA"                                    , "Cells_Granularity_3_AGP"                                    ,
 "Cells_Granularity_3_DNA"                                    , "Cells_Granularity_3_ER"                                     , "Cells_Granularity_3_Mito"                                   , "Cells_Granularity_3_RNA"                                    ,
 "Cells_Granularity_4_AGP"                                    , "Cells_Granularity_4_DNA"                                    , "Cells_Granularity_4_ER"                                     , "Cells_Granularity_4_Mito"                                   ,
 "Cells_Granularity_4_RNA"                                    , "Cells_Granularity_5_AGP"                                    , "Cells_Granularity_5_DNA"                                    , "Cells_Granularity_5_ER"                                     ,
 "Cells_Granularity_5_Mito"                                   , "Cells_Granularity_5_RNA"                                    , "Cells_Granularity_6_AGP"                                    , "Cells_Granularity_6_DNA"                                    ,
 "Cells_Granularity_6_ER"                                     , "Cells_Granularity_6_Mito"                                   , "Cells_Granularity_6_RNA"                                    , "Cells_Granularity_7_AGP"                                    ,
 "Cells_Granularity_7_DNA"                                    , "Cells_Granularity_7_ER"                                     , "Cells_Granularity_7_Mito"                                   , "Cells_Granularity_7_RNA"                                    ,
 "Cells_Granularity_8_AGP"                                    , "Cells_Granularity_8_DNA"                                    , "Cells_Granularity_8_ER"                                     , "Cells_Granularity_8_Mito"                                   ,
 "Cells_Granularity_8_RNA"                                    , "Cells_Granularity_9_AGP"                                    , "Cells_Granularity_9_DNA"                                    , "Cells_Granularity_9_ER"                                     ,
 "Cells_Granularity_9_Mito"                                   , "Cells_Granularity_9_RNA"                                    , "Cells_Intensity_IntegratedIntensityEdge_AGP"                , "Cells_Intensity_IntegratedIntensityEdge_DNA"                ,
 "Cells_Intensity_IntegratedIntensityEdge_ER"                 , "Cells_Intensity_IntegratedIntensityEdge_Mito"               , "Cells_Intensity_IntegratedIntensityEdge_RNA"                , "Cells_Intensity_IntegratedIntensity_AGP"                    ,
 "Cells_Intensity_IntegratedIntensity_DNA"                    , "Cells_Intensity_IntegratedIntensity_ER"                     , "Cells_Intensity_IntegratedIntensity_Mito"                   , "Cells_Intensity_IntegratedIntensity_RNA"                    ,
 "Cells_Intensity_LowerQuartileIntensity_AGP"                 , "Cells_Intensity_LowerQuartileIntensity_DNA"                 , "Cells_Intensity_LowerQuartileIntensity_ER"                  , "Cells_Intensity_LowerQuartileIntensity_Mito"                ,
 "Cells_Intensity_LowerQuartileIntensity_RNA"                 , "Cells_Intensity_MADIntensity_AGP"                           , "Cells_Intensity_MADIntensity_DNA"                           , "Cells_Intensity_MADIntensity_ER"                            ,
 "Cells_Intensity_MADIntensity_Mito"                          , "Cells_Intensity_MADIntensity_RNA"                           , "Cells_Intensity_MassDisplacement_AGP"                       , "Cells_Intensity_MassDisplacement_DNA"                       ,
 "Cells_Intensity_MassDisplacement_ER"                        , "Cells_Intensity_MassDisplacement_Mito"                      , "Cells_Intensity_MaxIntensityEdge_AGP"                       , "Cells_Intensity_MaxIntensityEdge_DNA"                       ,
 "Cells_Intensity_MaxIntensityEdge_Mito"                      , "Cells_Intensity_MaxIntensityEdge_RNA"                       , "Cells_Intensity_MaxIntensity_AGP"                           , "Cells_Intensity_MaxIntensity_DNA"                           ,
 "Cells_Intensity_MaxIntensity_ER"                            , "Cells_Intensity_MaxIntensity_Mito"                          , "Cells_Intensity_MaxIntensity_RNA"                           , "Cells_Intensity_MeanIntensityEdge_AGP"                      ,
 "Cells_Intensity_MeanIntensityEdge_DNA"                      , "Cells_Intensity_MeanIntensityEdge_ER"                       , "Cells_Intensity_MeanIntensity_DNA"                          , "Cells_Intensity_MeanIntensity_ER"                           ,
 "Cells_Intensity_MedianIntensity_DNA"                        , "Cells_Intensity_MedianIntensity_ER"                         , "Cells_Intensity_MedianIntensity_Mito"                       , "Cells_Intensity_MinIntensityEdge_RNA"                       ,
 "Cells_Intensity_MinIntensity_AGP"                           , "Cells_Intensity_MinIntensity_DNA"                           , "Cells_Intensity_MinIntensity_ER"                            , "Cells_Intensity_MinIntensity_Mito"                          ,
 "Cells_Intensity_StdIntensityEdge_AGP"                       , "Cells_Intensity_StdIntensityEdge_ER"                        , "Cells_Intensity_StdIntensityEdge_RNA"                       , "Cells_Intensity_StdIntensity_AGP"                           ,
 "Cells_Intensity_StdIntensity_DNA"                           , "Cells_Intensity_StdIntensity_ER"                            , "Cells_Intensity_StdIntensity_Mito"                          , "Cells_Intensity_StdIntensity_RNA"                           ,
 "Cells_Intensity_UpperQuartileIntensity_AGP"                 , "Cells_Intensity_UpperQuartileIntensity_DNA"                 , "Cells_Intensity_UpperQuartileIntensity_RNA"                 , "Cells_Neighbors_AngleBetweenNeighbors_5"                    ,
 "Cells_Neighbors_FirstClosestDistance_5"                     , "Cells_Neighbors_NumberOfNeighbors_Adjacent"                 , "Cells_Neighbors_SecondClosestDistance_Adjacent"             , "Cells_Neighbors_SecondClosestObjectNumber_Adjacent"         ,
 "Cells_RadialDistribution_FracAtD_AGP_1of4"                  , "Cells_RadialDistribution_FracAtD_AGP_3of4"                  , "Cells_RadialDistribution_FracAtD_AGP_4of4"                  , "Cells_RadialDistribution_FracAtD_DNA_1of4"                  ,
 "Cells_RadialDistribution_FracAtD_DNA_2of4"                  , "Cells_RadialDistribution_FracAtD_DNA_3of4"                  , "Cells_RadialDistribution_FracAtD_ER_1of4"                   , "Cells_RadialDistribution_FracAtD_ER_3of4"                   ,
 "Cells_RadialDistribution_FracAtD_ER_4of4"                   , "Cells_RadialDistribution_FracAtD_Mito_2of4"                 , "Cells_RadialDistribution_FracAtD_Mito_3of4"                 , "Cells_RadialDistribution_FracAtD_Mito_4of4"                 ,
 "Cells_RadialDistribution_FracAtD_RNA_1of4"                  , "Cells_RadialDistribution_FracAtD_RNA_3of4"                  , "Cells_RadialDistribution_FracAtD_RNA_4of4"                  , "Cells_RadialDistribution_FracAtD_mito_tubeness_10of16"      ,
 "Cells_RadialDistribution_FracAtD_mito_tubeness_10of20"      , "Cells_RadialDistribution_FracAtD_mito_tubeness_12of16"      , "Cells_RadialDistribution_FracAtD_mito_tubeness_13of20"      , "Cells_RadialDistribution_FracAtD_mito_tubeness_14of20"      ,
 "Cells_RadialDistribution_FracAtD_mito_tubeness_19of20"      , "Cells_RadialDistribution_FracAtD_mito_tubeness_1of16"       , "Cells_RadialDistribution_FracAtD_mito_tubeness_1of20"       , "Cells_RadialDistribution_FracAtD_mito_tubeness_20of20"      ,
 "Cells_RadialDistribution_FracAtD_mito_tubeness_2of20"       , "Cells_RadialDistribution_FracAtD_mito_tubeness_3of16"       , "Cells_RadialDistribution_FracAtD_mito_tubeness_7of20"       , "Cells_RadialDistribution_FracAtD_mito_tubeness_9of16"       ,
 "Cells_RadialDistribution_FracAtD_mito_tubeness_Overflow"    , "Cells_RadialDistribution_MeanFrac_AGP_1of4"                 , "Cells_RadialDistribution_MeanFrac_AGP_3of4"                 , "Cells_RadialDistribution_MeanFrac_AGP_4of4"                 ,
 "Cells_RadialDistribution_MeanFrac_DNA_2of4"                 , "Cells_RadialDistribution_MeanFrac_DNA_3of4"                 , "Cells_RadialDistribution_MeanFrac_DNA_4of4"                 , "Cells_RadialDistribution_MeanFrac_ER_1of4"                  ,
 "Cells_RadialDistribution_MeanFrac_ER_3of4"                  , "Cells_RadialDistribution_MeanFrac_ER_4of4"                  , "Cells_RadialDistribution_MeanFrac_Mito_1of4"                , "Cells_RadialDistribution_MeanFrac_Mito_3of4"                ,
 "Cells_RadialDistribution_MeanFrac_Mito_4of4"                , "Cells_RadialDistribution_MeanFrac_RNA_1of4"                 , "Cells_RadialDistribution_MeanFrac_RNA_3of4"                 , "Cells_RadialDistribution_MeanFrac_RNA_4of4"                 ,
 "Cells_RadialDistribution_MeanFrac_mito_tubeness_10of16"     , "Cells_RadialDistribution_MeanFrac_mito_tubeness_10of20"     , "Cells_RadialDistribution_MeanFrac_mito_tubeness_12of16"     , "Cells_RadialDistribution_MeanFrac_mito_tubeness_13of20"     ,
 "Cells_RadialDistribution_MeanFrac_mito_tubeness_14of20"     , "Cells_RadialDistribution_MeanFrac_mito_tubeness_16of20"     , "Cells_RadialDistribution_MeanFrac_mito_tubeness_17of20"     , "Cells_RadialDistribution_MeanFrac_mito_tubeness_19of20"     ,
 "Cells_RadialDistribution_MeanFrac_mito_tubeness_1of20"      , "Cells_RadialDistribution_MeanFrac_mito_tubeness_20of20"     , "Cells_RadialDistribution_MeanFrac_mito_tubeness_2of20"      , "Cells_RadialDistribution_MeanFrac_mito_tubeness_7of20"      ,
 "Cells_RadialDistribution_MeanFrac_mito_tubeness_9of16"      , "Cells_RadialDistribution_MeanFrac_mito_tubeness_Overflow"   , "Cells_RadialDistribution_RadialCV_AGP_1of4"                 , "Cells_RadialDistribution_RadialCV_AGP_2of4"                 ,
 "Cells_RadialDistribution_RadialCV_AGP_3of4"                 , "Cells_RadialDistribution_RadialCV_AGP_4of4"                 , "Cells_RadialDistribution_RadialCV_DNA_1of4"                 , "Cells_RadialDistribution_RadialCV_DNA_2of4"                 ,
 "Cells_RadialDistribution_RadialCV_DNA_3of4"                 , "Cells_RadialDistribution_RadialCV_DNA_4of4"                 , "Cells_RadialDistribution_RadialCV_ER_1of4"                  , "Cells_RadialDistribution_RadialCV_ER_2of4"                  ,
 "Cells_RadialDistribution_RadialCV_ER_3of4"                  , "Cells_RadialDistribution_RadialCV_ER_4of4"                  , "Cells_RadialDistribution_RadialCV_Mito_1of4"                , "Cells_RadialDistribution_RadialCV_Mito_2of4"                ,
 "Cells_RadialDistribution_RadialCV_Mito_3of4"                , "Cells_RadialDistribution_RadialCV_Mito_4of4"                , "Cells_RadialDistribution_RadialCV_RNA_1of4"                 , "Cells_RadialDistribution_RadialCV_RNA_2of4"                 ,
 "Cells_RadialDistribution_RadialCV_RNA_3of4"                 , "Cells_RadialDistribution_RadialCV_mito_tubeness_12of16"     , "Cells_RadialDistribution_RadialCV_mito_tubeness_13of20"     , "Cells_RadialDistribution_RadialCV_mito_tubeness_16of16"     ,
 "Cells_RadialDistribution_RadialCV_mito_tubeness_16of20"     , "Cells_RadialDistribution_RadialCV_mito_tubeness_17of20"     , "Cells_RadialDistribution_RadialCV_mito_tubeness_19of20"     , "Cells_RadialDistribution_RadialCV_mito_tubeness_1of16"      ,
 "Cells_RadialDistribution_RadialCV_mito_tubeness_1of20"      , "Cells_RadialDistribution_RadialCV_mito_tubeness_20of20"     , "Cells_RadialDistribution_RadialCV_mito_tubeness_2of16"      , "Cells_RadialDistribution_RadialCV_mito_tubeness_2of20"      ,
 "Cells_RadialDistribution_RadialCV_mito_tubeness_3of16"      , "Cells_RadialDistribution_RadialCV_mito_tubeness_3of20"      , "Cells_RadialDistribution_RadialCV_mito_tubeness_4of20"      , "Cells_RadialDistribution_RadialCV_mito_tubeness_5of16"      ,
 "Cells_RadialDistribution_RadialCV_mito_tubeness_5of20"      , "Cells_RadialDistribution_RadialCV_mito_tubeness_6of16"      , "Cells_RadialDistribution_RadialCV_mito_tubeness_7of16"      , "Cells_RadialDistribution_RadialCV_mito_tubeness_8of16"      ,
 "Cells_RadialDistribution_RadialCV_mito_tubeness_8of20"      , "Cells_RadialDistribution_RadialCV_mito_tubeness_9of16"      , "Cells_RadialDistribution_RadialCV_mito_tubeness_Overflow"   
}

correlation_dropped_cols_1 = \
{"Cells_AreaShape_Area"                                       , "Cells_AreaShape_BoundingBoxArea"                            , "Cells_AreaShape_BoundingBoxMaximum_Y"                       , "Cells_AreaShape_BoundingBoxMinimum_X"                       ,
 "Cells_AreaShape_BoundingBoxMinimum_Y"                       , "Cells_AreaShape_EquivalentDiameter"                         , "Cells_AreaShape_MaxFeretDiameter"                           , "Cells_AreaShape_MaximumRadius"                              ,
 "Cells_AreaShape_MeanRadius"                                 , "Cells_AreaShape_MinFeretDiameter"                           , "Cells_Correlation_Correlation_AGP_RNA"                      , "Cells_Granularity_1_RNA"                                    ,
 "Cells_Intensity_MassDisplacement_RNA"                       , "Cells_Intensity_MaxIntensityEdge_ER"                        , "Cells_Intensity_MeanIntensityEdge_Mito"                     , "Cells_Intensity_MeanIntensityEdge_RNA"                      ,
 "Cells_Intensity_MeanIntensity_AGP"                          , "Cells_Intensity_MeanIntensity_Mito"                         , "Cells_Intensity_MeanIntensity_RNA"                          , "Cells_Intensity_MedianIntensity_AGP"                        ,
 "Cells_Intensity_MedianIntensity_RNA"                        , "Cells_Intensity_MinIntensityEdge_AGP"                       , "Cells_Intensity_MinIntensityEdge_DNA"                       , "Cells_Intensity_MinIntensityEdge_ER"                        ,
 "Cells_Intensity_MinIntensityEdge_Mito"                      , "Cells_Intensity_MinIntensity_RNA"                           , "Cells_Intensity_StdIntensityEdge_DNA"                       , "Cells_Intensity_StdIntensityEdge_Mito"                      ,
 "Cells_Intensity_UpperQuartileIntensity_ER"                  , "Cells_Intensity_UpperQuartileIntensity_Mito"                , "Cells_Neighbors_AngleBetweenNeighbors_Adjacent"             , "Cells_Neighbors_FirstClosestDistance_Adjacent"              ,
 "Cells_Neighbors_FirstClosestObjectNumber_5"                 , "Cells_Neighbors_FirstClosestObjectNumber_Adjacent"          , "Cells_Neighbors_NumberOfNeighbors_5"                        , "Cells_Neighbors_PercentTouching_5"                          ,
 "Cells_Neighbors_PercentTouching_Adjacent"                   , "Cells_Neighbors_SecondClosestDistance_5"                    , "Cells_Neighbors_SecondClosestObjectNumber_5"                , "Cells_Parent_CellsIncludingEdges"                           ,
 "Cells_RadialDistribution_FracAtD_AGP_2of4"                  , "Cells_RadialDistribution_FracAtD_DNA_4of4"                  , "Cells_RadialDistribution_FracAtD_ER_2of4"                   , "Cells_RadialDistribution_FracAtD_Mito_1of4"                 ,
 "Cells_RadialDistribution_FracAtD_RNA_2of4"                  , "Cells_RadialDistribution_FracAtD_mito_tubeness_11of16"      , "Cells_RadialDistribution_FracAtD_mito_tubeness_11of20"      , "Cells_RadialDistribution_FracAtD_mito_tubeness_12of20"      ,
 "Cells_RadialDistribution_FracAtD_mito_tubeness_13of16"      , "Cells_RadialDistribution_FracAtD_mito_tubeness_14of16"      , "Cells_RadialDistribution_FracAtD_mito_tubeness_15of16"      , "Cells_RadialDistribution_FracAtD_mito_tubeness_15of20"      ,
 "Cells_RadialDistribution_FracAtD_mito_tubeness_16of16"      , "Cells_RadialDistribution_FracAtD_mito_tubeness_16of20"      , "Cells_RadialDistribution_FracAtD_mito_tubeness_17of20"      , "Cells_RadialDistribution_FracAtD_mito_tubeness_18of20"      ,
 "Cells_RadialDistribution_FracAtD_mito_tubeness_2of16"       , "Cells_RadialDistribution_FracAtD_mito_tubeness_3of20"       , "Cells_RadialDistribution_FracAtD_mito_tubeness_4of16"       , "Cells_RadialDistribution_FracAtD_mito_tubeness_4of20"       ,
 "Cells_RadialDistribution_FracAtD_mito_tubeness_5of16"       , "Cells_RadialDistribution_FracAtD_mito_tubeness_5of20"       , "Cells_RadialDistribution_FracAtD_mito_tubeness_6of16"       , "Cells_RadialDistribution_FracAtD_mito_tubeness_6of20"       ,
 "Cells_RadialDistribution_FracAtD_mito_tubeness_7of16"       , "Cells_RadialDistribution_FracAtD_mito_tubeness_8of16"       , "Cells_RadialDistribution_FracAtD_mito_tubeness_8of20"       , "Cells_RadialDistribution_FracAtD_mito_tubeness_9of20"       ,
 "Cells_RadialDistribution_MeanFrac_AGP_2of4"                 , "Cells_RadialDistribution_MeanFrac_DNA_1of4"                 , "Cells_RadialDistribution_MeanFrac_ER_2of4"                  , "Cells_RadialDistribution_MeanFrac_Mito_2of4"                ,
 "Cells_RadialDistribution_MeanFrac_RNA_2of4"                 , "Cells_RadialDistribution_MeanFrac_mito_tubeness_11of16"     , "Cells_RadialDistribution_MeanFrac_mito_tubeness_11of20"     , "Cells_RadialDistribution_MeanFrac_mito_tubeness_12of20"     ,
 "Cells_RadialDistribution_MeanFrac_mito_tubeness_13of16"     , "Cells_RadialDistribution_MeanFrac_mito_tubeness_14of16"     , "Cells_RadialDistribution_MeanFrac_mito_tubeness_15of16"     , "Cells_RadialDistribution_MeanFrac_mito_tubeness_15of20"     ,
 "Cells_RadialDistribution_MeanFrac_mito_tubeness_16of16"     , "Cells_RadialDistribution_MeanFrac_mito_tubeness_18of20"     , "Cells_RadialDistribution_MeanFrac_mito_tubeness_1of16"      , "Cells_RadialDistribution_MeanFrac_mito_tubeness_2of16"      ,
 "Cells_RadialDistribution_MeanFrac_mito_tubeness_3of16"      , "Cells_RadialDistribution_MeanFrac_mito_tubeness_3of20"      , "Cells_RadialDistribution_MeanFrac_mito_tubeness_4of16"      , "Cells_RadialDistribution_MeanFrac_mito_tubeness_4of20"      ,
 "Cells_RadialDistribution_MeanFrac_mito_tubeness_5of16"      , "Cells_RadialDistribution_MeanFrac_mito_tubeness_5of20"      , "Cells_RadialDistribution_MeanFrac_mito_tubeness_6of16"      , "Cells_RadialDistribution_MeanFrac_mito_tubeness_6of20"      ,
 "Cells_RadialDistribution_MeanFrac_mito_tubeness_7of16"      , "Cells_RadialDistribution_MeanFrac_mito_tubeness_8of16"      , "Cells_RadialDistribution_MeanFrac_mito_tubeness_8of20"      , "Cells_RadialDistribution_MeanFrac_mito_tubeness_9of20"      ,
 "Cells_RadialDistribution_RadialCV_RNA_4of4"                 , "Cells_RadialDistribution_RadialCV_mito_tubeness_10of16"     , "Cells_RadialDistribution_RadialCV_mito_tubeness_10of20"     , "Cells_RadialDistribution_RadialCV_mito_tubeness_11of16"     ,
 "Cells_RadialDistribution_RadialCV_mito_tubeness_11of20"     , "Cells_RadialDistribution_RadialCV_mito_tubeness_12of20"     , "Cells_RadialDistribution_RadialCV_mito_tubeness_13of16"     , "Cells_RadialDistribution_RadialCV_mito_tubeness_14of16"     ,
 "Cells_RadialDistribution_RadialCV_mito_tubeness_14of20"     , "Cells_RadialDistribution_RadialCV_mito_tubeness_15of16"     , "Cells_RadialDistribution_RadialCV_mito_tubeness_15of20"     , "Cells_RadialDistribution_RadialCV_mito_tubeness_18of20"     ,
 "Cells_RadialDistribution_RadialCV_mito_tubeness_4of16"      , "Cells_RadialDistribution_RadialCV_mito_tubeness_6of20"      , "Cells_RadialDistribution_RadialCV_mito_tubeness_7of20"      , "Cells_RadialDistribution_RadialCV_mito_tubeness_9of20"      
}

variance_dropped_cols_1 = set()

In [268]:
variance_dropped_cols_1 & correlation_dropped_cols_1
print(f" Number of cols dropped - high correlation: {len(correlation_dropped_cols_1)}")
print(f" Number of cols dropped - low variance    : {len(variance_dropped_cols_1)}")
print(f" Number of selected cols                  : {len(selected_cols_1)}")
print(f" Total                                    : {len(selected_cols_1)+len(correlation_dropped_cols_1)+len(variance_dropped_cols_1)}")

set()

 Number of cols dropped - high correlation: 112
 Number of cols dropped - low variance    : 0
 Number of selected cols                  : 363
 Total                                    : 475


In [274]:
(correlation_dropped_cols_1) & set(zero_features)
(variance_dropped_cols_1) & set(zero_features)
(selected_cols_1) & set(zero_features)

set()

set()

{'Cells_RadialDistribution_FracAtD_mito_tubeness_Overflow',
 'Cells_RadialDistribution_MeanFrac_mito_tubeness_Overflow',
 'Cells_RadialDistribution_RadialCV_mito_tubeness_Overflow'}

### Cells - group 2

In [275]:
selected_cols_2 = \
{"Cells_Texture_AngularSecondMoment_AGP_3_00_256"             , "Cells_Texture_AngularSecondMoment_DNA_10_03_256"            , "Cells_Texture_AngularSecondMoment_ER_10_01_256"             , "Cells_Texture_AngularSecondMoment_Mito_10_01_256"           ,
 "Cells_Texture_AngularSecondMoment_RNA_10_03_256"            , "Cells_Texture_Contrast_DNA_5_00_256"                        , "Cells_Texture_Contrast_Mito_3_00_256"                       , "Cells_Texture_Contrast_RNA_10_02_256"                       ,
 "Cells_Texture_Correlation_AGP_10_00_256"                    , "Cells_Texture_Correlation_AGP_10_01_256"                    , "Cells_Texture_Correlation_AGP_10_02_256"                    , "Cells_Texture_Correlation_AGP_10_03_256"                    ,
 "Cells_Texture_Correlation_AGP_3_00_256"                     , "Cells_Texture_Correlation_AGP_3_01_256"                     , "Cells_Texture_Correlation_AGP_3_02_256"                     , "Cells_Texture_Correlation_AGP_3_03_256"                     ,
 "Cells_Texture_Correlation_AGP_5_01_256"                     , "Cells_Texture_Correlation_DNA_10_00_256"                    , "Cells_Texture_Correlation_DNA_10_01_256"                    , "Cells_Texture_Correlation_DNA_10_02_256"                    ,
 "Cells_Texture_Correlation_DNA_10_03_256"                    , "Cells_Texture_Correlation_DNA_3_00_256"                     , "Cells_Texture_Correlation_DNA_3_01_256"                     , "Cells_Texture_Correlation_DNA_3_02_256"                     ,
 "Cells_Texture_Correlation_DNA_3_03_256"                     , "Cells_Texture_Correlation_DNA_5_01_256"                     , "Cells_Texture_Correlation_DNA_5_03_256"                     , "Cells_Texture_Correlation_ER_10_00_256"                     ,
 "Cells_Texture_Correlation_ER_10_01_256"                     , "Cells_Texture_Correlation_ER_10_02_256"                     , "Cells_Texture_Correlation_ER_10_03_256"                     , "Cells_Texture_Correlation_ER_3_00_256"                      ,
 "Cells_Texture_Correlation_Mito_10_00_256"                   , "Cells_Texture_Correlation_Mito_10_01_256"                   , "Cells_Texture_Correlation_Mito_10_02_256"                   , "Cells_Texture_Correlation_Mito_10_03_256"                   ,
 "Cells_Texture_Correlation_Mito_3_00_256"                    , "Cells_Texture_Correlation_Mito_3_02_256"                    , "Cells_Texture_Correlation_RNA_10_00_256"                    , "Cells_Texture_Correlation_RNA_10_01_256"                    ,
 "Cells_Texture_Correlation_RNA_10_02_256"                    , "Cells_Texture_Correlation_RNA_10_03_256"                    , "Cells_Texture_Correlation_RNA_3_00_256"                     , "Cells_Texture_DifferenceEntropy_AGP_3_00_256"               ,
 "Cells_Texture_DifferenceEntropy_RNA_3_00_256"               , "Cells_Texture_DifferenceVariance_AGP_3_00_256"              , "Cells_Texture_DifferenceVariance_DNA_10_01_256"             , "Cells_Texture_DifferenceVariance_ER_10_03_256"              ,
 "Cells_Texture_DifferenceVariance_Mito_3_00_256"             , "Cells_Texture_DifferenceVariance_RNA_10_03_256"             , "Cells_Texture_Entropy_AGP_10_03_256"                        , "Cells_Texture_Entropy_DNA_10_03_256"                        ,
 "Cells_Texture_Entropy_ER_10_01_256"                         , "Cells_Texture_Entropy_Mito_10_01_256"                       , "Cells_Texture_Entropy_RNA_10_03_256"                        , "Cells_Texture_InfoMeas1_AGP_10_00_256"                      ,
 "Cells_Texture_InfoMeas1_AGP_10_02_256"                      , "Cells_Texture_InfoMeas1_AGP_10_03_256"                      , "Cells_Texture_InfoMeas1_AGP_3_00_256"                       , "Cells_Texture_InfoMeas1_AGP_3_01_256"                       ,
 "Cells_Texture_InfoMeas1_AGP_3_02_256"                       , "Cells_Texture_InfoMeas1_AGP_3_03_256"                       , "Cells_Texture_InfoMeas1_DNA_10_00_256"                      , "Cells_Texture_InfoMeas1_DNA_10_01_256"                      ,
 "Cells_Texture_InfoMeas1_DNA_10_02_256"                      , "Cells_Texture_InfoMeas1_DNA_10_03_256"                      , "Cells_Texture_InfoMeas1_DNA_3_00_256"                       , "Cells_Texture_InfoMeas1_DNA_3_01_256"                       ,
 "Cells_Texture_InfoMeas1_DNA_3_02_256"                       , "Cells_Texture_InfoMeas1_DNA_3_03_256"                       , "Cells_Texture_InfoMeas1_DNA_5_00_256"                       , "Cells_Texture_InfoMeas1_DNA_5_01_256"                       ,
 "Cells_Texture_InfoMeas1_DNA_5_02_256"                       , "Cells_Texture_InfoMeas1_DNA_5_03_256"                       , "Cells_Texture_InfoMeas1_ER_10_00_256"                       , "Cells_Texture_InfoMeas1_ER_10_01_256"                       ,
 "Cells_Texture_InfoMeas1_ER_10_02_256"                       , "Cells_Texture_InfoMeas1_ER_10_03_256"                       , "Cells_Texture_InfoMeas1_ER_5_00_256"                        , "Cells_Texture_InfoMeas1_ER_5_01_256"                        ,
 "Cells_Texture_InfoMeas1_ER_5_02_256"                        , "Cells_Texture_InfoMeas1_ER_5_03_256"                        , "Cells_Texture_InfoMeas1_Mito_10_00_256"                     , "Cells_Texture_InfoMeas1_Mito_10_01_256"                     ,
 "Cells_Texture_InfoMeas1_Mito_10_02_256"                     , "Cells_Texture_InfoMeas1_Mito_10_03_256"                     , "Cells_Texture_InfoMeas1_Mito_3_00_256"                      , "Cells_Texture_InfoMeas1_Mito_3_02_256"                      ,
 "Cells_Texture_InfoMeas1_Mito_5_01_256"                      , "Cells_Texture_InfoMeas1_Mito_5_03_256"                      , "Cells_Texture_InfoMeas1_RNA_10_00_256"                      , "Cells_Texture_InfoMeas1_RNA_10_01_256"                      ,
 "Cells_Texture_InfoMeas1_RNA_10_02_256"                      , "Cells_Texture_InfoMeas1_RNA_10_03_256"                      , "Cells_Texture_InfoMeas1_RNA_5_00_256"                       , "Cells_Texture_InfoMeas1_RNA_5_01_256"                       ,
 "Cells_Texture_InfoMeas1_RNA_5_02_256"                       , "Cells_Texture_InfoMeas1_RNA_5_03_256"                       , "Cells_Texture_InfoMeas2_AGP_10_02_256"                      , "Cells_Texture_InfoMeas2_AGP_10_03_256"                      ,
 "Cells_Texture_InfoMeas2_AGP_3_00_256"                       , "Cells_Texture_InfoMeas2_AGP_3_01_256"                       , "Cells_Texture_InfoMeas2_AGP_3_02_256"                       , "Cells_Texture_InfoMeas2_AGP_3_03_256"                       ,
 "Cells_Texture_InfoMeas2_DNA_10_00_256"                      , "Cells_Texture_InfoMeas2_DNA_10_01_256"                      , "Cells_Texture_InfoMeas2_DNA_10_02_256"                      , "Cells_Texture_InfoMeas2_DNA_10_03_256"                      ,
 "Cells_Texture_InfoMeas2_DNA_3_02_256"                       , "Cells_Texture_InfoMeas2_ER_10_00_256"                       , "Cells_Texture_InfoMeas2_ER_10_01_256"                       , "Cells_Texture_InfoMeas2_ER_10_02_256"                       ,
 "Cells_Texture_InfoMeas2_ER_10_03_256"                       , "Cells_Texture_InfoMeas2_ER_3_02_256"                        , "Cells_Texture_InfoMeas2_Mito_3_02_256"                      , "Cells_Texture_InfoMeas2_RNA_10_00_256"                      ,
 "Cells_Texture_InfoMeas2_RNA_10_02_256"                      , "Cells_Texture_InfoMeas2_RNA_10_03_256"                      , "Cells_Texture_InfoMeas2_RNA_3_02_256"                       , "Cells_Texture_InverseDifferenceMoment_AGP_3_00_256"         ,
 "Cells_Texture_InverseDifferenceMoment_DNA_10_03_256"        , "Cells_Texture_InverseDifferenceMoment_ER_10_01_256"         , "Cells_Texture_InverseDifferenceMoment_ER_3_02_256"          , "Cells_Texture_InverseDifferenceMoment_Mito_10_01_256"       ,
 "Cells_Texture_InverseDifferenceMoment_RNA_10_03_256"        , "Cells_Texture_SumAverage_AGP_3_00_256"                      , "Cells_Texture_SumAverage_DNA_10_03_256"                     , "Cells_Texture_SumAverage_ER_3_00_256"                       ,
 "Cells_Texture_SumAverage_Mito_3_00_256"                     , "Cells_Texture_SumAverage_RNA_3_00_256"                      , "Cells_Texture_SumVariance_AGP_10_03_256"                    , "Cells_Texture_SumVariance_DNA_10_01_256"                    ,
 "Cells_Texture_SumVariance_ER_10_01_256"                     , "Cells_Texture_SumVariance_Mito_10_03_256"                   , "Cells_Texture_SumVariance_RNA_10_03_256"                        
}

correlation_dropped_cols_2 = \
{"Cells_Texture_AngularSecondMoment_AGP_10_00_256"            , "Cells_Texture_AngularSecondMoment_AGP_10_01_256"            , "Cells_Texture_AngularSecondMoment_AGP_10_02_256"            , "Cells_Texture_AngularSecondMoment_AGP_10_03_256"            ,
 "Cells_Texture_AngularSecondMoment_AGP_3_01_256"             , "Cells_Texture_AngularSecondMoment_AGP_3_02_256"             , "Cells_Texture_AngularSecondMoment_AGP_3_03_256"             , "Cells_Texture_AngularSecondMoment_AGP_5_00_256"             ,
 "Cells_Texture_AngularSecondMoment_AGP_5_01_256"             , "Cells_Texture_AngularSecondMoment_AGP_5_02_256"             , "Cells_Texture_AngularSecondMoment_AGP_5_03_256"             , "Cells_Texture_AngularSecondMoment_DNA_10_00_256"            ,
 "Cells_Texture_AngularSecondMoment_DNA_10_01_256"            , "Cells_Texture_AngularSecondMoment_DNA_10_02_256"            , "Cells_Texture_AngularSecondMoment_DNA_3_00_256"             , "Cells_Texture_AngularSecondMoment_DNA_3_01_256"             ,
 "Cells_Texture_AngularSecondMoment_DNA_3_02_256"             , "Cells_Texture_AngularSecondMoment_DNA_3_03_256"             , "Cells_Texture_AngularSecondMoment_DNA_5_00_256"             , "Cells_Texture_AngularSecondMoment_DNA_5_01_256"             ,
 "Cells_Texture_AngularSecondMoment_DNA_5_02_256"             , "Cells_Texture_AngularSecondMoment_DNA_5_03_256"             , "Cells_Texture_AngularSecondMoment_ER_10_00_256"             , "Cells_Texture_AngularSecondMoment_ER_10_02_256"             ,
 "Cells_Texture_AngularSecondMoment_ER_10_03_256"             , "Cells_Texture_AngularSecondMoment_ER_3_00_256"              , "Cells_Texture_AngularSecondMoment_ER_3_01_256"              , "Cells_Texture_AngularSecondMoment_ER_3_02_256"              ,
 "Cells_Texture_AngularSecondMoment_ER_3_03_256"              , "Cells_Texture_AngularSecondMoment_ER_5_00_256"              , "Cells_Texture_AngularSecondMoment_ER_5_01_256"              , "Cells_Texture_AngularSecondMoment_ER_5_02_256"              ,
 "Cells_Texture_AngularSecondMoment_ER_5_03_256"              , "Cells_Texture_AngularSecondMoment_Mito_10_00_256"           , "Cells_Texture_AngularSecondMoment_Mito_10_02_256"           , "Cells_Texture_AngularSecondMoment_Mito_10_03_256"           ,
 "Cells_Texture_AngularSecondMoment_Mito_3_00_256"            , "Cells_Texture_AngularSecondMoment_Mito_3_01_256"            , "Cells_Texture_AngularSecondMoment_Mito_3_02_256"            , "Cells_Texture_AngularSecondMoment_Mito_3_03_256"            ,
 "Cells_Texture_AngularSecondMoment_Mito_5_00_256"            , "Cells_Texture_AngularSecondMoment_Mito_5_01_256"            , "Cells_Texture_AngularSecondMoment_Mito_5_02_256"            , "Cells_Texture_AngularSecondMoment_Mito_5_03_256"            ,
 "Cells_Texture_AngularSecondMoment_RNA_10_00_256"            , "Cells_Texture_AngularSecondMoment_RNA_10_01_256"            , "Cells_Texture_AngularSecondMoment_RNA_10_02_256"            , "Cells_Texture_AngularSecondMoment_RNA_3_00_256"             ,
 "Cells_Texture_AngularSecondMoment_RNA_3_01_256"             , "Cells_Texture_AngularSecondMoment_RNA_3_02_256"             , "Cells_Texture_AngularSecondMoment_RNA_3_03_256"             , "Cells_Texture_AngularSecondMoment_RNA_5_00_256"             ,
 "Cells_Texture_AngularSecondMoment_RNA_5_01_256"             , "Cells_Texture_AngularSecondMoment_RNA_5_02_256"             , "Cells_Texture_AngularSecondMoment_RNA_5_03_256"             , "Cells_Texture_Contrast_AGP_10_00_256"                       ,
 "Cells_Texture_Contrast_AGP_10_01_256"                       , "Cells_Texture_Contrast_AGP_10_02_256"                       , "Cells_Texture_Contrast_AGP_10_03_256"                       , "Cells_Texture_Contrast_AGP_3_00_256"                        ,
 "Cells_Texture_Contrast_AGP_3_01_256"                        , "Cells_Texture_Contrast_AGP_3_02_256"                        , "Cells_Texture_Contrast_AGP_3_03_256"                        , "Cells_Texture_Contrast_AGP_5_00_256"                        ,
 "Cells_Texture_Contrast_AGP_5_01_256"                        , "Cells_Texture_Contrast_AGP_5_02_256"                        , "Cells_Texture_Contrast_AGP_5_03_256"                        , "Cells_Texture_Contrast_DNA_10_00_256"                       ,
 "Cells_Texture_Contrast_DNA_10_01_256"                       , "Cells_Texture_Contrast_DNA_10_02_256"                       , "Cells_Texture_Contrast_DNA_10_03_256"                       , "Cells_Texture_Contrast_DNA_3_00_256"                        ,
 "Cells_Texture_Contrast_DNA_3_01_256"                        , "Cells_Texture_Contrast_DNA_3_02_256"                        , "Cells_Texture_Contrast_DNA_3_03_256"                        , "Cells_Texture_Contrast_DNA_5_01_256"                        ,
 "Cells_Texture_Contrast_DNA_5_02_256"                        , "Cells_Texture_Contrast_DNA_5_03_256"                        , "Cells_Texture_Contrast_ER_10_00_256"                        , "Cells_Texture_Contrast_ER_10_01_256"                        ,
 "Cells_Texture_Contrast_ER_10_02_256"                        , "Cells_Texture_Contrast_ER_10_03_256"                        , "Cells_Texture_Contrast_ER_3_00_256"                         , "Cells_Texture_Contrast_ER_3_01_256"                         ,
 "Cells_Texture_Contrast_ER_3_02_256"                         , "Cells_Texture_Contrast_ER_3_03_256"                         , "Cells_Texture_Contrast_ER_5_00_256"                         , "Cells_Texture_Contrast_ER_5_01_256"                         ,
 "Cells_Texture_Contrast_ER_5_02_256"                         , "Cells_Texture_Contrast_ER_5_03_256"                         , "Cells_Texture_Contrast_Mito_10_00_256"                      , "Cells_Texture_Contrast_Mito_10_01_256"                      ,
 "Cells_Texture_Contrast_Mito_10_02_256"                      , "Cells_Texture_Contrast_Mito_10_03_256"                      , "Cells_Texture_Contrast_Mito_3_01_256"                       , "Cells_Texture_Contrast_Mito_3_02_256"                       ,
 "Cells_Texture_Contrast_Mito_3_03_256"                       , "Cells_Texture_Contrast_Mito_5_00_256"                       , "Cells_Texture_Contrast_Mito_5_01_256"                       , "Cells_Texture_Contrast_Mito_5_02_256"                       ,
 "Cells_Texture_Contrast_Mito_5_03_256"                       , "Cells_Texture_Contrast_RNA_10_00_256"                       , "Cells_Texture_Contrast_RNA_10_01_256"                       , "Cells_Texture_Contrast_RNA_10_03_256"                       ,
 "Cells_Texture_Contrast_RNA_3_00_256"                        , "Cells_Texture_Contrast_RNA_3_01_256"                        , "Cells_Texture_Contrast_RNA_3_02_256"                        , "Cells_Texture_Contrast_RNA_3_03_256"                        ,
 "Cells_Texture_Contrast_RNA_5_00_256"                        , "Cells_Texture_Contrast_RNA_5_01_256"                        , "Cells_Texture_Contrast_RNA_5_02_256"                        , "Cells_Texture_Contrast_RNA_5_03_256"                        ,
 "Cells_Texture_Correlation_AGP_5_00_256"                     , "Cells_Texture_Correlation_AGP_5_02_256"                     , "Cells_Texture_Correlation_AGP_5_03_256"                     , "Cells_Texture_Correlation_DNA_5_00_256"                     ,
 "Cells_Texture_Correlation_DNA_5_02_256"                     , "Cells_Texture_Correlation_ER_3_01_256"                      , "Cells_Texture_Correlation_ER_3_02_256"                      , "Cells_Texture_Correlation_ER_3_03_256"                      ,
 "Cells_Texture_Correlation_ER_5_00_256"                      , "Cells_Texture_Correlation_ER_5_01_256"                      , "Cells_Texture_Correlation_ER_5_02_256"                      , "Cells_Texture_Correlation_ER_5_03_256"                      ,
 "Cells_Texture_Correlation_Mito_3_01_256"                    , "Cells_Texture_Correlation_Mito_3_03_256"                    , "Cells_Texture_Correlation_Mito_5_00_256"                    , "Cells_Texture_Correlation_Mito_5_01_256"                    ,
 "Cells_Texture_Correlation_Mito_5_02_256"                    , "Cells_Texture_Correlation_Mito_5_03_256"                    , "Cells_Texture_Correlation_RNA_3_01_256"                     , "Cells_Texture_Correlation_RNA_3_02_256"                     ,
 "Cells_Texture_Correlation_RNA_3_03_256"                     , "Cells_Texture_Correlation_RNA_5_00_256"                     , "Cells_Texture_Correlation_RNA_5_01_256"                     , "Cells_Texture_Correlation_RNA_5_02_256"                     ,
 "Cells_Texture_Correlation_RNA_5_03_256"                     , "Cells_Texture_DifferenceEntropy_AGP_10_00_256"              , "Cells_Texture_DifferenceEntropy_AGP_10_01_256"              , "Cells_Texture_DifferenceEntropy_AGP_10_02_256"              ,
 "Cells_Texture_DifferenceEntropy_AGP_10_03_256"              , "Cells_Texture_DifferenceEntropy_AGP_3_01_256"               , "Cells_Texture_DifferenceEntropy_AGP_3_02_256"               , "Cells_Texture_DifferenceEntropy_AGP_3_03_256"               ,
 "Cells_Texture_DifferenceEntropy_AGP_5_00_256"               , "Cells_Texture_DifferenceEntropy_AGP_5_01_256"               , "Cells_Texture_DifferenceEntropy_AGP_5_02_256"               , "Cells_Texture_DifferenceEntropy_AGP_5_03_256"               ,
 "Cells_Texture_DifferenceEntropy_DNA_10_00_256"              , "Cells_Texture_DifferenceEntropy_DNA_10_01_256"              , "Cells_Texture_DifferenceEntropy_DNA_10_02_256"              , "Cells_Texture_DifferenceEntropy_DNA_10_03_256"              ,
 "Cells_Texture_DifferenceEntropy_DNA_3_00_256"               , "Cells_Texture_DifferenceEntropy_DNA_3_01_256"               , "Cells_Texture_DifferenceEntropy_DNA_3_02_256"               , "Cells_Texture_DifferenceEntropy_DNA_3_03_256"               ,
 "Cells_Texture_DifferenceEntropy_DNA_5_00_256"               , "Cells_Texture_DifferenceEntropy_DNA_5_01_256"               , "Cells_Texture_DifferenceEntropy_DNA_5_02_256"               , "Cells_Texture_DifferenceEntropy_DNA_5_03_256"               ,
 "Cells_Texture_DifferenceEntropy_ER_10_00_256"               , "Cells_Texture_DifferenceEntropy_ER_10_01_256"               , "Cells_Texture_DifferenceEntropy_ER_10_02_256"               , "Cells_Texture_DifferenceEntropy_ER_10_03_256"               ,
 "Cells_Texture_DifferenceEntropy_ER_3_00_256"                , "Cells_Texture_DifferenceEntropy_ER_3_01_256"                , "Cells_Texture_DifferenceEntropy_ER_3_02_256"                , "Cells_Texture_DifferenceEntropy_ER_3_03_256"                ,
 "Cells_Texture_DifferenceEntropy_ER_5_00_256"                , "Cells_Texture_DifferenceEntropy_ER_5_01_256"                , "Cells_Texture_DifferenceEntropy_ER_5_02_256"                , "Cells_Texture_DifferenceEntropy_ER_5_03_256"                ,
 "Cells_Texture_DifferenceEntropy_Mito_10_00_256"             , "Cells_Texture_DifferenceEntropy_Mito_10_01_256"             , "Cells_Texture_DifferenceEntropy_Mito_10_02_256"             , "Cells_Texture_DifferenceEntropy_Mito_10_03_256"             ,
 "Cells_Texture_DifferenceEntropy_Mito_3_00_256"              , "Cells_Texture_DifferenceEntropy_Mito_3_01_256"              , "Cells_Texture_DifferenceEntropy_Mito_3_02_256"              , "Cells_Texture_DifferenceEntropy_Mito_3_03_256"              ,
 "Cells_Texture_DifferenceEntropy_Mito_5_00_256"              , "Cells_Texture_DifferenceEntropy_Mito_5_01_256"              , "Cells_Texture_DifferenceEntropy_Mito_5_02_256"              , "Cells_Texture_DifferenceEntropy_Mito_5_03_256"              ,
 "Cells_Texture_DifferenceEntropy_RNA_10_00_256"              , "Cells_Texture_DifferenceEntropy_RNA_10_01_256"              , "Cells_Texture_DifferenceEntropy_RNA_10_02_256"              , "Cells_Texture_DifferenceEntropy_RNA_10_03_256"              ,
 "Cells_Texture_DifferenceEntropy_RNA_3_01_256"               , "Cells_Texture_DifferenceEntropy_RNA_3_02_256"               , "Cells_Texture_DifferenceEntropy_RNA_3_03_256"               , "Cells_Texture_DifferenceEntropy_RNA_5_00_256"               ,
 "Cells_Texture_DifferenceEntropy_RNA_5_01_256"               , "Cells_Texture_DifferenceEntropy_RNA_5_02_256"               , "Cells_Texture_DifferenceEntropy_RNA_5_03_256"               , "Cells_Texture_DifferenceVariance_AGP_10_00_256"             ,
 "Cells_Texture_DifferenceVariance_AGP_10_01_256"             , "Cells_Texture_DifferenceVariance_AGP_10_02_256"             , "Cells_Texture_DifferenceVariance_AGP_10_03_256"             , "Cells_Texture_DifferenceVariance_AGP_3_01_256"              ,
 "Cells_Texture_DifferenceVariance_AGP_3_02_256"              , "Cells_Texture_DifferenceVariance_AGP_3_03_256"              , "Cells_Texture_DifferenceVariance_AGP_5_00_256"              , "Cells_Texture_DifferenceVariance_AGP_5_01_256"              ,
 "Cells_Texture_DifferenceVariance_AGP_5_02_256"              , "Cells_Texture_DifferenceVariance_AGP_5_03_256"              , "Cells_Texture_DifferenceVariance_DNA_10_00_256"             , "Cells_Texture_DifferenceVariance_DNA_10_02_256"             ,
 "Cells_Texture_DifferenceVariance_DNA_10_03_256"             , "Cells_Texture_DifferenceVariance_DNA_3_00_256"              , "Cells_Texture_DifferenceVariance_DNA_3_01_256"              , "Cells_Texture_DifferenceVariance_DNA_3_02_256"              ,
 "Cells_Texture_DifferenceVariance_DNA_3_03_256"              , "Cells_Texture_DifferenceVariance_DNA_5_00_256"              , "Cells_Texture_DifferenceVariance_DNA_5_01_256"              , "Cells_Texture_DifferenceVariance_DNA_5_02_256"              ,
 "Cells_Texture_DifferenceVariance_DNA_5_03_256"              , "Cells_Texture_DifferenceVariance_ER_10_00_256"              , "Cells_Texture_DifferenceVariance_ER_10_01_256"              , "Cells_Texture_DifferenceVariance_ER_10_02_256"              ,
 "Cells_Texture_DifferenceVariance_ER_3_00_256"               , "Cells_Texture_DifferenceVariance_ER_3_01_256"               , "Cells_Texture_DifferenceVariance_ER_3_02_256"               , "Cells_Texture_DifferenceVariance_ER_3_03_256"               ,
 "Cells_Texture_DifferenceVariance_ER_5_00_256"               , "Cells_Texture_DifferenceVariance_ER_5_01_256"               , "Cells_Texture_DifferenceVariance_ER_5_02_256"               , "Cells_Texture_DifferenceVariance_ER_5_03_256"               ,
 "Cells_Texture_DifferenceVariance_Mito_10_00_256"            , "Cells_Texture_DifferenceVariance_Mito_10_01_256"            , "Cells_Texture_DifferenceVariance_Mito_10_02_256"            , "Cells_Texture_DifferenceVariance_Mito_10_03_256"            ,
 "Cells_Texture_DifferenceVariance_Mito_3_01_256"             , "Cells_Texture_DifferenceVariance_Mito_3_02_256"             , "Cells_Texture_DifferenceVariance_Mito_3_03_256"             , "Cells_Texture_DifferenceVariance_Mito_5_00_256"             ,
 "Cells_Texture_DifferenceVariance_Mito_5_01_256"             , "Cells_Texture_DifferenceVariance_Mito_5_02_256"             , "Cells_Texture_DifferenceVariance_Mito_5_03_256"             , "Cells_Texture_DifferenceVariance_RNA_10_00_256"             ,
 "Cells_Texture_DifferenceVariance_RNA_10_01_256"             , "Cells_Texture_DifferenceVariance_RNA_10_02_256"             , "Cells_Texture_DifferenceVariance_RNA_3_00_256"              , "Cells_Texture_DifferenceVariance_RNA_3_01_256"              ,
 "Cells_Texture_DifferenceVariance_RNA_3_02_256"              , "Cells_Texture_DifferenceVariance_RNA_3_03_256"              , "Cells_Texture_DifferenceVariance_RNA_5_00_256"              , "Cells_Texture_DifferenceVariance_RNA_5_01_256"              ,
 "Cells_Texture_DifferenceVariance_RNA_5_02_256"              , "Cells_Texture_DifferenceVariance_RNA_5_03_256"              , "Cells_Texture_Entropy_AGP_10_00_256"                        , "Cells_Texture_Entropy_AGP_10_01_256"                        ,
 "Cells_Texture_Entropy_AGP_10_02_256"                        , "Cells_Texture_Entropy_AGP_3_00_256"                         , "Cells_Texture_Entropy_AGP_3_01_256"                         , "Cells_Texture_Entropy_AGP_3_02_256"                         ,
 "Cells_Texture_Entropy_AGP_3_03_256"                         , "Cells_Texture_Entropy_AGP_5_00_256"                         , "Cells_Texture_Entropy_AGP_5_01_256"                         , "Cells_Texture_Entropy_AGP_5_02_256"                         ,
 "Cells_Texture_Entropy_AGP_5_03_256"                         , "Cells_Texture_Entropy_DNA_10_00_256"                        , "Cells_Texture_Entropy_DNA_10_01_256"                        , "Cells_Texture_Entropy_DNA_10_02_256"                        ,
 "Cells_Texture_Entropy_DNA_3_00_256"                         , "Cells_Texture_Entropy_DNA_3_01_256"                         , "Cells_Texture_Entropy_DNA_3_02_256"                         , "Cells_Texture_Entropy_DNA_3_03_256"                         ,
 "Cells_Texture_Entropy_DNA_5_00_256"                         , "Cells_Texture_Entropy_DNA_5_01_256"                         , "Cells_Texture_Entropy_DNA_5_02_256"                         , "Cells_Texture_Entropy_DNA_5_03_256"                         ,
 "Cells_Texture_Entropy_ER_10_00_256"                         , "Cells_Texture_Entropy_ER_10_02_256"                         , "Cells_Texture_Entropy_ER_10_03_256"                         , "Cells_Texture_Entropy_ER_3_00_256"                          ,
 "Cells_Texture_Entropy_ER_3_01_256"                          , "Cells_Texture_Entropy_ER_3_02_256"                          , "Cells_Texture_Entropy_ER_3_03_256"                          , "Cells_Texture_Entropy_ER_5_00_256"                          ,
 "Cells_Texture_Entropy_ER_5_01_256"                          , "Cells_Texture_Entropy_ER_5_02_256"                          , "Cells_Texture_Entropy_ER_5_03_256"                          , "Cells_Texture_Entropy_Mito_10_00_256"                       ,
 "Cells_Texture_Entropy_Mito_10_02_256"                       , "Cells_Texture_Entropy_Mito_10_03_256"                       , "Cells_Texture_Entropy_Mito_3_00_256"                        , "Cells_Texture_Entropy_Mito_3_01_256"                        ,
 "Cells_Texture_Entropy_Mito_3_02_256"                        , "Cells_Texture_Entropy_Mito_3_03_256"                        , "Cells_Texture_Entropy_Mito_5_00_256"                        , "Cells_Texture_Entropy_Mito_5_01_256"                        ,
 "Cells_Texture_Entropy_Mito_5_02_256"                        , "Cells_Texture_Entropy_Mito_5_03_256"                        , "Cells_Texture_Entropy_RNA_10_00_256"                        , "Cells_Texture_Entropy_RNA_10_01_256"                        ,
 "Cells_Texture_Entropy_RNA_10_02_256"                        , "Cells_Texture_Entropy_RNA_3_00_256"                         , "Cells_Texture_Entropy_RNA_3_01_256"                         , "Cells_Texture_Entropy_RNA_3_02_256"                         ,
 "Cells_Texture_Entropy_RNA_3_03_256"                         , "Cells_Texture_Entropy_RNA_5_00_256"                         , "Cells_Texture_Entropy_RNA_5_01_256"                         , "Cells_Texture_Entropy_RNA_5_02_256"                         ,
 "Cells_Texture_Entropy_RNA_5_03_256"                         , "Cells_Texture_InfoMeas1_AGP_10_01_256"                      , "Cells_Texture_InfoMeas1_AGP_5_00_256"                       , "Cells_Texture_InfoMeas1_AGP_5_01_256"                       ,
 "Cells_Texture_InfoMeas1_AGP_5_02_256"                       , "Cells_Texture_InfoMeas1_AGP_5_03_256"                       , "Cells_Texture_InfoMeas1_ER_3_00_256"                        , "Cells_Texture_InfoMeas1_ER_3_01_256"                        ,
 "Cells_Texture_InfoMeas1_ER_3_02_256"                        , "Cells_Texture_InfoMeas1_ER_3_03_256"                        , "Cells_Texture_InfoMeas1_Mito_3_01_256"                      , "Cells_Texture_InfoMeas1_Mito_3_03_256"                      ,
 "Cells_Texture_InfoMeas1_Mito_5_00_256"                      , "Cells_Texture_InfoMeas1_Mito_5_02_256"                      , "Cells_Texture_InfoMeas1_RNA_3_00_256"                       , "Cells_Texture_InfoMeas1_RNA_3_01_256"                       ,
 "Cells_Texture_InfoMeas1_RNA_3_02_256"                       , "Cells_Texture_InfoMeas1_RNA_3_03_256"                       , "Cells_Texture_InfoMeas2_AGP_10_00_256"                      , "Cells_Texture_InfoMeas2_AGP_10_01_256"                      ,
 "Cells_Texture_InfoMeas2_AGP_5_00_256"                       , "Cells_Texture_InfoMeas2_AGP_5_01_256"                       , "Cells_Texture_InfoMeas2_AGP_5_02_256"                       , "Cells_Texture_InfoMeas2_AGP_5_03_256"                       ,
 "Cells_Texture_InfoMeas2_DNA_3_00_256"                       , "Cells_Texture_InfoMeas2_DNA_3_01_256"                       , "Cells_Texture_InfoMeas2_DNA_3_03_256"                       , "Cells_Texture_InfoMeas2_DNA_5_00_256"                       ,
 "Cells_Texture_InfoMeas2_DNA_5_01_256"                       , "Cells_Texture_InfoMeas2_DNA_5_02_256"                       , "Cells_Texture_InfoMeas2_DNA_5_03_256"                       , "Cells_Texture_InfoMeas2_ER_3_00_256"                        ,
 "Cells_Texture_InfoMeas2_ER_3_01_256"                        , "Cells_Texture_InfoMeas2_ER_3_03_256"                        , "Cells_Texture_InfoMeas2_ER_5_00_256"                        , "Cells_Texture_InfoMeas2_ER_5_01_256"                        ,
 "Cells_Texture_InfoMeas2_ER_5_02_256"                        , "Cells_Texture_InfoMeas2_ER_5_03_256"                        , "Cells_Texture_InfoMeas2_Mito_10_00_256"                     , "Cells_Texture_InfoMeas2_Mito_10_01_256"                     ,
 "Cells_Texture_InfoMeas2_Mito_10_02_256"                     , "Cells_Texture_InfoMeas2_Mito_10_03_256"                     , "Cells_Texture_InfoMeas2_Mito_3_00_256"                      , "Cells_Texture_InfoMeas2_Mito_3_01_256"                      ,
 "Cells_Texture_InfoMeas2_Mito_3_03_256"                      , "Cells_Texture_InfoMeas2_Mito_5_00_256"                      , "Cells_Texture_InfoMeas2_Mito_5_01_256"                      , "Cells_Texture_InfoMeas2_Mito_5_02_256"                      ,
 "Cells_Texture_InfoMeas2_Mito_5_03_256"                      , "Cells_Texture_InfoMeas2_RNA_10_01_256"                      , "Cells_Texture_InfoMeas2_RNA_3_00_256"                       , "Cells_Texture_InfoMeas2_RNA_3_01_256"                       ,
 "Cells_Texture_InfoMeas2_RNA_3_03_256"                       , "Cells_Texture_InfoMeas2_RNA_5_00_256"                       , "Cells_Texture_InfoMeas2_RNA_5_01_256"                       , "Cells_Texture_InfoMeas2_RNA_5_02_256"                       ,
 "Cells_Texture_InfoMeas2_RNA_5_03_256"                       , "Cells_Texture_InverseDifferenceMoment_AGP_10_00_256"        , "Cells_Texture_InverseDifferenceMoment_AGP_10_01_256"        , "Cells_Texture_InverseDifferenceMoment_AGP_10_02_256"        ,
 "Cells_Texture_InverseDifferenceMoment_AGP_10_03_256"        , "Cells_Texture_InverseDifferenceMoment_AGP_3_01_256"         , "Cells_Texture_InverseDifferenceMoment_AGP_3_02_256"         , "Cells_Texture_InverseDifferenceMoment_AGP_3_03_256"         ,
 "Cells_Texture_InverseDifferenceMoment_AGP_5_00_256"         , "Cells_Texture_InverseDifferenceMoment_AGP_5_01_256"         , "Cells_Texture_InverseDifferenceMoment_AGP_5_02_256"         , "Cells_Texture_InverseDifferenceMoment_AGP_5_03_256"         ,
 "Cells_Texture_InverseDifferenceMoment_DNA_10_00_256"        , "Cells_Texture_InverseDifferenceMoment_DNA_10_01_256"        , "Cells_Texture_InverseDifferenceMoment_DNA_10_02_256"        , "Cells_Texture_InverseDifferenceMoment_DNA_3_00_256"         ,
 "Cells_Texture_InverseDifferenceMoment_DNA_3_01_256"         , "Cells_Texture_InverseDifferenceMoment_DNA_3_02_256"         , "Cells_Texture_InverseDifferenceMoment_DNA_3_03_256"         , "Cells_Texture_InverseDifferenceMoment_DNA_5_00_256"         ,
 "Cells_Texture_InverseDifferenceMoment_DNA_5_01_256"         , "Cells_Texture_InverseDifferenceMoment_DNA_5_02_256"         , "Cells_Texture_InverseDifferenceMoment_DNA_5_03_256"         , "Cells_Texture_InverseDifferenceMoment_ER_10_00_256"         ,
 "Cells_Texture_InverseDifferenceMoment_ER_10_02_256"         , "Cells_Texture_InverseDifferenceMoment_ER_10_03_256"         , "Cells_Texture_InverseDifferenceMoment_ER_3_00_256"          , "Cells_Texture_InverseDifferenceMoment_ER_3_01_256"          ,
 "Cells_Texture_InverseDifferenceMoment_ER_3_03_256"          , "Cells_Texture_InverseDifferenceMoment_ER_5_00_256"          , "Cells_Texture_InverseDifferenceMoment_ER_5_01_256"          , "Cells_Texture_InverseDifferenceMoment_ER_5_02_256"          ,
 "Cells_Texture_InverseDifferenceMoment_ER_5_03_256"          , "Cells_Texture_InverseDifferenceMoment_Mito_10_00_256"       , "Cells_Texture_InverseDifferenceMoment_Mito_10_02_256"       , "Cells_Texture_InverseDifferenceMoment_Mito_10_03_256"       ,
 "Cells_Texture_InverseDifferenceMoment_Mito_3_00_256"        , "Cells_Texture_InverseDifferenceMoment_Mito_3_01_256"        , "Cells_Texture_InverseDifferenceMoment_Mito_3_02_256"        , "Cells_Texture_InverseDifferenceMoment_Mito_3_03_256"        ,
 "Cells_Texture_InverseDifferenceMoment_Mito_5_00_256"        , "Cells_Texture_InverseDifferenceMoment_Mito_5_01_256"        , "Cells_Texture_InverseDifferenceMoment_Mito_5_02_256"        , "Cells_Texture_InverseDifferenceMoment_Mito_5_03_256"        ,
 "Cells_Texture_InverseDifferenceMoment_RNA_10_00_256"        , "Cells_Texture_InverseDifferenceMoment_RNA_10_01_256"        , "Cells_Texture_InverseDifferenceMoment_RNA_10_02_256"        , "Cells_Texture_InverseDifferenceMoment_RNA_3_00_256"         ,
 "Cells_Texture_InverseDifferenceMoment_RNA_3_01_256"         , "Cells_Texture_InverseDifferenceMoment_RNA_3_02_256"         , "Cells_Texture_InverseDifferenceMoment_RNA_3_03_256"         , "Cells_Texture_InverseDifferenceMoment_RNA_5_00_256"         ,
 "Cells_Texture_InverseDifferenceMoment_RNA_5_01_256"         , "Cells_Texture_InverseDifferenceMoment_RNA_5_02_256"         , "Cells_Texture_InverseDifferenceMoment_RNA_5_03_256"         , "Cells_Texture_SumAverage_AGP_10_00_256"                     ,
 "Cells_Texture_SumAverage_AGP_10_01_256"                     , "Cells_Texture_SumAverage_AGP_10_02_256"                     , "Cells_Texture_SumAverage_AGP_10_03_256"                     , "Cells_Texture_SumAverage_AGP_3_01_256"                      ,
 "Cells_Texture_SumAverage_AGP_3_02_256"                      , "Cells_Texture_SumAverage_AGP_3_03_256"                      , "Cells_Texture_SumAverage_AGP_5_00_256"                      , "Cells_Texture_SumAverage_AGP_5_01_256"                      ,
 "Cells_Texture_SumAverage_AGP_5_02_256"                      , "Cells_Texture_SumAverage_AGP_5_03_256"                      , "Cells_Texture_SumAverage_DNA_10_00_256"                     , "Cells_Texture_SumAverage_DNA_10_01_256"                     ,
 "Cells_Texture_SumAverage_DNA_10_02_256"                     , "Cells_Texture_SumAverage_DNA_3_00_256"                      , "Cells_Texture_SumAverage_DNA_3_01_256"                      , "Cells_Texture_SumAverage_DNA_3_02_256"                      ,
 "Cells_Texture_SumAverage_DNA_3_03_256"                      , "Cells_Texture_SumAverage_DNA_5_00_256"                      , "Cells_Texture_SumAverage_DNA_5_01_256"                      , "Cells_Texture_SumAverage_DNA_5_02_256"                      ,
 "Cells_Texture_SumAverage_DNA_5_03_256"                      , "Cells_Texture_SumAverage_ER_10_00_256"                      , "Cells_Texture_SumAverage_ER_10_01_256"                      , "Cells_Texture_SumAverage_ER_10_02_256"                      ,
 "Cells_Texture_SumAverage_ER_10_03_256"                      , "Cells_Texture_SumAverage_ER_3_01_256"                       , "Cells_Texture_SumAverage_ER_3_02_256"                       , "Cells_Texture_SumAverage_ER_3_03_256"                       ,
 "Cells_Texture_SumAverage_ER_5_00_256"                       , "Cells_Texture_SumAverage_ER_5_01_256"                       , "Cells_Texture_SumAverage_ER_5_02_256"                       , "Cells_Texture_SumAverage_ER_5_03_256"                       ,
 "Cells_Texture_SumAverage_Mito_10_00_256"                    , "Cells_Texture_SumAverage_Mito_10_01_256"                    , "Cells_Texture_SumAverage_Mito_10_02_256"                    , "Cells_Texture_SumAverage_Mito_10_03_256"                    ,
 "Cells_Texture_SumAverage_Mito_3_01_256"                     , "Cells_Texture_SumAverage_Mito_3_02_256"                     , "Cells_Texture_SumAverage_Mito_3_03_256"                     , "Cells_Texture_SumAverage_Mito_5_00_256"                     ,
 "Cells_Texture_SumAverage_Mito_5_01_256"                     , "Cells_Texture_SumAverage_Mito_5_02_256"                     , "Cells_Texture_SumAverage_Mito_5_03_256"                     , "Cells_Texture_SumAverage_RNA_10_00_256"                     ,
 "Cells_Texture_SumAverage_RNA_10_01_256"                     , "Cells_Texture_SumAverage_RNA_10_02_256"                     , "Cells_Texture_SumAverage_RNA_10_03_256"                     , "Cells_Texture_SumAverage_RNA_3_01_256"                      ,
 "Cells_Texture_SumAverage_RNA_3_02_256"                      , "Cells_Texture_SumAverage_RNA_3_03_256"                      , "Cells_Texture_SumAverage_RNA_5_00_256"                      , "Cells_Texture_SumAverage_RNA_5_01_256"                      ,
 "Cells_Texture_SumAverage_RNA_5_02_256"                      , "Cells_Texture_SumAverage_RNA_5_03_256"                      , "Cells_Texture_SumEntropy_AGP_10_00_256"                     , "Cells_Texture_SumEntropy_AGP_10_01_256"                     ,
 "Cells_Texture_SumEntropy_AGP_10_02_256"                     , "Cells_Texture_SumEntropy_AGP_10_03_256"                     , "Cells_Texture_SumEntropy_AGP_3_00_256"                      , "Cells_Texture_SumEntropy_AGP_3_01_256"                      ,
 "Cells_Texture_SumEntropy_AGP_3_02_256"                      , "Cells_Texture_SumEntropy_AGP_3_03_256"                      , "Cells_Texture_SumEntropy_AGP_5_00_256"                      , "Cells_Texture_SumEntropy_AGP_5_01_256"                      ,
 "Cells_Texture_SumEntropy_AGP_5_02_256"                      , "Cells_Texture_SumEntropy_AGP_5_03_256"                      , "Cells_Texture_SumEntropy_DNA_10_00_256"                     , "Cells_Texture_SumEntropy_DNA_10_01_256"                     ,
 "Cells_Texture_SumEntropy_DNA_10_02_256"                     , "Cells_Texture_SumEntropy_DNA_10_03_256"                     , "Cells_Texture_SumEntropy_DNA_3_00_256"                      , "Cells_Texture_SumEntropy_DNA_3_01_256"                      ,
 "Cells_Texture_SumEntropy_DNA_3_02_256"                      , "Cells_Texture_SumEntropy_DNA_3_03_256"                      , "Cells_Texture_SumEntropy_DNA_5_00_256"                      , "Cells_Texture_SumEntropy_DNA_5_01_256"                      ,
 "Cells_Texture_SumEntropy_DNA_5_02_256"                      , "Cells_Texture_SumEntropy_DNA_5_03_256"                      , "Cells_Texture_SumEntropy_ER_10_00_256"                      , "Cells_Texture_SumEntropy_ER_10_01_256"                      ,
 "Cells_Texture_SumEntropy_ER_10_02_256"                      , "Cells_Texture_SumEntropy_ER_10_03_256"                      , "Cells_Texture_SumEntropy_ER_3_00_256"                       , "Cells_Texture_SumEntropy_ER_3_01_256"                       ,
 "Cells_Texture_SumEntropy_ER_3_02_256"                       , "Cells_Texture_SumEntropy_ER_3_03_256"                       , "Cells_Texture_SumEntropy_ER_5_00_256"                       , "Cells_Texture_SumEntropy_ER_5_01_256"                       ,
 "Cells_Texture_SumEntropy_ER_5_02_256"                       , "Cells_Texture_SumEntropy_ER_5_03_256"                       , "Cells_Texture_SumEntropy_Mito_10_00_256"                    , "Cells_Texture_SumEntropy_Mito_10_01_256"                    ,
 "Cells_Texture_SumEntropy_Mito_10_02_256"                    , "Cells_Texture_SumEntropy_Mito_10_03_256"                    , "Cells_Texture_SumEntropy_Mito_3_00_256"                     , "Cells_Texture_SumEntropy_Mito_3_01_256"                     ,
 "Cells_Texture_SumEntropy_Mito_3_02_256"                     , "Cells_Texture_SumEntropy_Mito_3_03_256"                     , "Cells_Texture_SumEntropy_Mito_5_00_256"                     , "Cells_Texture_SumEntropy_Mito_5_01_256"                     ,
 "Cells_Texture_SumEntropy_Mito_5_02_256"                     , "Cells_Texture_SumEntropy_Mito_5_03_256"                     , "Cells_Texture_SumEntropy_RNA_10_00_256"                     , "Cells_Texture_SumEntropy_RNA_10_01_256"                     ,
 "Cells_Texture_SumEntropy_RNA_10_02_256"                     , "Cells_Texture_SumEntropy_RNA_10_03_256"                     , "Cells_Texture_SumEntropy_RNA_3_00_256"                      , "Cells_Texture_SumEntropy_RNA_3_01_256"                      ,
 "Cells_Texture_SumEntropy_RNA_3_02_256"                      , "Cells_Texture_SumEntropy_RNA_3_03_256"                      , "Cells_Texture_SumEntropy_RNA_5_00_256"                      , "Cells_Texture_SumEntropy_RNA_5_01_256"                      ,
 "Cells_Texture_SumEntropy_RNA_5_02_256"                      , "Cells_Texture_SumEntropy_RNA_5_03_256"                      , "Cells_Texture_SumVariance_AGP_10_00_256"                    , "Cells_Texture_SumVariance_AGP_10_01_256"                    ,
 "Cells_Texture_SumVariance_AGP_10_02_256"                    , "Cells_Texture_SumVariance_AGP_3_00_256"                     , "Cells_Texture_SumVariance_AGP_3_01_256"                     , "Cells_Texture_SumVariance_AGP_3_02_256"                     ,
 "Cells_Texture_SumVariance_AGP_3_03_256"                     , "Cells_Texture_SumVariance_AGP_5_00_256"                     , "Cells_Texture_SumVariance_AGP_5_01_256"                     , "Cells_Texture_SumVariance_AGP_5_02_256"                     ,
 "Cells_Texture_SumVariance_AGP_5_03_256"                     , "Cells_Texture_SumVariance_DNA_10_00_256"                    , "Cells_Texture_SumVariance_DNA_10_02_256"                    , "Cells_Texture_SumVariance_DNA_10_03_256"                    ,
 "Cells_Texture_SumVariance_DNA_3_00_256"                     , "Cells_Texture_SumVariance_DNA_3_01_256"                     , "Cells_Texture_SumVariance_DNA_3_02_256"                     , "Cells_Texture_SumVariance_DNA_3_03_256"                     ,
 "Cells_Texture_SumVariance_DNA_5_00_256"                     , "Cells_Texture_SumVariance_DNA_5_01_256"                     , "Cells_Texture_SumVariance_DNA_5_02_256"                     , "Cells_Texture_SumVariance_DNA_5_03_256"                     ,
 "Cells_Texture_SumVariance_ER_10_00_256"                     , "Cells_Texture_SumVariance_ER_10_02_256"                     , "Cells_Texture_SumVariance_ER_10_03_256"                     , "Cells_Texture_SumVariance_ER_3_00_256"                      ,
 "Cells_Texture_SumVariance_ER_3_01_256"                      , "Cells_Texture_SumVariance_ER_3_02_256"                      , "Cells_Texture_SumVariance_ER_3_03_256"                      , "Cells_Texture_SumVariance_ER_5_00_256"                      ,
 "Cells_Texture_SumVariance_ER_5_01_256"                      , "Cells_Texture_SumVariance_ER_5_02_256"                      , "Cells_Texture_SumVariance_ER_5_03_256"                      , "Cells_Texture_SumVariance_Mito_10_00_256"                   ,
 "Cells_Texture_SumVariance_Mito_10_01_256"                   , "Cells_Texture_SumVariance_Mito_10_02_256"                   , "Cells_Texture_SumVariance_Mito_3_00_256"                    , "Cells_Texture_SumVariance_Mito_3_01_256"                    ,
 "Cells_Texture_SumVariance_Mito_3_02_256"                    , "Cells_Texture_SumVariance_Mito_3_03_256"                    , "Cells_Texture_SumVariance_Mito_5_00_256"                    , "Cells_Texture_SumVariance_Mito_5_01_256"                    ,
 "Cells_Texture_SumVariance_Mito_5_02_256"                    , "Cells_Texture_SumVariance_Mito_5_03_256"                    , "Cells_Texture_SumVariance_RNA_10_00_256"                    , "Cells_Texture_SumVariance_RNA_10_01_256"                    ,
 "Cells_Texture_SumVariance_RNA_10_02_256"                    , "Cells_Texture_SumVariance_RNA_3_00_256"                     , "Cells_Texture_SumVariance_RNA_3_01_256"                     , "Cells_Texture_SumVariance_RNA_3_02_256"                     ,
 "Cells_Texture_SumVariance_RNA_3_03_256"                     , "Cells_Texture_SumVariance_RNA_5_00_256"                     , "Cells_Texture_SumVariance_RNA_5_01_256"                     , "Cells_Texture_SumVariance_RNA_5_02_256"                     ,
 "Cells_Texture_SumVariance_RNA_5_03_256"                     , "Cells_Texture_Variance_AGP_10_00_256"                       , "Cells_Texture_Variance_AGP_10_01_256"                       , "Cells_Texture_Variance_AGP_10_02_256"                       ,
 "Cells_Texture_Variance_AGP_10_03_256"                       , "Cells_Texture_Variance_AGP_3_00_256"                        , "Cells_Texture_Variance_AGP_3_01_256"                        , "Cells_Texture_Variance_AGP_3_02_256"                        ,
 "Cells_Texture_Variance_AGP_3_03_256"                        , "Cells_Texture_Variance_AGP_5_00_256"                        , "Cells_Texture_Variance_AGP_5_01_256"                        , "Cells_Texture_Variance_AGP_5_02_256"                        ,
 "Cells_Texture_Variance_AGP_5_03_256"                        , "Cells_Texture_Variance_DNA_10_00_256"                       , "Cells_Texture_Variance_DNA_10_01_256"                       , "Cells_Texture_Variance_DNA_10_02_256"                       ,
 "Cells_Texture_Variance_DNA_10_03_256"                       , "Cells_Texture_Variance_DNA_3_00_256"                        , "Cells_Texture_Variance_DNA_3_01_256"                        , "Cells_Texture_Variance_DNA_3_02_256"                        ,
 "Cells_Texture_Variance_DNA_3_03_256"                        , "Cells_Texture_Variance_DNA_5_00_256"                        , "Cells_Texture_Variance_DNA_5_01_256"                        , "Cells_Texture_Variance_DNA_5_02_256"                        ,
 "Cells_Texture_Variance_DNA_5_03_256"                        , "Cells_Texture_Variance_ER_10_00_256"                        , "Cells_Texture_Variance_ER_10_01_256"                        , "Cells_Texture_Variance_ER_10_02_256"                        ,
 "Cells_Texture_Variance_ER_10_03_256"                        , "Cells_Texture_Variance_ER_3_00_256"                         , "Cells_Texture_Variance_ER_3_01_256"                         , "Cells_Texture_Variance_ER_3_02_256"                         ,
 "Cells_Texture_Variance_ER_3_03_256"                         , "Cells_Texture_Variance_ER_5_00_256"                         , "Cells_Texture_Variance_ER_5_01_256"                         , "Cells_Texture_Variance_ER_5_02_256"                         ,
 "Cells_Texture_Variance_ER_5_03_256"                         , "Cells_Texture_Variance_Mito_10_00_256"                      , "Cells_Texture_Variance_Mito_10_01_256"                      , "Cells_Texture_Variance_Mito_10_02_256"                      ,
 "Cells_Texture_Variance_Mito_10_03_256"                      , "Cells_Texture_Variance_Mito_3_00_256"                       , "Cells_Texture_Variance_Mito_3_01_256"                       , "Cells_Texture_Variance_Mito_3_02_256"                       ,
 "Cells_Texture_Variance_Mito_3_03_256"                       , "Cells_Texture_Variance_Mito_5_00_256"                       , "Cells_Texture_Variance_Mito_5_01_256"                       , "Cells_Texture_Variance_Mito_5_02_256"                       ,
 "Cells_Texture_Variance_Mito_5_03_256"                       , "Cells_Texture_Variance_RNA_10_00_256"                       , "Cells_Texture_Variance_RNA_10_01_256"                       , "Cells_Texture_Variance_RNA_10_02_256"                       ,
 "Cells_Texture_Variance_RNA_10_03_256"                       , "Cells_Texture_Variance_RNA_3_00_256"                        , "Cells_Texture_Variance_RNA_3_01_256"                        , "Cells_Texture_Variance_RNA_3_02_256"                        ,
 "Cells_Texture_Variance_RNA_3_03_256"                        , "Cells_Texture_Variance_RNA_5_00_256"                        , "Cells_Texture_Variance_RNA_5_01_256"                        , "Cells_Texture_Variance_RNA_5_02_256"                        ,
 "Cells_Texture_Variance_RNA_5_03_256"                        }

variance_dropped_cols_2 = set()

In [276]:
variance_dropped_cols_2 & correlation_dropped_cols_2
print(f" Number of cols dropped - high correlation: {len(correlation_dropped_cols_2)}")
print(f" Number of cols dropped - low variance    : {len(variance_dropped_cols_2)}")
print(f" Number of selected cols                  : {len(selected_cols_2)}")
print(f" Total                                    : {len(selected_cols_2)+len(correlation_dropped_cols_2)+len(variance_dropped_cols_2)}")

set()

 Number of cols dropped - high correlation: 645
 Number of cols dropped - low variance    : 0
 Number of selected cols                  : 135
 Total                                    : 780


In [277]:
(correlation_dropped_cols_2) & set(zero_features)
(variance_dropped_cols_2) & set(zero_features)
(selected_cols_2) & set(zero_features)

set()

set()

set()

### Group 3 - Cytoplasm

In [278]:
selected_cols_3 = \
{'Cytoplasm_AreaShape_BoundingBoxMaximum_X', 'Cytoplasm_AreaShape_Center_X', 'Cytoplasm_AreaShape_Center_Y', 'Cytoplasm_AreaShape_Compactness', 'Cytoplasm_AreaShape_Eccentricity', 
 'Cytoplasm_AreaShape_EulerNumber', 'Cytoplasm_AreaShape_Extent', 'Cytoplasm_AreaShape_MaximumRadius', 'Cytoplasm_AreaShape_MinorAxisLength', 'Cytoplasm_AreaShape_Orientation', 
 'Cytoplasm_AreaShape_Perimeter', 'Cytoplasm_AreaShape_Solidity', 'Cytoplasm_AreaShape_Zernike_0_0', 'Cytoplasm_AreaShape_Zernike_1_1', 'Cytoplasm_AreaShape_Zernike_2_0', 
 'Cytoplasm_AreaShape_Zernike_2_2', 'Cytoplasm_AreaShape_Zernike_3_1', 'Cytoplasm_AreaShape_Zernike_3_3', 'Cytoplasm_AreaShape_Zernike_4_0', 'Cytoplasm_AreaShape_Zernike_4_2', 
 'Cytoplasm_AreaShape_Zernike_4_4', 'Cytoplasm_AreaShape_Zernike_5_1', 'Cytoplasm_AreaShape_Zernike_5_3', 'Cytoplasm_AreaShape_Zernike_5_5', 'Cytoplasm_AreaShape_Zernike_6_0', 
 'Cytoplasm_AreaShape_Zernike_6_2', 'Cytoplasm_AreaShape_Zernike_6_4', 'Cytoplasm_AreaShape_Zernike_6_6', 'Cytoplasm_AreaShape_Zernike_7_1', 'Cytoplasm_AreaShape_Zernike_7_3', 
 'Cytoplasm_AreaShape_Zernike_7_5', 'Cytoplasm_AreaShape_Zernike_7_7', 'Cytoplasm_AreaShape_Zernike_8_0', 'Cytoplasm_AreaShape_Zernike_8_2', 'Cytoplasm_AreaShape_Zernike_8_4', 
 'Cytoplasm_AreaShape_Zernike_8_6', 'Cytoplasm_AreaShape_Zernike_8_8', 'Cytoplasm_AreaShape_Zernike_9_1', 'Cytoplasm_AreaShape_Zernike_9_3', 'Cytoplasm_AreaShape_Zernike_9_5', 
 'Cytoplasm_AreaShape_Zernike_9_7', 'Cytoplasm_AreaShape_Zernike_9_9', 'Cytoplasm_Correlation_Correlation_AGP_DNA', 'Cytoplasm_Correlation_Correlation_AGP_ER', 'Cytoplasm_Correlation_Correlation_AGP_Mito', 
 'Cytoplasm_Correlation_Correlation_DNA_ER', 'Cytoplasm_Correlation_Correlation_DNA_Mito', 'Cytoplasm_Correlation_Correlation_DNA_RNA', 'Cytoplasm_Correlation_Correlation_ER_Mito', 'Cytoplasm_Correlation_Correlation_ER_RNA', 
 'Cytoplasm_Correlation_Correlation_Mito_RNA', 'Cytoplasm_Correlation_K_AGP_DNA', 'Cytoplasm_Correlation_K_AGP_ER', 'Cytoplasm_Correlation_K_AGP_Mito', 'Cytoplasm_Correlation_K_AGP_RNA', 
 'Cytoplasm_Correlation_K_DNA_AGP',  'Cytoplasm_Correlation_K_DNA_ER', 'Cytoplasm_Correlation_K_DNA_Mito', 'Cytoplasm_Correlation_K_DNA_RNA', 'Cytoplasm_Correlation_K_ER_AGP', 
 'Cytoplasm_Correlation_K_ER_DNA', 'Cytoplasm_Correlation_K_ER_Mito', 'Cytoplasm_Correlation_K_ER_RNA', 'Cytoplasm_Correlation_K_Mito_AGP', 'Cytoplasm_Correlation_K_Mito_DNA',
 'Cytoplasm_Correlation_K_Mito_ER', 'Cytoplasm_Correlation_K_Mito_RNA', 'Cytoplasm_Correlation_K_RNA_AGP', 'Cytoplasm_Correlation_K_RNA_DNA', 'Cytoplasm_Correlation_K_RNA_ER', 
 'Cytoplasm_Correlation_K_RNA_Mito', 'Cytoplasm_Correlation_Manders_AGP_ER', 'Cytoplasm_Correlation_Manders_AGP_Mito', 'Cytoplasm_Correlation_Manders_AGP_RNA', 'Cytoplasm_Correlation_Manders_DNA_AGP', 
 'Cytoplasm_Correlation_Manders_DNA_ER', 'Cytoplasm_Correlation_Manders_DNA_Mito', 'Cytoplasm_Correlation_Manders_DNA_RNA', 'Cytoplasm_Correlation_Manders_ER_DNA', 'Cytoplasm_Correlation_Manders_ER_Mito', 
 'Cytoplasm_Correlation_Manders_ER_RNA', 'Cytoplasm_Correlation_Manders_Mito_AGP', 'Cytoplasm_Correlation_Manders_Mito_DNA', 'Cytoplasm_Correlation_Manders_Mito_ER', 'Cytoplasm_Correlation_Manders_Mito_RNA', 
 'Cytoplasm_Correlation_Manders_RNA_AGP', 'Cytoplasm_Correlation_Manders_RNA_DNA', 'Cytoplasm_Correlation_Manders_RNA_ER', 'Cytoplasm_Correlation_Manders_RNA_Mito', 'Cytoplasm_Correlation_Overlap_AGP_DNA', 
 'Cytoplasm_Correlation_Overlap_AGP_ER', 'Cytoplasm_Correlation_Overlap_AGP_Mito', 'Cytoplasm_Correlation_Overlap_AGP_RNA', 'Cytoplasm_Correlation_Overlap_DNA_ER', 'Cytoplasm_Correlation_Overlap_DNA_Mito', 
 'Cytoplasm_Correlation_Overlap_DNA_RNA', 'Cytoplasm_Correlation_Overlap_ER_Mito', 'Cytoplasm_Correlation_Overlap_ER_RNA', 'Cytoplasm_Correlation_Overlap_Mito_RNA', 'Cytoplasm_Correlation_RWC_AGP_DNA', 
 'Cytoplasm_Correlation_RWC_AGP_ER', 'Cytoplasm_Correlation_RWC_AGP_Mito', 'Cytoplasm_Correlation_RWC_AGP_RNA', 'Cytoplasm_Correlation_RWC_DNA_AGP', 'Cytoplasm_Correlation_RWC_DNA_ER', 
 'Cytoplasm_Correlation_RWC_DNA_Mito', 'Cytoplasm_Correlation_RWC_DNA_RNA', 'Cytoplasm_Correlation_RWC_ER_AGP', 'Cytoplasm_Correlation_RWC_ER_DNA', 'Cytoplasm_Correlation_RWC_ER_Mito', 
 'Cytoplasm_Correlation_RWC_ER_RNA', 'Cytoplasm_Correlation_RWC_Mito_AGP', 'Cytoplasm_Correlation_RWC_Mito_DNA', 'Cytoplasm_Correlation_RWC_Mito_ER', 'Cytoplasm_Correlation_RWC_Mito_RNA', 
 'Cytoplasm_Correlation_RWC_RNA_AGP', 'Cytoplasm_Correlation_RWC_RNA_DNA', 'Cytoplasm_Correlation_RWC_RNA_ER', 'Cytoplasm_Correlation_RWC_RNA_Mito', 'Cytoplasm_Granularity_10_AGP', 
 'Cytoplasm_Granularity_10_DNA', 'Cytoplasm_Granularity_10_ER', 'Cytoplasm_Granularity_10_Mito', 'Cytoplasm_Granularity_10_RNA', 'Cytoplasm_Granularity_11_AGP', 'Cytoplasm_Granularity_11_DNA', 
 'Cytoplasm_Granularity_11_ER', 'Cytoplasm_Granularity_11_Mito', 'Cytoplasm_Granularity_11_RNA', 'Cytoplasm_Granularity_12_AGP', 'Cytoplasm_Granularity_12_DNA', 'Cytoplasm_Granularity_12_ER', 
 'Cytoplasm_Granularity_12_Mito', 'Cytoplasm_Granularity_12_RNA', 'Cytoplasm_Granularity_13_AGP', 'Cytoplasm_Granularity_13_DNA', 'Cytoplasm_Granularity_13_ER', 'Cytoplasm_Granularity_13_Mito', 
 'Cytoplasm_Granularity_13_RNA', 'Cytoplasm_Granularity_14_AGP', 'Cytoplasm_Granularity_14_DNA', 'Cytoplasm_Granularity_14_ER', 'Cytoplasm_Granularity_14_Mito', 'Cytoplasm_Granularity_14_RNA', 
 'Cytoplasm_Granularity_15_AGP', 'Cytoplasm_Granularity_15_DNA', 'Cytoplasm_Granularity_15_ER', 'Cytoplasm_Granularity_15_Mito', 'Cytoplasm_Granularity_15_RNA', 'Cytoplasm_Granularity_16_AGP', 
 'Cytoplasm_Granularity_16_DNA', 'Cytoplasm_Granularity_16_ER', 'Cytoplasm_Granularity_16_Mito', 'Cytoplasm_Granularity_16_RNA', 'Cytoplasm_Granularity_1_AGP', 'Cytoplasm_Granularity_1_DNA', 
 'Cytoplasm_Granularity_1_Mito', 'Cytoplasm_Granularity_1_RNA', 'Cytoplasm_Granularity_2_AGP', 'Cytoplasm_Granularity_2_DNA', 'Cytoplasm_Granularity_2_ER', 'Cytoplasm_Granularity_2_Mito', 
 'Cytoplasm_Granularity_2_RNA', 'Cytoplasm_Granularity_3_AGP', 'Cytoplasm_Granularity_3_DNA', 'Cytoplasm_Granularity_3_ER', 'Cytoplasm_Granularity_3_Mito', 'Cytoplasm_Granularity_3_RNA', 
 'Cytoplasm_Granularity_4_AGP', 'Cytoplasm_Granularity_4_DNA', 'Cytoplasm_Granularity_4_ER', 'Cytoplasm_Granularity_4_Mito', 'Cytoplasm_Granularity_4_RNA', 'Cytoplasm_Granularity_5_AGP',
 'Cytoplasm_Granularity_5_DNA', 'Cytoplasm_Granularity_5_ER', 'Cytoplasm_Granularity_5_Mito', 'Cytoplasm_Granularity_5_RNA', 'Cytoplasm_Granularity_6_AGP', 'Cytoplasm_Granularity_6_DNA',
 'Cytoplasm_Granularity_6_ER', 'Cytoplasm_Granularity_6_Mito', 'Cytoplasm_Granularity_6_RNA', 'Cytoplasm_Granularity_7_AGP', 'Cytoplasm_Granularity_7_DNA', 'Cytoplasm_Granularity_7_ER', 
 'Cytoplasm_Granularity_7_Mito', 'Cytoplasm_Granularity_7_RNA', 'Cytoplasm_Granularity_8_AGP', 'Cytoplasm_Granularity_8_DNA', 'Cytoplasm_Granularity_8_ER', 'Cytoplasm_Granularity_8_Mito', 
 'Cytoplasm_Granularity_8_RNA', 'Cytoplasm_Granularity_9_AGP', 'Cytoplasm_Granularity_9_DNA', 'Cytoplasm_Granularity_9_ER', 'Cytoplasm_Granularity_9_Mito', 'Cytoplasm_Granularity_9_RNA', 
 'Cytoplasm_Intensity_IntegratedIntensityEdge_AGP', 'Cytoplasm_Intensity_IntegratedIntensityEdge_DNA', 'Cytoplasm_Intensity_IntegratedIntensityEdge_ER', 'Cytoplasm_Intensity_IntegratedIntensityEdge_Mito', 
 'Cytoplasm_Intensity_IntegratedIntensityEdge_RNA', 'Cytoplasm_Intensity_IntegratedIntensity_AGP', 'Cytoplasm_Intensity_IntegratedIntensity_DNA', 'Cytoplasm_Intensity_IntegratedIntensity_ER', 
 'Cytoplasm_Intensity_IntegratedIntensity_Mito', 'Cytoplasm_Intensity_IntegratedIntensity_RNA', 'Cytoplasm_Intensity_LowerQuartileIntensity_DNA', 'Cytoplasm_Intensity_LowerQuartileIntensity_ER', 
 'Cytoplasm_Intensity_LowerQuartileIntensity_Mito', 'Cytoplasm_Intensity_LowerQuartileIntensity_RNA', 'Cytoplasm_Intensity_MADIntensity_AGP', 'Cytoplasm_Intensity_MADIntensity_DNA', 'Cytoplasm_Intensity_MADIntensity_ER', 
 'Cytoplasm_Intensity_MADIntensity_Mito', 'Cytoplasm_Intensity_MADIntensity_RNA', 'Cytoplasm_Intensity_MassDisplacement_AGP', 'Cytoplasm_Intensity_MassDisplacement_DNA', 'Cytoplasm_Intensity_MassDisplacement_ER', 
 'Cytoplasm_Intensity_MassDisplacement_Mito', 'Cytoplasm_Intensity_MassDisplacement_RNA', 'Cytoplasm_Intensity_MaxIntensityEdge_AGP', 'Cytoplasm_Intensity_MaxIntensityEdge_DNA', 'Cytoplasm_Intensity_MaxIntensityEdge_ER', 
 'Cytoplasm_Intensity_MaxIntensityEdge_RNA', 'Cytoplasm_Intensity_MaxIntensity_ER', 'Cytoplasm_Intensity_MaxIntensity_Mito', 'Cytoplasm_Intensity_MeanIntensityEdge_DNA', 'Cytoplasm_Intensity_MeanIntensityEdge_ER', 
 'Cytoplasm_Intensity_MeanIntensityEdge_Mito', 'Cytoplasm_Intensity_MeanIntensity_ER', 'Cytoplasm_Intensity_MeanIntensity_RNA', 'Cytoplasm_Intensity_MedianIntensity_ER', 'Cytoplasm_Intensity_MinIntensityEdge_DNA', 
 'Cytoplasm_Intensity_MinIntensityEdge_ER', 'Cytoplasm_Intensity_MinIntensityEdge_Mito', 'Cytoplasm_Intensity_MinIntensityEdge_RNA', 'Cytoplasm_Intensity_MinIntensity_AGP', 'Cytoplasm_Intensity_MinIntensity_DNA', 
 'Cytoplasm_Intensity_StdIntensityEdge_DNA', 'Cytoplasm_Intensity_StdIntensityEdge_ER', 'Cytoplasm_Intensity_StdIntensity_AGP', 'Cytoplasm_Intensity_StdIntensity_DNA', 'Cytoplasm_Intensity_StdIntensity_ER',
 'Cytoplasm_Intensity_StdIntensity_RNA', 'Cytoplasm_Intensity_UpperQuartileIntensity_AGP', 'Cytoplasm_Intensity_UpperQuartileIntensity_DNA', 'Cytoplasm_Number_Object_Number', 
 'Cytoplasm_RadialDistribution_FracAtD_AGP_2of4', 'Cytoplasm_RadialDistribution_FracAtD_AGP_3of4', 'Cytoplasm_RadialDistribution_FracAtD_AGP_4of4', 'Cytoplasm_RadialDistribution_FracAtD_DNA_1of4',
 'Cytoplasm_RadialDistribution_FracAtD_DNA_4of4', 'Cytoplasm_RadialDistribution_FracAtD_ER_2of4', 'Cytoplasm_RadialDistribution_FracAtD_ER_3of4', 'Cytoplasm_RadialDistribution_FracAtD_ER_4of4', 
 'Cytoplasm_RadialDistribution_FracAtD_Mito_2of4', 'Cytoplasm_RadialDistribution_FracAtD_Mito_3of4', 'Cytoplasm_RadialDistribution_FracAtD_Mito_4of4', 'Cytoplasm_RadialDistribution_FracAtD_mito_tubeness_10of16',
 'Cytoplasm_RadialDistribution_FracAtD_mito_tubeness_10of20', 'Cytoplasm_RadialDistribution_FracAtD_mito_tubeness_11of20', 'Cytoplasm_RadialDistribution_FracAtD_mito_tubeness_12of20', 
 'Cytoplasm_RadialDistribution_FracAtD_mito_tubeness_13of16', 'Cytoplasm_RadialDistribution_FracAtD_mito_tubeness_13of20', 'Cytoplasm_RadialDistribution_FracAtD_mito_tubeness_14of20', 
 'Cytoplasm_RadialDistribution_FracAtD_mito_tubeness_15of20', 'Cytoplasm_RadialDistribution_FracAtD_mito_tubeness_17of20', 'Cytoplasm_RadialDistribution_FracAtD_mito_tubeness_19of20', 
 'Cytoplasm_RadialDistribution_FracAtD_mito_tubeness_1of20', 'Cytoplasm_RadialDistribution_FracAtD_mito_tubeness_20of20', 'Cytoplasm_RadialDistribution_FracAtD_mito_tubeness_2of16', 
 'Cytoplasm_RadialDistribution_FracAtD_mito_tubeness_2of20', 'Cytoplasm_RadialDistribution_FracAtD_mito_tubeness_3of16', 'Cytoplasm_RadialDistribution_FracAtD_mito_tubeness_4of20', 
 'Cytoplasm_RadialDistribution_FracAtD_mito_tubeness_5of16', 'Cytoplasm_RadialDistribution_FracAtD_mito_tubeness_5of20', 'Cytoplasm_RadialDistribution_FracAtD_mito_tubeness_6of16', 
 'Cytoplasm_RadialDistribution_FracAtD_mito_tubeness_6of20', 'Cytoplasm_RadialDistribution_FracAtD_mito_tubeness_7of20', 'Cytoplasm_RadialDistribution_FracAtD_mito_tubeness_8of20', 
 'Cytoplasm_RadialDistribution_FracAtD_mito_tubeness_9of16', 'Cytoplasm_RadialDistribution_FracAtD_mito_tubeness_9of20', 'Cytoplasm_RadialDistribution_FracAtD_mito_tubeness_Overflow',
 'Cytoplasm_RadialDistribution_MeanFrac_AGP_1of4', 'Cytoplasm_RadialDistribution_MeanFrac_AGP_3of4', 'Cytoplasm_RadialDistribution_MeanFrac_AGP_4of4', 'Cytoplasm_RadialDistribution_MeanFrac_DNA_1of4', 
 'Cytoplasm_RadialDistribution_MeanFrac_DNA_3of4', 'Cytoplasm_RadialDistribution_MeanFrac_DNA_4of4', 'Cytoplasm_RadialDistribution_MeanFrac_ER_2of4', 'Cytoplasm_RadialDistribution_MeanFrac_ER_3of4', 
 'Cytoplasm_RadialDistribution_MeanFrac_ER_4of4', 'Cytoplasm_RadialDistribution_MeanFrac_Mito_2of4', 'Cytoplasm_RadialDistribution_MeanFrac_Mito_3of4', 'Cytoplasm_RadialDistribution_MeanFrac_Mito_4of4', 
 'Cytoplasm_RadialDistribution_MeanFrac_RNA_2of4', 'Cytoplasm_RadialDistribution_MeanFrac_RNA_3of4', 'Cytoplasm_RadialDistribution_MeanFrac_RNA_4of4', 'Cytoplasm_RadialDistribution_MeanFrac_mito_tubeness_10of16', 
 'Cytoplasm_RadialDistribution_MeanFrac_mito_tubeness_10of20', 'Cytoplasm_RadialDistribution_MeanFrac_mito_tubeness_11of20', 'Cytoplasm_RadialDistribution_MeanFrac_mito_tubeness_12of16', 
 'Cytoplasm_RadialDistribution_MeanFrac_mito_tubeness_12of20', 'Cytoplasm_RadialDistribution_MeanFrac_mito_tubeness_13of20', 'Cytoplasm_RadialDistribution_MeanFrac_mito_tubeness_14of16', 
 'Cytoplasm_RadialDistribution_MeanFrac_mito_tubeness_14of20', 'Cytoplasm_RadialDistribution_MeanFrac_mito_tubeness_1of20', 'Cytoplasm_RadialDistribution_MeanFrac_mito_tubeness_20of20', 
 'Cytoplasm_RadialDistribution_MeanFrac_mito_tubeness_3of16', 'Cytoplasm_RadialDistribution_MeanFrac_mito_tubeness_3of20', 'Cytoplasm_RadialDistribution_MeanFrac_mito_tubeness_4of20', 
 'Cytoplasm_RadialDistribution_MeanFrac_mito_tubeness_5of16', 'Cytoplasm_RadialDistribution_MeanFrac_mito_tubeness_5of20', 'Cytoplasm_RadialDistribution_MeanFrac_mito_tubeness_6of16', 
 'Cytoplasm_RadialDistribution_MeanFrac_mito_tubeness_6of20', 'Cytoplasm_RadialDistribution_MeanFrac_mito_tubeness_7of16', 'Cytoplasm_RadialDistribution_MeanFrac_mito_tubeness_8of20', 
 'Cytoplasm_RadialDistribution_MeanFrac_mito_tubeness_9of16', 'Cytoplasm_RadialDistribution_MeanFrac_mito_tubeness_Overflow', 'Cytoplasm_RadialDistribution_RadialCV_AGP_1of4', 
 'Cytoplasm_RadialDistribution_RadialCV_AGP_2of4', 'Cytoplasm_RadialDistribution_RadialCV_AGP_3of4', 'Cytoplasm_RadialDistribution_RadialCV_AGP_4of4', 'Cytoplasm_RadialDistribution_RadialCV_DNA_1of4', 
 'Cytoplasm_RadialDistribution_RadialCV_DNA_2of4', 'Cytoplasm_RadialDistribution_RadialCV_DNA_3of4', 'Cytoplasm_RadialDistribution_RadialCV_DNA_4of4', 'Cytoplasm_RadialDistribution_RadialCV_ER_1of4', 
 'Cytoplasm_RadialDistribution_RadialCV_ER_2of4', 'Cytoplasm_RadialDistribution_RadialCV_ER_4of4', 'Cytoplasm_RadialDistribution_RadialCV_Mito_1of4', 'Cytoplasm_RadialDistribution_RadialCV_Mito_2of4',
 'Cytoplasm_RadialDistribution_RadialCV_Mito_4of4', 'Cytoplasm_RadialDistribution_RadialCV_RNA_1of4', 'Cytoplasm_RadialDistribution_RadialCV_RNA_2of4', 'Cytoplasm_RadialDistribution_RadialCV_RNA_4of4', 
 'Cytoplasm_RadialDistribution_RadialCV_mito_tubeness_10of16', 'Cytoplasm_RadialDistribution_RadialCV_mito_tubeness_10of20', 'Cytoplasm_RadialDistribution_RadialCV_mito_tubeness_11of20', 
 'Cytoplasm_RadialDistribution_RadialCV_mito_tubeness_12of16', 'Cytoplasm_RadialDistribution_RadialCV_mito_tubeness_12of20', 'Cytoplasm_RadialDistribution_RadialCV_mito_tubeness_13of16', 
 'Cytoplasm_RadialDistribution_RadialCV_mito_tubeness_13of20', 'Cytoplasm_RadialDistribution_RadialCV_mito_tubeness_14of20', 'Cytoplasm_RadialDistribution_RadialCV_mito_tubeness_1of16', 
 'Cytoplasm_RadialDistribution_RadialCV_mito_tubeness_1of20', 'Cytoplasm_RadialDistribution_RadialCV_mito_tubeness_20of20', 'Cytoplasm_RadialDistribution_RadialCV_mito_tubeness_2of16', 
 'Cytoplasm_RadialDistribution_RadialCV_mito_tubeness_2of20', 'Cytoplasm_RadialDistribution_RadialCV_mito_tubeness_3of16', 'Cytoplasm_RadialDistribution_RadialCV_mito_tubeness_3of20', 
 'Cytoplasm_RadialDistribution_RadialCV_mito_tubeness_4of16', 'Cytoplasm_RadialDistribution_RadialCV_mito_tubeness_4of20', 'Cytoplasm_RadialDistribution_RadialCV_mito_tubeness_5of16', 
 'Cytoplasm_RadialDistribution_RadialCV_mito_tubeness_5of20', 'Cytoplasm_RadialDistribution_RadialCV_mito_tubeness_6of16', 'Cytoplasm_RadialDistribution_RadialCV_mito_tubeness_6of20', 
 'Cytoplasm_RadialDistribution_RadialCV_mito_tubeness_7of16', 'Cytoplasm_RadialDistribution_RadialCV_mito_tubeness_7of20', 'Cytoplasm_RadialDistribution_RadialCV_mito_tubeness_8of20',
 'Cytoplasm_RadialDistribution_RadialCV_mito_tubeness_9of16', 'Cytoplasm_RadialDistribution_RadialCV_mito_tubeness_9of20', 'Cytoplasm_RadialDistribution_RadialCV_mito_tubeness_Overflow'

 }
correlation_dropped_cols_3 = \
{'Cytoplasm_AreaShape_Area', 'Cytoplasm_AreaShape_BoundingBoxArea', 'Cytoplasm_AreaShape_BoundingBoxMaximum_Y', 'Cytoplasm_AreaShape_BoundingBoxMinimum_X', 'Cytoplasm_AreaShape_BoundingBoxMinimum_Y', 
 'Cytoplasm_AreaShape_EquivalentDiameter', 'Cytoplasm_AreaShape_MajorAxisLength', 'Cytoplasm_AreaShape_MaxFeretDiameter', 'Cytoplasm_AreaShape_MeanRadius', 'Cytoplasm_AreaShape_MedianRadius', 
 'Cytoplasm_AreaShape_MinFeretDiameter', 'Cytoplasm_Correlation_Correlation_AGP_RNA', 'Cytoplasm_Correlation_Manders_AGP_DNA', 'Cytoplasm_Correlation_Manders_ER_AGP', 'Cytoplasm_Granularity_1_ER', 
 'Cytoplasm_Intensity_LowerQuartileIntensity_AGP', 'Cytoplasm_Intensity_MaxIntensityEdge_Mito', 'Cytoplasm_Intensity_MaxIntensity_AGP', 'Cytoplasm_Intensity_MaxIntensity_DNA', 'Cytoplasm_Intensity_MaxIntensity_RNA', 
 'Cytoplasm_Intensity_MeanIntensityEdge_AGP', 'Cytoplasm_Intensity_MeanIntensityEdge_RNA', 'Cytoplasm_Intensity_MeanIntensity_AGP', 'Cytoplasm_Intensity_MeanIntensity_DNA', 'Cytoplasm_Intensity_MeanIntensity_Mito', 
 'Cytoplasm_Intensity_MedianIntensity_AGP', 'Cytoplasm_Intensity_MedianIntensity_DNA', 'Cytoplasm_Intensity_MedianIntensity_Mito', 'Cytoplasm_Intensity_MedianIntensity_RNA', 'Cytoplasm_Intensity_MinIntensityEdge_AGP', 
 'Cytoplasm_Intensity_MinIntensity_ER', 'Cytoplasm_Intensity_MinIntensity_Mito', 'Cytoplasm_Intensity_MinIntensity_RNA', 'Cytoplasm_Intensity_StdIntensityEdge_AGP', 'Cytoplasm_Intensity_StdIntensityEdge_Mito', 
 'Cytoplasm_Intensity_StdIntensityEdge_RNA', 'Cytoplasm_Intensity_StdIntensity_Mito', 'Cytoplasm_Intensity_UpperQuartileIntensity_ER', 'Cytoplasm_Intensity_UpperQuartileIntensity_Mito', 'Cytoplasm_Intensity_UpperQuartileIntensity_RNA', 
 'Cytoplasm_RadialDistribution_FracAtD_AGP_1of4', 'Cytoplasm_RadialDistribution_FracAtD_DNA_2of4', 'Cytoplasm_RadialDistribution_FracAtD_DNA_3of4', 'Cytoplasm_RadialDistribution_FracAtD_ER_1of4', 
 'Cytoplasm_RadialDistribution_FracAtD_Mito_1of4', 'Cytoplasm_RadialDistribution_FracAtD_RNA_1of4', 'Cytoplasm_RadialDistribution_FracAtD_RNA_2of4', 'Cytoplasm_RadialDistribution_FracAtD_RNA_3of4', 
 'Cytoplasm_RadialDistribution_FracAtD_RNA_4of4', 'Cytoplasm_RadialDistribution_FracAtD_mito_tubeness_11of16', 'Cytoplasm_RadialDistribution_FracAtD_mito_tubeness_12of16', 'Cytoplasm_RadialDistribution_FracAtD_mito_tubeness_14of16', 
 'Cytoplasm_RadialDistribution_FracAtD_mito_tubeness_15of16', 'Cytoplasm_RadialDistribution_FracAtD_mito_tubeness_16of16', 'Cytoplasm_RadialDistribution_FracAtD_mito_tubeness_16of20', 'Cytoplasm_RadialDistribution_FracAtD_mito_tubeness_18of20', 
 'Cytoplasm_RadialDistribution_FracAtD_mito_tubeness_1of16', 'Cytoplasm_RadialDistribution_FracAtD_mito_tubeness_3of20', 'Cytoplasm_RadialDistribution_FracAtD_mito_tubeness_4of16', 'Cytoplasm_RadialDistribution_FracAtD_mito_tubeness_7of16', 
 'Cytoplasm_RadialDistribution_FracAtD_mito_tubeness_8of16', 'Cytoplasm_RadialDistribution_MeanFrac_AGP_2of4', 'Cytoplasm_RadialDistribution_MeanFrac_DNA_2of4', 'Cytoplasm_RadialDistribution_MeanFrac_ER_1of4',
 'Cytoplasm_RadialDistribution_MeanFrac_Mito_1of4', 'Cytoplasm_RadialDistribution_MeanFrac_RNA_1of4', 'Cytoplasm_RadialDistribution_MeanFrac_mito_tubeness_11of16', 'Cytoplasm_RadialDistribution_MeanFrac_mito_tubeness_13of16', 
 'Cytoplasm_RadialDistribution_MeanFrac_mito_tubeness_15of16', 'Cytoplasm_RadialDistribution_MeanFrac_mito_tubeness_15of20', 'Cytoplasm_RadialDistribution_MeanFrac_mito_tubeness_16of16', 'Cytoplasm_RadialDistribution_MeanFrac_mito_tubeness_16of20', 
 'Cytoplasm_RadialDistribution_MeanFrac_mito_tubeness_17of20', 'Cytoplasm_RadialDistribution_MeanFrac_mito_tubeness_18of20', 'Cytoplasm_RadialDistribution_MeanFrac_mito_tubeness_19of20', 'Cytoplasm_RadialDistribution_MeanFrac_mito_tubeness_1of16', 
 'Cytoplasm_RadialDistribution_MeanFrac_mito_tubeness_2of16', 'Cytoplasm_RadialDistribution_MeanFrac_mito_tubeness_2of20', 'Cytoplasm_RadialDistribution_MeanFrac_mito_tubeness_4of16', 'Cytoplasm_RadialDistribution_MeanFrac_mito_tubeness_7of20', 
 'Cytoplasm_RadialDistribution_MeanFrac_mito_tubeness_8of16', 'Cytoplasm_RadialDistribution_MeanFrac_mito_tubeness_9of20', 'Cytoplasm_RadialDistribution_RadialCV_ER_3of4', 'Cytoplasm_RadialDistribution_RadialCV_Mito_3of4', 
 'Cytoplasm_RadialDistribution_RadialCV_RNA_3of4', 'Cytoplasm_RadialDistribution_RadialCV_mito_tubeness_11of16', 'Cytoplasm_RadialDistribution_RadialCV_mito_tubeness_14of16', 'Cytoplasm_RadialDistribution_RadialCV_mito_tubeness_15of16', 
 'Cytoplasm_RadialDistribution_RadialCV_mito_tubeness_15of20', 'Cytoplasm_RadialDistribution_RadialCV_mito_tubeness_16of16', 'Cytoplasm_RadialDistribution_RadialCV_mito_tubeness_16of20', 'Cytoplasm_RadialDistribution_RadialCV_mito_tubeness_17of20', 
 'Cytoplasm_RadialDistribution_RadialCV_mito_tubeness_18of20', 'Cytoplasm_RadialDistribution_RadialCV_mito_tubeness_19of20', 'Cytoplasm_RadialDistribution_RadialCV_mito_tubeness_8of16'}

variance_dropped_cols_3 = set()

In [279]:
variance_dropped_cols_3 & correlation_dropped_cols_3
print(f" Number of cols dropped - high correlation: {len(correlation_dropped_cols_3)}")
print(f" Number of cols dropped - low variance    : {len(variance_dropped_cols_3)}")
print(f" Number of selected cols                  : {len(selected_cols_3)}")
print(f" Total                                    : {len(selected_cols_3)+len(correlation_dropped_cols_3)+len(variance_dropped_cols_3)}")

set()

 Number of cols dropped - high correlation: 95
 Number of cols dropped - low variance    : 0
 Number of selected cols                  : 365
 Total                                    : 460


In [280]:
(correlation_dropped_cols_3) & set(zero_features)
(variance_dropped_cols_3) & set(zero_features)
(selected_cols_3) & set(zero_features)

set()

set()

{'Cytoplasm_RadialDistribution_FracAtD_mito_tubeness_2of20',
 'Cytoplasm_RadialDistribution_FracAtD_mito_tubeness_Overflow',
 'Cytoplasm_RadialDistribution_MeanFrac_mito_tubeness_Overflow',
 'Cytoplasm_RadialDistribution_RadialCV_mito_tubeness_1of16',
 'Cytoplasm_RadialDistribution_RadialCV_mito_tubeness_1of20',
 'Cytoplasm_RadialDistribution_RadialCV_mito_tubeness_2of20',
 'Cytoplasm_RadialDistribution_RadialCV_mito_tubeness_Overflow'}

### Group 4 - Cytoplasm

In [281]:
selected_cols_4 = \
{"Cytoplasm_Texture_AngularSecondMoment_AGP_10_03_256"       , "Cytoplasm_Texture_AngularSecondMoment_DNA_10_03_256"       , "Cytoplasm_Texture_AngularSecondMoment_ER_10_01_256"        , "Cytoplasm_Texture_AngularSecondMoment_Mito_10_01_256"       ,
 "Cytoplasm_Texture_AngularSecondMoment_RNA_10_03_256"       , "Cytoplasm_Texture_Contrast_DNA_5_00_256"                   , "Cytoplasm_Texture_Contrast_Mito_3_00_256"                  , "Cytoplasm_Texture_Contrast_RNA_10_02_256"                   ,
 "Cytoplasm_Texture_Correlation_AGP_10_00_256"               , "Cytoplasm_Texture_Correlation_AGP_10_01_256"               , "Cytoplasm_Texture_Correlation_AGP_10_02_256"               , "Cytoplasm_Texture_Correlation_AGP_10_03_256"                ,
 "Cytoplasm_Texture_Correlation_AGP_3_00_256"                , "Cytoplasm_Texture_Correlation_AGP_3_01_256"                , "Cytoplasm_Texture_Correlation_AGP_3_02_256"                , "Cytoplasm_Texture_Correlation_AGP_3_03_256"                 ,
 "Cytoplasm_Texture_Correlation_AGP_5_00_256"                , "Cytoplasm_Texture_Correlation_AGP_5_01_256"                , "Cytoplasm_Texture_Correlation_AGP_5_02_256"                , "Cytoplasm_Texture_Correlation_AGP_5_03_256"                 ,
 "Cytoplasm_Texture_Correlation_DNA_10_00_256"               , "Cytoplasm_Texture_Correlation_DNA_10_01_256"               , "Cytoplasm_Texture_Correlation_DNA_10_02_256"               , "Cytoplasm_Texture_Correlation_DNA_10_03_256"                ,
 "Cytoplasm_Texture_Correlation_DNA_3_00_256"                , "Cytoplasm_Texture_Correlation_DNA_3_01_256"                , "Cytoplasm_Texture_Correlation_DNA_3_02_256"                , "Cytoplasm_Texture_Correlation_DNA_3_03_256"                 ,
 "Cytoplasm_Texture_Correlation_DNA_5_00_256"                , "Cytoplasm_Texture_Correlation_DNA_5_01_256"                , "Cytoplasm_Texture_Correlation_DNA_5_02_256"                , "Cytoplasm_Texture_Correlation_DNA_5_03_256"                 ,
 "Cytoplasm_Texture_Correlation_ER_10_00_256"                , "Cytoplasm_Texture_Correlation_ER_10_01_256"                , "Cytoplasm_Texture_Correlation_ER_10_02_256"                , "Cytoplasm_Texture_Correlation_ER_10_03_256"                 ,
 "Cytoplasm_Texture_Correlation_ER_3_00_256"                 , "Cytoplasm_Texture_Correlation_ER_3_02_256"                 , "Cytoplasm_Texture_Correlation_Mito_10_00_256"              , "Cytoplasm_Texture_Correlation_Mito_10_01_256"               ,
 "Cytoplasm_Texture_Correlation_Mito_10_02_256"              , "Cytoplasm_Texture_Correlation_Mito_10_03_256"              , "Cytoplasm_Texture_Correlation_Mito_3_00_256"               , "Cytoplasm_Texture_Correlation_Mito_3_01_256"                ,
 "Cytoplasm_Texture_Correlation_Mito_3_02_256"               , "Cytoplasm_Texture_Correlation_Mito_3_03_256"               , "Cytoplasm_Texture_Correlation_Mito_5_01_256"               , "Cytoplasm_Texture_Correlation_Mito_5_03_256"                ,
 "Cytoplasm_Texture_Correlation_RNA_10_00_256"               , "Cytoplasm_Texture_Correlation_RNA_10_01_256"               , "Cytoplasm_Texture_Correlation_RNA_10_02_256"               , "Cytoplasm_Texture_Correlation_RNA_10_03_256"                ,
 "Cytoplasm_Texture_Correlation_RNA_3_00_256"                , "Cytoplasm_Texture_Correlation_RNA_3_02_256"                , "Cytoplasm_Texture_DifferenceEntropy_ER_10_01_256"          , "Cytoplasm_Texture_DifferenceVariance_AGP_3_00_256"          ,
 "Cytoplasm_Texture_DifferenceVariance_DNA_10_03_256"        , "Cytoplasm_Texture_DifferenceVariance_ER_10_01_256"         , "Cytoplasm_Texture_DifferenceVariance_Mito_10_01_256"       , "Cytoplasm_Texture_DifferenceVariance_RNA_10_03_256"         ,
 "Cytoplasm_Texture_Entropy_AGP_10_03_256"                   , "Cytoplasm_Texture_Entropy_DNA_3_02_256"                    , "Cytoplasm_Texture_Entropy_ER_10_01_256"                    , "Cytoplasm_Texture_Entropy_Mito_10_01_256"                   ,
 "Cytoplasm_Texture_Entropy_RNA_10_03_256"                   , "Cytoplasm_Texture_InfoMeas1_AGP_10_00_256"                 , "Cytoplasm_Texture_InfoMeas1_AGP_10_02_256"                 , "Cytoplasm_Texture_InfoMeas1_AGP_10_03_256"                  ,
 "Cytoplasm_Texture_InfoMeas1_AGP_3_00_256"                  , "Cytoplasm_Texture_InfoMeas1_AGP_3_01_256"                  , "Cytoplasm_Texture_InfoMeas1_AGP_3_02_256"                  , "Cytoplasm_Texture_InfoMeas1_AGP_3_03_256"                   ,
 "Cytoplasm_Texture_InfoMeas1_AGP_5_00_256"                  , "Cytoplasm_Texture_InfoMeas1_AGP_5_01_256"                  , "Cytoplasm_Texture_InfoMeas1_AGP_5_02_256"                  , "Cytoplasm_Texture_InfoMeas1_AGP_5_03_256"                   ,
 "Cytoplasm_Texture_InfoMeas1_DNA_10_00_256"                 , "Cytoplasm_Texture_InfoMeas1_DNA_10_01_256"                 , "Cytoplasm_Texture_InfoMeas1_DNA_10_02_256"                 , "Cytoplasm_Texture_InfoMeas1_DNA_10_03_256"                  ,
 "Cytoplasm_Texture_InfoMeas1_DNA_3_00_256"                  , "Cytoplasm_Texture_InfoMeas1_DNA_3_01_256"                  , "Cytoplasm_Texture_InfoMeas1_DNA_3_02_256"                  , "Cytoplasm_Texture_InfoMeas1_DNA_3_03_256"                   ,
 "Cytoplasm_Texture_InfoMeas1_DNA_5_00_256"                  , "Cytoplasm_Texture_InfoMeas1_DNA_5_01_256"                  , "Cytoplasm_Texture_InfoMeas1_DNA_5_02_256"                  , "Cytoplasm_Texture_InfoMeas1_DNA_5_03_256"                   ,
 "Cytoplasm_Texture_InfoMeas1_ER_10_00_256"                  , "Cytoplasm_Texture_InfoMeas1_ER_10_02_256"                  , "Cytoplasm_Texture_InfoMeas1_ER_10_03_256"                  , "Cytoplasm_Texture_InfoMeas1_ER_3_00_256"                    ,
 "Cytoplasm_Texture_InfoMeas1_ER_3_02_256"                   , "Cytoplasm_Texture_InfoMeas1_ER_5_00_256"                   , "Cytoplasm_Texture_InfoMeas1_ER_5_01_256"                   , "Cytoplasm_Texture_InfoMeas1_ER_5_02_256"                    ,
 "Cytoplasm_Texture_InfoMeas1_ER_5_03_256"                   , "Cytoplasm_Texture_InfoMeas1_Mito_10_00_256"                , "Cytoplasm_Texture_InfoMeas1_Mito_10_01_256"                , "Cytoplasm_Texture_InfoMeas1_Mito_10_02_256"                 ,
 "Cytoplasm_Texture_InfoMeas1_Mito_3_00_256"                 , "Cytoplasm_Texture_InfoMeas1_Mito_3_01_256"                 , "Cytoplasm_Texture_InfoMeas1_Mito_3_02_256"                 , "Cytoplasm_Texture_InfoMeas1_Mito_3_03_256"                  ,
 "Cytoplasm_Texture_InfoMeas1_Mito_5_00_256"                 , "Cytoplasm_Texture_InfoMeas1_Mito_5_01_256"                 , "Cytoplasm_Texture_InfoMeas1_Mito_5_03_256"                 , "Cytoplasm_Texture_InfoMeas1_RNA_10_00_256"                  ,
 "Cytoplasm_Texture_InfoMeas1_RNA_10_02_256"                 , "Cytoplasm_Texture_InfoMeas1_RNA_10_03_256"                 , "Cytoplasm_Texture_InfoMeas1_RNA_5_00_256"                  , "Cytoplasm_Texture_InfoMeas1_RNA_5_01_256"                   ,
 "Cytoplasm_Texture_InfoMeas1_RNA_5_02_256"                  , "Cytoplasm_Texture_InfoMeas1_RNA_5_03_256"                  , "Cytoplasm_Texture_InfoMeas2_AGP_10_02_256"                 , "Cytoplasm_Texture_InfoMeas2_AGP_3_00_256"                   ,
 "Cytoplasm_Texture_InfoMeas2_AGP_3_01_256"                  , "Cytoplasm_Texture_InfoMeas2_AGP_3_02_256"                  , "Cytoplasm_Texture_InfoMeas2_AGP_3_03_256"                  , "Cytoplasm_Texture_InfoMeas2_DNA_10_01_256"                  ,
 "Cytoplasm_Texture_InfoMeas2_DNA_10_02_256"                 , "Cytoplasm_Texture_InfoMeas2_DNA_10_03_256"                 , "Cytoplasm_Texture_InfoMeas2_DNA_3_02_256"                  , "Cytoplasm_Texture_InfoMeas2_ER_10_02_256"                   ,
 "Cytoplasm_Texture_InfoMeas2_ER_10_03_256"                  , "Cytoplasm_Texture_InfoMeas2_ER_3_00_256"                   , "Cytoplasm_Texture_InfoMeas2_ER_3_02_256"                   , "Cytoplasm_Texture_InfoMeas2_ER_5_01_256"                    ,
 "Cytoplasm_Texture_InfoMeas2_ER_5_03_256"                   , "Cytoplasm_Texture_InfoMeas2_Mito_3_02_256"                 , "Cytoplasm_Texture_InfoMeas2_RNA_10_02_256"                 , "Cytoplasm_Texture_InfoMeas2_RNA_10_03_256"                  ,
 "Cytoplasm_Texture_InfoMeas2_RNA_3_00_256"                  , "Cytoplasm_Texture_InfoMeas2_RNA_5_01_256"                  , "Cytoplasm_Texture_InverseDifferenceMoment_AGP_3_00_256"    , "Cytoplasm_Texture_InverseDifferenceMoment_DNA_3_02_256"     ,
 "Cytoplasm_Texture_InverseDifferenceMoment_ER_10_01_256"    , "Cytoplasm_Texture_InverseDifferenceMoment_Mito_10_01_256"  , "Cytoplasm_Texture_InverseDifferenceMoment_RNA_10_01_256"   , "Cytoplasm_Texture_SumAverage_AGP_3_00_256"                  ,
 "Cytoplasm_Texture_SumAverage_DNA_3_02_256"                 , "Cytoplasm_Texture_SumAverage_ER_3_00_256"                  , "Cytoplasm_Texture_SumAverage_Mito_3_00_256"                , "Cytoplasm_Texture_SumAverage_RNA_3_00_256"                  ,
 "Cytoplasm_Texture_SumEntropy_DNA_10_03_256"                , "Cytoplasm_Texture_SumVariance_AGP_10_03_256"               , "Cytoplasm_Texture_SumVariance_DNA_10_01_256"               , "Cytoplasm_Texture_SumVariance_DNA_10_03_256"                ,
 "Cytoplasm_Texture_SumVariance_DNA_3_02_256"                , "Cytoplasm_Texture_SumVariance_ER_10_01_256"                , "Cytoplasm_Texture_SumVariance_Mito_10_01_256"              , "Cytoplasm_Texture_SumVariance_RNA_10_03_256"                
}

correlation_dropped_cols_4 = \
{"Cytoplasm_Texture_AngularSecondMoment_AGP_10_00_256"       , "Cytoplasm_Texture_AngularSecondMoment_AGP_10_01_256"       , "Cytoplasm_Texture_AngularSecondMoment_AGP_10_02_256"       , "Cytoplasm_Texture_AngularSecondMoment_AGP_3_00_256"         ,
 "Cytoplasm_Texture_AngularSecondMoment_AGP_3_01_256"        , "Cytoplasm_Texture_AngularSecondMoment_AGP_3_02_256"        , "Cytoplasm_Texture_AngularSecondMoment_AGP_3_03_256"        , "Cytoplasm_Texture_AngularSecondMoment_AGP_5_00_256"         ,
 "Cytoplasm_Texture_AngularSecondMoment_AGP_5_01_256"        , "Cytoplasm_Texture_AngularSecondMoment_AGP_5_02_256"        , "Cytoplasm_Texture_AngularSecondMoment_AGP_5_03_256"        , "Cytoplasm_Texture_AngularSecondMoment_DNA_10_00_256"        ,
 "Cytoplasm_Texture_AngularSecondMoment_DNA_10_01_256"       , "Cytoplasm_Texture_AngularSecondMoment_DNA_10_02_256"       , "Cytoplasm_Texture_AngularSecondMoment_DNA_3_00_256"        , "Cytoplasm_Texture_AngularSecondMoment_DNA_3_01_256"         ,
 "Cytoplasm_Texture_AngularSecondMoment_DNA_3_02_256"        , "Cytoplasm_Texture_AngularSecondMoment_DNA_3_03_256"        , "Cytoplasm_Texture_AngularSecondMoment_DNA_5_00_256"        , "Cytoplasm_Texture_AngularSecondMoment_DNA_5_01_256"         ,
 "Cytoplasm_Texture_AngularSecondMoment_DNA_5_02_256"        , "Cytoplasm_Texture_AngularSecondMoment_DNA_5_03_256"        , "Cytoplasm_Texture_AngularSecondMoment_ER_10_00_256"        , "Cytoplasm_Texture_AngularSecondMoment_ER_10_02_256"         ,
 "Cytoplasm_Texture_AngularSecondMoment_ER_10_03_256"        , "Cytoplasm_Texture_AngularSecondMoment_ER_3_00_256"         , "Cytoplasm_Texture_AngularSecondMoment_ER_3_01_256"         , "Cytoplasm_Texture_AngularSecondMoment_ER_3_02_256"          ,
 "Cytoplasm_Texture_AngularSecondMoment_ER_3_03_256"         , "Cytoplasm_Texture_AngularSecondMoment_ER_5_00_256"         , "Cytoplasm_Texture_AngularSecondMoment_ER_5_01_256"         , "Cytoplasm_Texture_AngularSecondMoment_ER_5_02_256"          ,
 "Cytoplasm_Texture_AngularSecondMoment_ER_5_03_256"         , "Cytoplasm_Texture_AngularSecondMoment_Mito_10_00_256"      , "Cytoplasm_Texture_AngularSecondMoment_Mito_10_02_256"      , "Cytoplasm_Texture_AngularSecondMoment_Mito_10_03_256"       ,
 "Cytoplasm_Texture_AngularSecondMoment_Mito_3_00_256"       , "Cytoplasm_Texture_AngularSecondMoment_Mito_3_01_256"       , "Cytoplasm_Texture_AngularSecondMoment_Mito_3_02_256"       , "Cytoplasm_Texture_AngularSecondMoment_Mito_3_03_256"        ,
 "Cytoplasm_Texture_AngularSecondMoment_Mito_5_00_256"       , "Cytoplasm_Texture_AngularSecondMoment_Mito_5_01_256"       , "Cytoplasm_Texture_AngularSecondMoment_Mito_5_02_256"       , "Cytoplasm_Texture_AngularSecondMoment_Mito_5_03_256"        ,
 "Cytoplasm_Texture_AngularSecondMoment_RNA_10_00_256"       , "Cytoplasm_Texture_AngularSecondMoment_RNA_10_01_256"       , "Cytoplasm_Texture_AngularSecondMoment_RNA_10_02_256"       , "Cytoplasm_Texture_AngularSecondMoment_RNA_3_00_256"         ,
 "Cytoplasm_Texture_AngularSecondMoment_RNA_3_01_256"        , "Cytoplasm_Texture_AngularSecondMoment_RNA_3_02_256"        , "Cytoplasm_Texture_AngularSecondMoment_RNA_3_03_256"        , "Cytoplasm_Texture_AngularSecondMoment_RNA_5_00_256"         ,
 "Cytoplasm_Texture_AngularSecondMoment_RNA_5_01_256"        , "Cytoplasm_Texture_AngularSecondMoment_RNA_5_02_256"        , "Cytoplasm_Texture_AngularSecondMoment_RNA_5_03_256"        , "Cytoplasm_Texture_Contrast_AGP_10_00_256"                   ,
 "Cytoplasm_Texture_Contrast_AGP_10_01_256"                  , "Cytoplasm_Texture_Contrast_AGP_10_02_256"                  , "Cytoplasm_Texture_Contrast_AGP_10_03_256"                  , "Cytoplasm_Texture_Contrast_AGP_3_00_256"                    ,
 "Cytoplasm_Texture_Contrast_AGP_3_01_256"                   , "Cytoplasm_Texture_Contrast_AGP_3_02_256"                   , "Cytoplasm_Texture_Contrast_AGP_3_03_256"                   , "Cytoplasm_Texture_Contrast_AGP_5_00_256"                    ,
 "Cytoplasm_Texture_Contrast_AGP_5_01_256"                   , "Cytoplasm_Texture_Contrast_AGP_5_02_256"                   , "Cytoplasm_Texture_Contrast_AGP_5_03_256"                   , "Cytoplasm_Texture_Contrast_DNA_10_00_256"                   ,
 "Cytoplasm_Texture_Contrast_DNA_10_01_256"                  , "Cytoplasm_Texture_Contrast_DNA_10_02_256"                  , "Cytoplasm_Texture_Contrast_DNA_10_03_256"                  , "Cytoplasm_Texture_Contrast_DNA_3_00_256"                    ,
 "Cytoplasm_Texture_Contrast_DNA_3_01_256"                   , "Cytoplasm_Texture_Contrast_DNA_3_02_256"                   , "Cytoplasm_Texture_Contrast_DNA_3_03_256"                   , "Cytoplasm_Texture_Contrast_DNA_5_01_256"                    ,
 "Cytoplasm_Texture_Contrast_DNA_5_02_256"                   , "Cytoplasm_Texture_Contrast_DNA_5_03_256"                   , "Cytoplasm_Texture_Contrast_ER_10_00_256"                   , "Cytoplasm_Texture_Contrast_ER_10_01_256"                    ,
 "Cytoplasm_Texture_Contrast_ER_10_02_256"                   , "Cytoplasm_Texture_Contrast_ER_10_03_256"                   , "Cytoplasm_Texture_Contrast_ER_3_00_256"                    , "Cytoplasm_Texture_Contrast_ER_3_01_256"                     ,
 "Cytoplasm_Texture_Contrast_ER_3_02_256"                    , "Cytoplasm_Texture_Contrast_ER_3_03_256"                    , "Cytoplasm_Texture_Contrast_ER_5_00_256"                    , "Cytoplasm_Texture_Contrast_ER_5_01_256"                     ,
 "Cytoplasm_Texture_Contrast_ER_5_02_256"                    , "Cytoplasm_Texture_Contrast_ER_5_03_256"                    , "Cytoplasm_Texture_Contrast_Mito_10_00_256"                 , "Cytoplasm_Texture_Contrast_Mito_10_01_256"                  ,
 "Cytoplasm_Texture_Contrast_Mito_10_02_256"                 , "Cytoplasm_Texture_Contrast_Mito_10_03_256"                 , "Cytoplasm_Texture_Contrast_Mito_3_01_256"                  , "Cytoplasm_Texture_Contrast_Mito_3_02_256"                   ,
 "Cytoplasm_Texture_Contrast_Mito_3_03_256"                  , "Cytoplasm_Texture_Contrast_Mito_5_00_256"                  , "Cytoplasm_Texture_Contrast_Mito_5_01_256"                  , "Cytoplasm_Texture_Contrast_Mito_5_02_256"                   ,
 "Cytoplasm_Texture_Contrast_Mito_5_03_256"                  , "Cytoplasm_Texture_Contrast_RNA_10_00_256"                  , "Cytoplasm_Texture_Contrast_RNA_10_01_256"                  , "Cytoplasm_Texture_Contrast_RNA_10_03_256"                   ,
 "Cytoplasm_Texture_Contrast_RNA_3_00_256"                   , "Cytoplasm_Texture_Contrast_RNA_3_01_256"                   , "Cytoplasm_Texture_Contrast_RNA_3_02_256"                   , "Cytoplasm_Texture_Contrast_RNA_3_03_256"                    ,
 "Cytoplasm_Texture_Contrast_RNA_5_00_256"                   , "Cytoplasm_Texture_Contrast_RNA_5_01_256"                   , "Cytoplasm_Texture_Contrast_RNA_5_02_256"                   , "Cytoplasm_Texture_Contrast_RNA_5_03_256"                    ,
 "Cytoplasm_Texture_Correlation_ER_3_01_256"                 , "Cytoplasm_Texture_Correlation_ER_3_03_256"                 , "Cytoplasm_Texture_Correlation_ER_5_00_256"                 , "Cytoplasm_Texture_Correlation_ER_5_01_256"                  ,
 "Cytoplasm_Texture_Correlation_ER_5_02_256"                 , "Cytoplasm_Texture_Correlation_ER_5_03_256"                 , "Cytoplasm_Texture_Correlation_Mito_5_00_256"               , "Cytoplasm_Texture_Correlation_Mito_5_02_256"                ,
 "Cytoplasm_Texture_Correlation_RNA_3_01_256"                , "Cytoplasm_Texture_Correlation_RNA_3_03_256"                , "Cytoplasm_Texture_Correlation_RNA_5_00_256"                , "Cytoplasm_Texture_Correlation_RNA_5_01_256"                 ,
 "Cytoplasm_Texture_Correlation_RNA_5_02_256"                , "Cytoplasm_Texture_Correlation_RNA_5_03_256"                , "Cytoplasm_Texture_DifferenceEntropy_AGP_10_00_256"         , "Cytoplasm_Texture_DifferenceEntropy_AGP_10_01_256"          ,
 "Cytoplasm_Texture_DifferenceEntropy_AGP_10_02_256"         , "Cytoplasm_Texture_DifferenceEntropy_AGP_10_03_256"         , "Cytoplasm_Texture_DifferenceEntropy_AGP_3_00_256"          , "Cytoplasm_Texture_DifferenceEntropy_AGP_3_01_256"           ,
 "Cytoplasm_Texture_DifferenceEntropy_AGP_3_02_256"          , "Cytoplasm_Texture_DifferenceEntropy_AGP_3_03_256"          , "Cytoplasm_Texture_DifferenceEntropy_AGP_5_00_256"          , "Cytoplasm_Texture_DifferenceEntropy_AGP_5_01_256"           ,
 "Cytoplasm_Texture_DifferenceEntropy_AGP_5_02_256"          , "Cytoplasm_Texture_DifferenceEntropy_AGP_5_03_256"          , "Cytoplasm_Texture_DifferenceEntropy_DNA_10_00_256"         , "Cytoplasm_Texture_DifferenceEntropy_DNA_10_01_256"          ,
 "Cytoplasm_Texture_DifferenceEntropy_DNA_10_02_256"         , "Cytoplasm_Texture_DifferenceEntropy_DNA_10_03_256"         , "Cytoplasm_Texture_DifferenceEntropy_DNA_3_00_256"          , "Cytoplasm_Texture_DifferenceEntropy_DNA_3_01_256"           ,
 "Cytoplasm_Texture_DifferenceEntropy_DNA_3_02_256"          , "Cytoplasm_Texture_DifferenceEntropy_DNA_3_03_256"          , "Cytoplasm_Texture_DifferenceEntropy_DNA_5_00_256"          , "Cytoplasm_Texture_DifferenceEntropy_DNA_5_01_256"           ,
 "Cytoplasm_Texture_DifferenceEntropy_DNA_5_02_256"          , "Cytoplasm_Texture_DifferenceEntropy_DNA_5_03_256"          , "Cytoplasm_Texture_DifferenceEntropy_ER_10_00_256"          , "Cytoplasm_Texture_DifferenceEntropy_ER_10_02_256"           ,
 "Cytoplasm_Texture_DifferenceEntropy_ER_10_03_256"          , "Cytoplasm_Texture_DifferenceEntropy_ER_3_00_256"           , "Cytoplasm_Texture_DifferenceEntropy_ER_3_01_256"           , "Cytoplasm_Texture_DifferenceEntropy_ER_3_02_256"            ,
 "Cytoplasm_Texture_DifferenceEntropy_ER_3_03_256"           , "Cytoplasm_Texture_DifferenceEntropy_ER_5_00_256"           , "Cytoplasm_Texture_DifferenceEntropy_ER_5_01_256"           , "Cytoplasm_Texture_DifferenceEntropy_ER_5_02_256"            ,
 "Cytoplasm_Texture_DifferenceEntropy_ER_5_03_256"           , "Cytoplasm_Texture_DifferenceEntropy_Mito_10_00_256"        , "Cytoplasm_Texture_DifferenceEntropy_Mito_10_01_256"        , "Cytoplasm_Texture_DifferenceEntropy_Mito_10_02_256"         ,
 "Cytoplasm_Texture_DifferenceEntropy_Mito_10_03_256"        , "Cytoplasm_Texture_DifferenceEntropy_Mito_3_00_256"         , "Cytoplasm_Texture_DifferenceEntropy_Mito_3_01_256"         , "Cytoplasm_Texture_DifferenceEntropy_Mito_3_02_256"          ,
 "Cytoplasm_Texture_DifferenceEntropy_Mito_3_03_256"         , "Cytoplasm_Texture_DifferenceEntropy_Mito_5_00_256"         , "Cytoplasm_Texture_DifferenceEntropy_Mito_5_01_256"         , "Cytoplasm_Texture_DifferenceEntropy_Mito_5_02_256"          ,
 "Cytoplasm_Texture_DifferenceEntropy_Mito_5_03_256"         , "Cytoplasm_Texture_DifferenceEntropy_RNA_10_00_256"         , "Cytoplasm_Texture_DifferenceEntropy_RNA_10_01_256"         , "Cytoplasm_Texture_DifferenceEntropy_RNA_10_02_256"          ,
 "Cytoplasm_Texture_DifferenceEntropy_RNA_10_03_256"         , "Cytoplasm_Texture_DifferenceEntropy_RNA_3_00_256"          , "Cytoplasm_Texture_DifferenceEntropy_RNA_3_01_256"          , "Cytoplasm_Texture_DifferenceEntropy_RNA_3_02_256"           ,
 "Cytoplasm_Texture_DifferenceEntropy_RNA_3_03_256"          , "Cytoplasm_Texture_DifferenceEntropy_RNA_5_00_256"          , "Cytoplasm_Texture_DifferenceEntropy_RNA_5_01_256"          , "Cytoplasm_Texture_DifferenceEntropy_RNA_5_02_256"           ,
 "Cytoplasm_Texture_DifferenceEntropy_RNA_5_03_256"          , "Cytoplasm_Texture_DifferenceVariance_AGP_10_00_256"        , "Cytoplasm_Texture_DifferenceVariance_AGP_10_01_256"        , "Cytoplasm_Texture_DifferenceVariance_AGP_10_02_256"         ,
 "Cytoplasm_Texture_DifferenceVariance_AGP_10_03_256"        , "Cytoplasm_Texture_DifferenceVariance_AGP_3_01_256"         , "Cytoplasm_Texture_DifferenceVariance_AGP_3_02_256"         , "Cytoplasm_Texture_DifferenceVariance_AGP_3_03_256"          ,
 "Cytoplasm_Texture_DifferenceVariance_AGP_5_00_256"         , "Cytoplasm_Texture_DifferenceVariance_AGP_5_01_256"         , "Cytoplasm_Texture_DifferenceVariance_AGP_5_02_256"         , "Cytoplasm_Texture_DifferenceVariance_AGP_5_03_256"          ,
 "Cytoplasm_Texture_DifferenceVariance_DNA_10_00_256"        , "Cytoplasm_Texture_DifferenceVariance_DNA_10_01_256"        , "Cytoplasm_Texture_DifferenceVariance_DNA_10_02_256"        , "Cytoplasm_Texture_DifferenceVariance_DNA_3_00_256"          ,
 "Cytoplasm_Texture_DifferenceVariance_DNA_3_01_256"         , "Cytoplasm_Texture_DifferenceVariance_DNA_3_02_256"         , "Cytoplasm_Texture_DifferenceVariance_DNA_3_03_256"         , "Cytoplasm_Texture_DifferenceVariance_DNA_5_00_256"          ,
 "Cytoplasm_Texture_DifferenceVariance_DNA_5_01_256"         , "Cytoplasm_Texture_DifferenceVariance_DNA_5_02_256"         , "Cytoplasm_Texture_DifferenceVariance_DNA_5_03_256"         , "Cytoplasm_Texture_DifferenceVariance_ER_10_00_256"          ,
 "Cytoplasm_Texture_DifferenceVariance_ER_10_02_256"         , "Cytoplasm_Texture_DifferenceVariance_ER_10_03_256"         , "Cytoplasm_Texture_DifferenceVariance_ER_3_00_256"          , "Cytoplasm_Texture_DifferenceVariance_ER_3_01_256"           ,
 "Cytoplasm_Texture_DifferenceVariance_ER_3_02_256"          , "Cytoplasm_Texture_DifferenceVariance_ER_3_03_256"          , "Cytoplasm_Texture_DifferenceVariance_ER_5_00_256"          , "Cytoplasm_Texture_DifferenceVariance_ER_5_01_256"           ,
 "Cytoplasm_Texture_DifferenceVariance_ER_5_02_256"          , "Cytoplasm_Texture_DifferenceVariance_ER_5_03_256"          , "Cytoplasm_Texture_DifferenceVariance_Mito_10_00_256"       , "Cytoplasm_Texture_DifferenceVariance_Mito_10_02_256"        ,
 "Cytoplasm_Texture_DifferenceVariance_Mito_10_03_256"       , "Cytoplasm_Texture_DifferenceVariance_Mito_3_00_256"        , "Cytoplasm_Texture_DifferenceVariance_Mito_3_01_256"        , "Cytoplasm_Texture_DifferenceVariance_Mito_3_02_256"         ,
 "Cytoplasm_Texture_DifferenceVariance_Mito_3_03_256"        , "Cytoplasm_Texture_DifferenceVariance_Mito_5_00_256"        , "Cytoplasm_Texture_DifferenceVariance_Mito_5_01_256"        , "Cytoplasm_Texture_DifferenceVariance_Mito_5_02_256"         ,
 "Cytoplasm_Texture_DifferenceVariance_Mito_5_03_256"        , "Cytoplasm_Texture_DifferenceVariance_RNA_10_00_256"        , "Cytoplasm_Texture_DifferenceVariance_RNA_10_01_256"        , "Cytoplasm_Texture_DifferenceVariance_RNA_10_02_256"         ,
 "Cytoplasm_Texture_DifferenceVariance_RNA_3_00_256"         , "Cytoplasm_Texture_DifferenceVariance_RNA_3_01_256"         , "Cytoplasm_Texture_DifferenceVariance_RNA_3_02_256"         , "Cytoplasm_Texture_DifferenceVariance_RNA_3_03_256"          ,
 "Cytoplasm_Texture_DifferenceVariance_RNA_5_00_256"         , "Cytoplasm_Texture_DifferenceVariance_RNA_5_01_256"         , "Cytoplasm_Texture_DifferenceVariance_RNA_5_02_256"         , "Cytoplasm_Texture_DifferenceVariance_RNA_5_03_256"          ,
 "Cytoplasm_Texture_Entropy_AGP_10_00_256"                   , "Cytoplasm_Texture_Entropy_AGP_10_01_256"                   , "Cytoplasm_Texture_Entropy_AGP_10_02_256"                   , "Cytoplasm_Texture_Entropy_AGP_3_00_256"                     ,
 "Cytoplasm_Texture_Entropy_AGP_3_01_256"                    , "Cytoplasm_Texture_Entropy_AGP_3_02_256"                    , "Cytoplasm_Texture_Entropy_AGP_3_03_256"                    , "Cytoplasm_Texture_Entropy_AGP_5_00_256"                     ,
 "Cytoplasm_Texture_Entropy_AGP_5_01_256"                    , "Cytoplasm_Texture_Entropy_AGP_5_02_256"                    , "Cytoplasm_Texture_Entropy_AGP_5_03_256"                    , "Cytoplasm_Texture_Entropy_DNA_10_00_256"                    ,
 "Cytoplasm_Texture_Entropy_DNA_10_01_256"                   , "Cytoplasm_Texture_Entropy_DNA_10_02_256"                   , "Cytoplasm_Texture_Entropy_DNA_10_03_256"                   , "Cytoplasm_Texture_Entropy_DNA_3_00_256"                     ,
 "Cytoplasm_Texture_Entropy_DNA_3_01_256"                    , "Cytoplasm_Texture_Entropy_DNA_3_03_256"                    , "Cytoplasm_Texture_Entropy_DNA_5_00_256"                    , "Cytoplasm_Texture_Entropy_DNA_5_01_256"                     ,
 "Cytoplasm_Texture_Entropy_DNA_5_02_256"                    , "Cytoplasm_Texture_Entropy_DNA_5_03_256"                    , "Cytoplasm_Texture_Entropy_ER_10_00_256"                    , "Cytoplasm_Texture_Entropy_ER_10_02_256"                     ,
 "Cytoplasm_Texture_Entropy_ER_10_03_256"                    , "Cytoplasm_Texture_Entropy_ER_3_00_256"                     , "Cytoplasm_Texture_Entropy_ER_3_01_256"                     , "Cytoplasm_Texture_Entropy_ER_3_02_256"                      ,
 "Cytoplasm_Texture_Entropy_ER_3_03_256"                     , "Cytoplasm_Texture_Entropy_ER_5_00_256"                     , "Cytoplasm_Texture_Entropy_ER_5_01_256"                     , "Cytoplasm_Texture_Entropy_ER_5_02_256"                      ,
 "Cytoplasm_Texture_Entropy_ER_5_03_256"                     , "Cytoplasm_Texture_Entropy_Mito_10_00_256"                  , "Cytoplasm_Texture_Entropy_Mito_10_02_256"                  , "Cytoplasm_Texture_Entropy_Mito_10_03_256"                   ,
 "Cytoplasm_Texture_Entropy_Mito_3_00_256"                   , "Cytoplasm_Texture_Entropy_Mito_3_01_256"                   , "Cytoplasm_Texture_Entropy_Mito_3_02_256"                   , "Cytoplasm_Texture_Entropy_Mito_3_03_256"                    ,
 "Cytoplasm_Texture_Entropy_Mito_5_00_256"                   , "Cytoplasm_Texture_Entropy_Mito_5_01_256"                   , "Cytoplasm_Texture_Entropy_Mito_5_02_256"                   , "Cytoplasm_Texture_Entropy_Mito_5_03_256"                    ,
 "Cytoplasm_Texture_Entropy_RNA_10_00_256"                   , "Cytoplasm_Texture_Entropy_RNA_10_01_256"                   , "Cytoplasm_Texture_Entropy_RNA_10_02_256"                   , "Cytoplasm_Texture_Entropy_RNA_3_00_256"                     ,
 "Cytoplasm_Texture_Entropy_RNA_3_01_256"                    , "Cytoplasm_Texture_Entropy_RNA_3_02_256"                    , "Cytoplasm_Texture_Entropy_RNA_3_03_256"                    , "Cytoplasm_Texture_Entropy_RNA_5_00_256"                     ,
 "Cytoplasm_Texture_Entropy_RNA_5_01_256"                    , "Cytoplasm_Texture_Entropy_RNA_5_02_256"                    , "Cytoplasm_Texture_Entropy_RNA_5_03_256"                    , "Cytoplasm_Texture_InfoMeas1_AGP_10_01_256"                  ,
 "Cytoplasm_Texture_InfoMeas1_ER_10_01_256"                  , "Cytoplasm_Texture_InfoMeas1_ER_3_01_256"                   , "Cytoplasm_Texture_InfoMeas1_ER_3_03_256"                   , "Cytoplasm_Texture_InfoMeas1_Mito_10_03_256"                 ,
 "Cytoplasm_Texture_InfoMeas1_Mito_5_02_256"                 , "Cytoplasm_Texture_InfoMeas1_RNA_10_01_256"                 , "Cytoplasm_Texture_InfoMeas1_RNA_3_00_256"                  , "Cytoplasm_Texture_InfoMeas1_RNA_3_01_256"                   ,
 "Cytoplasm_Texture_InfoMeas1_RNA_3_02_256"                  , "Cytoplasm_Texture_InfoMeas1_RNA_3_03_256"                  , "Cytoplasm_Texture_InfoMeas2_AGP_10_00_256"                 , "Cytoplasm_Texture_InfoMeas2_AGP_10_01_256"                  ,
 "Cytoplasm_Texture_InfoMeas2_AGP_10_03_256"                 , "Cytoplasm_Texture_InfoMeas2_AGP_5_00_256"                  , "Cytoplasm_Texture_InfoMeas2_AGP_5_01_256"                  , "Cytoplasm_Texture_InfoMeas2_AGP_5_02_256"                   ,
 "Cytoplasm_Texture_InfoMeas2_AGP_5_03_256"                  , "Cytoplasm_Texture_InfoMeas2_DNA_10_00_256"                 , "Cytoplasm_Texture_InfoMeas2_DNA_3_00_256"                  , "Cytoplasm_Texture_InfoMeas2_DNA_3_01_256"                   ,
 "Cytoplasm_Texture_InfoMeas2_DNA_3_03_256"                  , "Cytoplasm_Texture_InfoMeas2_DNA_5_00_256"                  , "Cytoplasm_Texture_InfoMeas2_DNA_5_01_256"                  , "Cytoplasm_Texture_InfoMeas2_DNA_5_02_256"                   ,
 "Cytoplasm_Texture_InfoMeas2_DNA_5_03_256"                  , "Cytoplasm_Texture_InfoMeas2_ER_10_00_256"                  , "Cytoplasm_Texture_InfoMeas2_ER_10_01_256"                  , "Cytoplasm_Texture_InfoMeas2_ER_3_01_256"                    ,
 "Cytoplasm_Texture_InfoMeas2_ER_3_03_256"                   , "Cytoplasm_Texture_InfoMeas2_ER_5_00_256"                   , "Cytoplasm_Texture_InfoMeas2_ER_5_02_256"                   , "Cytoplasm_Texture_InfoMeas2_Mito_10_00_256"                 ,
 "Cytoplasm_Texture_InfoMeas2_Mito_10_01_256"                , "Cytoplasm_Texture_InfoMeas2_Mito_10_02_256"                , "Cytoplasm_Texture_InfoMeas2_Mito_10_03_256"                , "Cytoplasm_Texture_InfoMeas2_Mito_3_00_256"                  ,
 "Cytoplasm_Texture_InfoMeas2_Mito_3_01_256"                 , "Cytoplasm_Texture_InfoMeas2_Mito_3_03_256"                 , "Cytoplasm_Texture_InfoMeas2_Mito_5_00_256"                 , "Cytoplasm_Texture_InfoMeas2_Mito_5_01_256"                  ,
 "Cytoplasm_Texture_InfoMeas2_Mito_5_02_256"                 , "Cytoplasm_Texture_InfoMeas2_Mito_5_03_256"                 , "Cytoplasm_Texture_InfoMeas2_RNA_10_00_256"                 , "Cytoplasm_Texture_InfoMeas2_RNA_10_01_256"                  ,
 "Cytoplasm_Texture_InfoMeas2_RNA_3_01_256"                  , "Cytoplasm_Texture_InfoMeas2_RNA_3_02_256"                  , "Cytoplasm_Texture_InfoMeas2_RNA_3_03_256"                  , "Cytoplasm_Texture_InfoMeas2_RNA_5_00_256"                   ,
 "Cytoplasm_Texture_InfoMeas2_RNA_5_02_256"                  , "Cytoplasm_Texture_InfoMeas2_RNA_5_03_256"                  , "Cytoplasm_Texture_InverseDifferenceMoment_AGP_10_00_256"   , "Cytoplasm_Texture_InverseDifferenceMoment_AGP_10_01_256"    ,
 "Cytoplasm_Texture_InverseDifferenceMoment_AGP_10_02_256"   , "Cytoplasm_Texture_InverseDifferenceMoment_AGP_10_03_256"   , "Cytoplasm_Texture_InverseDifferenceMoment_AGP_3_01_256"    , "Cytoplasm_Texture_InverseDifferenceMoment_AGP_3_02_256"     ,
 "Cytoplasm_Texture_InverseDifferenceMoment_AGP_3_03_256"    , "Cytoplasm_Texture_InverseDifferenceMoment_AGP_5_00_256"    , "Cytoplasm_Texture_InverseDifferenceMoment_AGP_5_01_256"    , "Cytoplasm_Texture_InverseDifferenceMoment_AGP_5_02_256"     ,
 "Cytoplasm_Texture_InverseDifferenceMoment_AGP_5_03_256"    , "Cytoplasm_Texture_InverseDifferenceMoment_DNA_10_00_256"   , "Cytoplasm_Texture_InverseDifferenceMoment_DNA_10_01_256"   , "Cytoplasm_Texture_InverseDifferenceMoment_DNA_10_02_256"    ,
 "Cytoplasm_Texture_InverseDifferenceMoment_DNA_10_03_256"   , "Cytoplasm_Texture_InverseDifferenceMoment_DNA_3_00_256"    , "Cytoplasm_Texture_InverseDifferenceMoment_DNA_3_01_256"    , "Cytoplasm_Texture_InverseDifferenceMoment_DNA_3_03_256"     ,
 "Cytoplasm_Texture_InverseDifferenceMoment_DNA_5_00_256"    , "Cytoplasm_Texture_InverseDifferenceMoment_DNA_5_01_256"    , "Cytoplasm_Texture_InverseDifferenceMoment_DNA_5_02_256"    , "Cytoplasm_Texture_InverseDifferenceMoment_DNA_5_03_256"     ,
 "Cytoplasm_Texture_InverseDifferenceMoment_ER_10_00_256"    , "Cytoplasm_Texture_InverseDifferenceMoment_ER_10_02_256"    , "Cytoplasm_Texture_InverseDifferenceMoment_ER_10_03_256"    , "Cytoplasm_Texture_InverseDifferenceMoment_ER_3_00_256"      ,
 "Cytoplasm_Texture_InverseDifferenceMoment_ER_3_01_256"     , "Cytoplasm_Texture_InverseDifferenceMoment_ER_3_02_256"     , "Cytoplasm_Texture_InverseDifferenceMoment_ER_3_03_256"     , "Cytoplasm_Texture_InverseDifferenceMoment_ER_5_00_256"      ,
 "Cytoplasm_Texture_InverseDifferenceMoment_ER_5_01_256"     , "Cytoplasm_Texture_InverseDifferenceMoment_ER_5_02_256"     , "Cytoplasm_Texture_InverseDifferenceMoment_ER_5_03_256"     , "Cytoplasm_Texture_InverseDifferenceMoment_Mito_10_00_256"   ,
 "Cytoplasm_Texture_InverseDifferenceMoment_Mito_10_02_256"  , "Cytoplasm_Texture_InverseDifferenceMoment_Mito_10_03_256"  , "Cytoplasm_Texture_InverseDifferenceMoment_Mito_3_00_256"   , "Cytoplasm_Texture_InverseDifferenceMoment_Mito_3_01_256"    ,
 "Cytoplasm_Texture_InverseDifferenceMoment_Mito_3_02_256"   , "Cytoplasm_Texture_InverseDifferenceMoment_Mito_3_03_256"   , "Cytoplasm_Texture_InverseDifferenceMoment_Mito_5_00_256"   , "Cytoplasm_Texture_InverseDifferenceMoment_Mito_5_01_256"    ,
 "Cytoplasm_Texture_InverseDifferenceMoment_Mito_5_02_256"   , "Cytoplasm_Texture_InverseDifferenceMoment_Mito_5_03_256"   , "Cytoplasm_Texture_InverseDifferenceMoment_RNA_10_00_256"   , "Cytoplasm_Texture_InverseDifferenceMoment_RNA_10_02_256"    ,
 "Cytoplasm_Texture_InverseDifferenceMoment_RNA_10_03_256"   , "Cytoplasm_Texture_InverseDifferenceMoment_RNA_3_00_256"    , "Cytoplasm_Texture_InverseDifferenceMoment_RNA_3_01_256"    , "Cytoplasm_Texture_InverseDifferenceMoment_RNA_3_02_256"     ,
 "Cytoplasm_Texture_InverseDifferenceMoment_RNA_3_03_256"    , "Cytoplasm_Texture_InverseDifferenceMoment_RNA_5_00_256"    , "Cytoplasm_Texture_InverseDifferenceMoment_RNA_5_01_256"    , "Cytoplasm_Texture_InverseDifferenceMoment_RNA_5_02_256"     ,
 "Cytoplasm_Texture_InverseDifferenceMoment_RNA_5_03_256"    , "Cytoplasm_Texture_SumAverage_AGP_10_00_256"                , "Cytoplasm_Texture_SumAverage_AGP_10_01_256"                , "Cytoplasm_Texture_SumAverage_AGP_10_02_256"                 ,
 "Cytoplasm_Texture_SumAverage_AGP_10_03_256"                , "Cytoplasm_Texture_SumAverage_AGP_3_01_256"                 , "Cytoplasm_Texture_SumAverage_AGP_3_02_256"                 , "Cytoplasm_Texture_SumAverage_AGP_3_03_256"                  ,
 "Cytoplasm_Texture_SumAverage_AGP_5_00_256"                 , "Cytoplasm_Texture_SumAverage_AGP_5_01_256"                 , "Cytoplasm_Texture_SumAverage_AGP_5_02_256"                 , "Cytoplasm_Texture_SumAverage_AGP_5_03_256"                  ,
 "Cytoplasm_Texture_SumAverage_DNA_10_00_256"                , "Cytoplasm_Texture_SumAverage_DNA_10_01_256"                , "Cytoplasm_Texture_SumAverage_DNA_10_02_256"                , "Cytoplasm_Texture_SumAverage_DNA_10_03_256"                 ,
 "Cytoplasm_Texture_SumAverage_DNA_3_00_256"                 , "Cytoplasm_Texture_SumAverage_DNA_3_01_256"                 , "Cytoplasm_Texture_SumAverage_DNA_3_03_256"                 , "Cytoplasm_Texture_SumAverage_DNA_5_00_256"                  ,
 "Cytoplasm_Texture_SumAverage_DNA_5_01_256"                 , "Cytoplasm_Texture_SumAverage_DNA_5_02_256"                 , "Cytoplasm_Texture_SumAverage_DNA_5_03_256"                 , "Cytoplasm_Texture_SumAverage_ER_10_00_256"                  ,
 "Cytoplasm_Texture_SumAverage_ER_10_01_256"                 , "Cytoplasm_Texture_SumAverage_ER_10_02_256"                 , "Cytoplasm_Texture_SumAverage_ER_10_03_256"                 , "Cytoplasm_Texture_SumAverage_ER_3_01_256"                   ,
 "Cytoplasm_Texture_SumAverage_ER_3_02_256"                  , "Cytoplasm_Texture_SumAverage_ER_3_03_256"                  , "Cytoplasm_Texture_SumAverage_ER_5_00_256"                  , "Cytoplasm_Texture_SumAverage_ER_5_01_256"                   ,
 "Cytoplasm_Texture_SumAverage_ER_5_02_256"                  , "Cytoplasm_Texture_SumAverage_ER_5_03_256"                  , "Cytoplasm_Texture_SumAverage_Mito_10_00_256"               , "Cytoplasm_Texture_SumAverage_Mito_10_01_256"                ,
 "Cytoplasm_Texture_SumAverage_Mito_10_02_256"               , "Cytoplasm_Texture_SumAverage_Mito_10_03_256"               , "Cytoplasm_Texture_SumAverage_Mito_3_01_256"                , "Cytoplasm_Texture_SumAverage_Mito_3_02_256"                 ,
 "Cytoplasm_Texture_SumAverage_Mito_3_03_256"                , "Cytoplasm_Texture_SumAverage_Mito_5_00_256"                , "Cytoplasm_Texture_SumAverage_Mito_5_01_256"                , "Cytoplasm_Texture_SumAverage_Mito_5_02_256"                 ,
 "Cytoplasm_Texture_SumAverage_Mito_5_03_256"                , "Cytoplasm_Texture_SumAverage_RNA_10_00_256"                , "Cytoplasm_Texture_SumAverage_RNA_10_01_256"                , "Cytoplasm_Texture_SumAverage_RNA_10_02_256"                 ,
 "Cytoplasm_Texture_SumAverage_RNA_10_03_256"                , "Cytoplasm_Texture_SumAverage_RNA_3_01_256"                 , "Cytoplasm_Texture_SumAverage_RNA_3_02_256"                 , "Cytoplasm_Texture_SumAverage_RNA_3_03_256"                  ,
 "Cytoplasm_Texture_SumAverage_RNA_5_00_256"                 , "Cytoplasm_Texture_SumAverage_RNA_5_01_256"                 , "Cytoplasm_Texture_SumAverage_RNA_5_02_256"                 , "Cytoplasm_Texture_SumAverage_RNA_5_03_256"                  ,
 "Cytoplasm_Texture_SumEntropy_AGP_10_00_256"                , "Cytoplasm_Texture_SumEntropy_AGP_10_01_256"                , "Cytoplasm_Texture_SumEntropy_AGP_10_02_256"                , "Cytoplasm_Texture_SumEntropy_AGP_10_03_256"                 ,
 "Cytoplasm_Texture_SumEntropy_AGP_3_00_256"                 , "Cytoplasm_Texture_SumEntropy_AGP_3_01_256"                 , "Cytoplasm_Texture_SumEntropy_AGP_3_02_256"                 , "Cytoplasm_Texture_SumEntropy_AGP_3_03_256"                  ,
 "Cytoplasm_Texture_SumEntropy_AGP_5_00_256"                 , "Cytoplasm_Texture_SumEntropy_AGP_5_01_256"                 , "Cytoplasm_Texture_SumEntropy_AGP_5_02_256"                 , "Cytoplasm_Texture_SumEntropy_AGP_5_03_256"                  ,
 "Cytoplasm_Texture_SumEntropy_DNA_10_00_256"                , "Cytoplasm_Texture_SumEntropy_DNA_10_01_256"                , "Cytoplasm_Texture_SumEntropy_DNA_10_02_256"                , "Cytoplasm_Texture_SumEntropy_DNA_3_00_256"                  ,
 "Cytoplasm_Texture_SumEntropy_DNA_3_01_256"                 , "Cytoplasm_Texture_SumEntropy_DNA_3_02_256"                 , "Cytoplasm_Texture_SumEntropy_DNA_3_03_256"                 , "Cytoplasm_Texture_SumEntropy_DNA_5_00_256"                  ,
 "Cytoplasm_Texture_SumEntropy_DNA_5_01_256"                 , "Cytoplasm_Texture_SumEntropy_DNA_5_02_256"                 , "Cytoplasm_Texture_SumEntropy_DNA_5_03_256"                 , "Cytoplasm_Texture_SumEntropy_ER_10_00_256"                  ,
 "Cytoplasm_Texture_SumEntropy_ER_10_01_256"                 , "Cytoplasm_Texture_SumEntropy_ER_10_02_256"                 , "Cytoplasm_Texture_SumEntropy_ER_10_03_256"                 , "Cytoplasm_Texture_SumEntropy_ER_3_00_256"                   ,
 "Cytoplasm_Texture_SumEntropy_ER_3_01_256"                  , "Cytoplasm_Texture_SumEntropy_ER_3_02_256"                  , "Cytoplasm_Texture_SumEntropy_ER_3_03_256"                  , "Cytoplasm_Texture_SumEntropy_ER_5_00_256"                   ,
 "Cytoplasm_Texture_SumEntropy_ER_5_01_256"                  , "Cytoplasm_Texture_SumEntropy_ER_5_02_256"                  , "Cytoplasm_Texture_SumEntropy_ER_5_03_256"                  , "Cytoplasm_Texture_SumEntropy_Mito_10_00_256"                ,
 "Cytoplasm_Texture_SumEntropy_Mito_10_01_256"               , "Cytoplasm_Texture_SumEntropy_Mito_10_02_256"               , "Cytoplasm_Texture_SumEntropy_Mito_10_03_256"               , "Cytoplasm_Texture_SumEntropy_Mito_3_00_256"                 ,
 "Cytoplasm_Texture_SumEntropy_Mito_3_01_256"                , "Cytoplasm_Texture_SumEntropy_Mito_3_02_256"                , "Cytoplasm_Texture_SumEntropy_Mito_3_03_256"                , "Cytoplasm_Texture_SumEntropy_Mito_5_00_256"                 ,
 "Cytoplasm_Texture_SumEntropy_Mito_5_01_256"                , "Cytoplasm_Texture_SumEntropy_Mito_5_02_256"                , "Cytoplasm_Texture_SumEntropy_Mito_5_03_256"                , "Cytoplasm_Texture_SumEntropy_RNA_10_00_256"                 ,
 "Cytoplasm_Texture_SumEntropy_RNA_10_01_256"                , "Cytoplasm_Texture_SumEntropy_RNA_10_02_256"                , "Cytoplasm_Texture_SumEntropy_RNA_10_03_256"                , "Cytoplasm_Texture_SumEntropy_RNA_3_00_256"                  ,
 "Cytoplasm_Texture_SumEntropy_RNA_3_01_256"                 , "Cytoplasm_Texture_SumEntropy_RNA_3_02_256"                 , "Cytoplasm_Texture_SumEntropy_RNA_3_03_256"                 , "Cytoplasm_Texture_SumEntropy_RNA_5_00_256"                  ,
 "Cytoplasm_Texture_SumEntropy_RNA_5_01_256"                 , "Cytoplasm_Texture_SumEntropy_RNA_5_02_256"                 , "Cytoplasm_Texture_SumEntropy_RNA_5_03_256"                 , "Cytoplasm_Texture_SumVariance_AGP_10_00_256"                ,
 "Cytoplasm_Texture_SumVariance_AGP_10_01_256"               , "Cytoplasm_Texture_SumVariance_AGP_10_02_256"               , "Cytoplasm_Texture_SumVariance_AGP_3_00_256"                , "Cytoplasm_Texture_SumVariance_AGP_3_01_256"                 ,
 "Cytoplasm_Texture_SumVariance_AGP_3_02_256"                , "Cytoplasm_Texture_SumVariance_AGP_3_03_256"                , "Cytoplasm_Texture_SumVariance_AGP_5_00_256"                , "Cytoplasm_Texture_SumVariance_AGP_5_01_256"                 ,
 "Cytoplasm_Texture_SumVariance_AGP_5_02_256"                , "Cytoplasm_Texture_SumVariance_AGP_5_03_256"                , "Cytoplasm_Texture_SumVariance_DNA_10_00_256"               , "Cytoplasm_Texture_SumVariance_DNA_10_02_256"                ,
 "Cytoplasm_Texture_SumVariance_DNA_3_00_256"                , "Cytoplasm_Texture_SumVariance_DNA_3_01_256"                , "Cytoplasm_Texture_SumVariance_DNA_3_03_256"                , "Cytoplasm_Texture_SumVariance_DNA_5_00_256"                 ,
 "Cytoplasm_Texture_SumVariance_DNA_5_01_256"                , "Cytoplasm_Texture_SumVariance_DNA_5_02_256"                , "Cytoplasm_Texture_SumVariance_DNA_5_03_256"                , "Cytoplasm_Texture_SumVariance_ER_10_00_256"                 ,
 "Cytoplasm_Texture_SumVariance_ER_10_02_256"                , "Cytoplasm_Texture_SumVariance_ER_10_03_256"                , "Cytoplasm_Texture_SumVariance_ER_3_00_256"                 , "Cytoplasm_Texture_SumVariance_ER_3_01_256"                  ,
 "Cytoplasm_Texture_SumVariance_ER_3_02_256"                 , "Cytoplasm_Texture_SumVariance_ER_3_03_256"                 , "Cytoplasm_Texture_SumVariance_ER_5_00_256"                 , "Cytoplasm_Texture_SumVariance_ER_5_01_256"                  ,
 "Cytoplasm_Texture_SumVariance_ER_5_02_256"                 , "Cytoplasm_Texture_SumVariance_ER_5_03_256"                 , "Cytoplasm_Texture_SumVariance_Mito_10_00_256"              , "Cytoplasm_Texture_SumVariance_Mito_10_02_256"               ,
 "Cytoplasm_Texture_SumVariance_Mito_10_03_256"              , "Cytoplasm_Texture_SumVariance_Mito_3_00_256"               , "Cytoplasm_Texture_SumVariance_Mito_3_01_256"               , "Cytoplasm_Texture_SumVariance_Mito_3_02_256"                ,
 "Cytoplasm_Texture_SumVariance_Mito_3_03_256"               , "Cytoplasm_Texture_SumVariance_Mito_5_00_256"               , "Cytoplasm_Texture_SumVariance_Mito_5_01_256"               , "Cytoplasm_Texture_SumVariance_Mito_5_02_256"                ,
 "Cytoplasm_Texture_SumVariance_Mito_5_03_256"               , "Cytoplasm_Texture_SumVariance_RNA_10_00_256"               , "Cytoplasm_Texture_SumVariance_RNA_10_01_256"               , "Cytoplasm_Texture_SumVariance_RNA_10_02_256"                ,
 "Cytoplasm_Texture_SumVariance_RNA_3_00_256"                , "Cytoplasm_Texture_SumVariance_RNA_3_01_256"                , "Cytoplasm_Texture_SumVariance_RNA_3_02_256"                , "Cytoplasm_Texture_SumVariance_RNA_3_03_256"                 ,
 "Cytoplasm_Texture_SumVariance_RNA_5_00_256"                , "Cytoplasm_Texture_SumVariance_RNA_5_01_256"                , "Cytoplasm_Texture_SumVariance_RNA_5_02_256"                , "Cytoplasm_Texture_SumVariance_RNA_5_03_256"                 ,
 "Cytoplasm_Texture_Variance_AGP_10_00_256"                  , "Cytoplasm_Texture_Variance_AGP_10_01_256"                  , "Cytoplasm_Texture_Variance_AGP_10_02_256"                  , "Cytoplasm_Texture_Variance_AGP_10_03_256"                   ,
 "Cytoplasm_Texture_Variance_AGP_3_00_256"                   , "Cytoplasm_Texture_Variance_AGP_3_01_256"                   , "Cytoplasm_Texture_Variance_AGP_3_02_256"                   , "Cytoplasm_Texture_Variance_AGP_3_03_256"                    ,
 "Cytoplasm_Texture_Variance_AGP_5_00_256"                   , "Cytoplasm_Texture_Variance_AGP_5_01_256"                   , "Cytoplasm_Texture_Variance_AGP_5_02_256"                   , "Cytoplasm_Texture_Variance_AGP_5_03_256"                    ,
 "Cytoplasm_Texture_Variance_DNA_10_00_256"                  , "Cytoplasm_Texture_Variance_DNA_10_01_256"                  , "Cytoplasm_Texture_Variance_DNA_10_02_256"                  , "Cytoplasm_Texture_Variance_DNA_10_03_256"                   ,
 "Cytoplasm_Texture_Variance_DNA_3_00_256"                   , "Cytoplasm_Texture_Variance_DNA_3_01_256"                   , "Cytoplasm_Texture_Variance_DNA_3_02_256"                   , "Cytoplasm_Texture_Variance_DNA_3_03_256"                    ,
 "Cytoplasm_Texture_Variance_DNA_5_00_256"                   , "Cytoplasm_Texture_Variance_DNA_5_01_256"                   , "Cytoplasm_Texture_Variance_DNA_5_02_256"                   , "Cytoplasm_Texture_Variance_DNA_5_03_256"                    ,
 "Cytoplasm_Texture_Variance_ER_10_00_256"                   , "Cytoplasm_Texture_Variance_ER_10_01_256"                   , "Cytoplasm_Texture_Variance_ER_10_02_256"                   , "Cytoplasm_Texture_Variance_ER_10_03_256"                    ,
 "Cytoplasm_Texture_Variance_ER_3_00_256"                    , "Cytoplasm_Texture_Variance_ER_3_01_256"                    , "Cytoplasm_Texture_Variance_ER_3_02_256"                    , "Cytoplasm_Texture_Variance_ER_3_03_256"                     ,
 "Cytoplasm_Texture_Variance_ER_5_00_256"                    , "Cytoplasm_Texture_Variance_ER_5_01_256"                    , "Cytoplasm_Texture_Variance_ER_5_02_256"                    , "Cytoplasm_Texture_Variance_ER_5_03_256"                     ,
 "Cytoplasm_Texture_Variance_Mito_10_00_256"                 , "Cytoplasm_Texture_Variance_Mito_10_01_256"                 , "Cytoplasm_Texture_Variance_Mito_10_02_256"                 , "Cytoplasm_Texture_Variance_Mito_10_03_256"                  ,
 "Cytoplasm_Texture_Variance_Mito_3_00_256"                  , "Cytoplasm_Texture_Variance_Mito_3_01_256"                  , "Cytoplasm_Texture_Variance_Mito_3_02_256"                  , "Cytoplasm_Texture_Variance_Mito_3_03_256"                   ,
 "Cytoplasm_Texture_Variance_Mito_5_00_256"                  , "Cytoplasm_Texture_Variance_Mito_5_01_256"                  , "Cytoplasm_Texture_Variance_Mito_5_02_256"                  , "Cytoplasm_Texture_Variance_Mito_5_03_256"                   ,
 "Cytoplasm_Texture_Variance_RNA_10_00_256"                  , "Cytoplasm_Texture_Variance_RNA_10_01_256"                  , "Cytoplasm_Texture_Variance_RNA_10_02_256"                  , "Cytoplasm_Texture_Variance_RNA_10_03_256"                   ,
 "Cytoplasm_Texture_Variance_RNA_3_00_256"                   , "Cytoplasm_Texture_Variance_RNA_3_01_256"                   , "Cytoplasm_Texture_Variance_RNA_3_02_256"                   , "Cytoplasm_Texture_Variance_RNA_3_03_256"                    ,
 "Cytoplasm_Texture_Variance_RNA_5_00_256"                   , "Cytoplasm_Texture_Variance_RNA_5_01_256"                   , "Cytoplasm_Texture_Variance_RNA_5_02_256"                   , "Cytoplasm_Texture_Variance_RNA_5_03_256"                    
}

variance_dropped_cols_4 = set()

In [282]:
variance_dropped_cols_4 & correlation_dropped_cols_4
print(f" Number of cols dropped - high correlation: {len(correlation_dropped_cols_4)}")
print(f" Number of cols dropped - low variance    : {len(variance_dropped_cols_4)}")
print(f" Number of selected cols                  : {len(selected_cols_4)}")
print(f" Total                                    : {len(selected_cols_4)+len(correlation_dropped_cols_4)+len(variance_dropped_cols_4)}")

set()

 Number of cols dropped - high correlation: 628
 Number of cols dropped - low variance    : 0
 Number of selected cols                  : 152
 Total                                    : 780


In [283]:
(correlation_dropped_cols_4) & set(zero_features)
(variance_dropped_cols_4) & set(zero_features)
(selected_cols_4) & set(zero_features)

set()

set()

set()

### Group 5 - Nuclei

In [284]:
selected_cols_5 = \
{"Nuclei_AreaShape_BoundingBoxArea"                           , "Nuclei_AreaShape_BoundingBoxMaximum_Y"                      , "Nuclei_AreaShape_Compactness"                               , "Nuclei_AreaShape_Eccentricity"                              ,
 "Nuclei_AreaShape_EulerNumber"                               , "Nuclei_AreaShape_Extent"                                    , "Nuclei_AreaShape_MinFeretDiameter"                          , "Nuclei_AreaShape_Orientation"                               ,
 "Nuclei_AreaShape_Solidity"                                  , "Nuclei_AreaShape_Zernike_0_0"                               , "Nuclei_AreaShape_Zernike_1_1"                               , "Nuclei_AreaShape_Zernike_2_0"                               ,
 "Nuclei_AreaShape_Zernike_2_2"                               , "Nuclei_AreaShape_Zernike_3_1"                               , "Nuclei_AreaShape_Zernike_3_3"                               , "Nuclei_AreaShape_Zernike_4_0"                               ,
 "Nuclei_AreaShape_Zernike_4_2"                               , "Nuclei_AreaShape_Zernike_4_4"                               , "Nuclei_AreaShape_Zernike_5_1"                               , "Nuclei_AreaShape_Zernike_5_3"                               ,
 "Nuclei_AreaShape_Zernike_5_5"                               , "Nuclei_AreaShape_Zernike_6_0"                               , "Nuclei_AreaShape_Zernike_6_2"                               , "Nuclei_AreaShape_Zernike_6_4"                               ,
 "Nuclei_AreaShape_Zernike_6_6"                               , "Nuclei_AreaShape_Zernike_7_1"                               , "Nuclei_AreaShape_Zernike_7_3"                               , "Nuclei_AreaShape_Zernike_7_5"                               ,
 "Nuclei_AreaShape_Zernike_7_7"                               , "Nuclei_AreaShape_Zernike_8_0"                               , "Nuclei_AreaShape_Zernike_8_2"                               , "Nuclei_AreaShape_Zernike_8_4"                               ,
 "Nuclei_AreaShape_Zernike_8_6"                               , "Nuclei_AreaShape_Zernike_8_8"                               , "Nuclei_AreaShape_Zernike_9_1"                               , "Nuclei_AreaShape_Zernike_9_3"                               ,
 "Nuclei_AreaShape_Zernike_9_5"                               , "Nuclei_AreaShape_Zernike_9_7"                               , "Nuclei_AreaShape_Zernike_9_9"                               , "Nuclei_Children_Cytoplasm_Count"                            ,
 "Nuclei_Correlation_Correlation_AGP_DNA"                     , "Nuclei_Correlation_Correlation_AGP_ER"                      , "Nuclei_Correlation_Correlation_AGP_Mito"                    , "Nuclei_Correlation_Correlation_AGP_RNA"                     ,
 "Nuclei_Correlation_Correlation_DNA_ER"                      , "Nuclei_Correlation_Correlation_DNA_Mito"                    , "Nuclei_Correlation_Correlation_DNA_RNA"                     , "Nuclei_Correlation_Correlation_ER_Mito"                     ,
 "Nuclei_Correlation_Correlation_ER_RNA"                      , "Nuclei_Correlation_Correlation_Mito_RNA"                    , "Nuclei_Correlation_K_AGP_DNA"                               , "Nuclei_Correlation_K_AGP_ER"                                ,
 "Nuclei_Correlation_K_AGP_Mito"                              , "Nuclei_Correlation_K_AGP_RNA"                               , "Nuclei_Correlation_K_DNA_AGP"                               , "Nuclei_Correlation_K_DNA_ER"                                ,
 "Nuclei_Correlation_K_DNA_Mito"                              , "Nuclei_Correlation_K_DNA_RNA"                               , "Nuclei_Correlation_K_ER_AGP"                                , "Nuclei_Correlation_K_ER_DNA"                                ,
 "Nuclei_Correlation_K_ER_Mito"                               , "Nuclei_Correlation_K_ER_RNA"                                , "Nuclei_Correlation_K_Mito_AGP"                              , "Nuclei_Correlation_K_Mito_DNA"                              ,
 "Nuclei_Correlation_K_Mito_ER"                               , "Nuclei_Correlation_K_Mito_RNA"                              , "Nuclei_Correlation_K_RNA_AGP"                               , "Nuclei_Correlation_K_RNA_DNA"                               ,
 "Nuclei_Correlation_K_RNA_ER"                                , "Nuclei_Correlation_K_RNA_Mito"                              , "Nuclei_Correlation_Manders_AGP_ER"                          , "Nuclei_Correlation_Manders_AGP_RNA"                         ,
 "Nuclei_Correlation_Manders_DNA_ER"                          , "Nuclei_Correlation_Manders_DNA_RNA"                         , "Nuclei_Correlation_Manders_ER_AGP"                          , "Nuclei_Correlation_Manders_ER_RNA"                          ,
 "Nuclei_Correlation_Manders_Mito_AGP"                        , "Nuclei_Correlation_Manders_Mito_DNA"                        , "Nuclei_Correlation_Manders_Mito_ER"                         , "Nuclei_Correlation_Manders_Mito_RNA"                        ,
 "Nuclei_Correlation_Manders_RNA_ER"                          , "Nuclei_Correlation_Manders_RNA_Mito"                        , "Nuclei_Correlation_Overlap_AGP_DNA"                         , "Nuclei_Correlation_Overlap_AGP_ER"                          ,
 "Nuclei_Correlation_Overlap_AGP_Mito"                        , "Nuclei_Correlation_Overlap_AGP_RNA"                         , "Nuclei_Correlation_Overlap_DNA_ER"                          , "Nuclei_Correlation_Overlap_DNA_Mito"                        ,
 "Nuclei_Correlation_Overlap_DNA_RNA"                         , "Nuclei_Correlation_Overlap_ER_Mito"                         , "Nuclei_Correlation_Overlap_ER_RNA"                          , "Nuclei_Correlation_Overlap_Mito_RNA"                        ,
 "Nuclei_Correlation_RWC_AGP_DNA"                             , "Nuclei_Correlation_RWC_AGP_ER"                              , "Nuclei_Correlation_RWC_AGP_Mito"                            , "Nuclei_Correlation_RWC_AGP_RNA"                             ,
 "Nuclei_Correlation_RWC_DNA_AGP"                             , "Nuclei_Correlation_RWC_DNA_ER"                              , "Nuclei_Correlation_RWC_DNA_Mito"                            , "Nuclei_Correlation_RWC_DNA_RNA"                             ,
 "Nuclei_Correlation_RWC_ER_AGP"                              , "Nuclei_Correlation_RWC_ER_DNA"                              , "Nuclei_Correlation_RWC_ER_Mito"                             , "Nuclei_Correlation_RWC_ER_RNA"                              ,
 "Nuclei_Correlation_RWC_Mito_AGP"                            , "Nuclei_Correlation_RWC_Mito_DNA"                            , "Nuclei_Correlation_RWC_Mito_ER"                             , "Nuclei_Correlation_RWC_Mito_RNA"                            ,
 "Nuclei_Correlation_RWC_RNA_AGP"                             , "Nuclei_Correlation_RWC_RNA_DNA"                             , "Nuclei_Correlation_RWC_RNA_Mito"                            , "Nuclei_Granularity_10_AGP"                                  ,
 "Nuclei_Granularity_10_DNA"                                  , "Nuclei_Granularity_10_ER"                                   , "Nuclei_Granularity_10_Mito"                                 , "Nuclei_Granularity_10_RNA"                                  ,
 "Nuclei_Granularity_11_AGP"                                  , "Nuclei_Granularity_11_DNA"                                  , "Nuclei_Granularity_11_ER"                                   , "Nuclei_Granularity_11_Mito"                                 ,
 "Nuclei_Granularity_11_RNA"                                  , "Nuclei_Granularity_12_AGP"                                  , "Nuclei_Granularity_12_DNA"                                  , "Nuclei_Granularity_12_ER"                                   ,
 "Nuclei_Granularity_12_Mito"                                 , "Nuclei_Granularity_12_RNA"                                  , "Nuclei_Granularity_13_AGP"                                  , "Nuclei_Granularity_13_DNA"                                  ,
 "Nuclei_Granularity_13_ER"                                   , "Nuclei_Granularity_13_Mito"                                 , "Nuclei_Granularity_13_RNA"                                  , "Nuclei_Granularity_14_AGP"                                  ,
 "Nuclei_Granularity_14_DNA"                                  , "Nuclei_Granularity_14_ER"                                   , "Nuclei_Granularity_14_Mito"                                 , "Nuclei_Granularity_14_RNA"                                  ,
 "Nuclei_Granularity_15_AGP"                                  , "Nuclei_Granularity_15_DNA"                                  , "Nuclei_Granularity_15_ER"                                   , "Nuclei_Granularity_15_Mito"                                 ,
 "Nuclei_Granularity_15_RNA"                                  , "Nuclei_Granularity_16_AGP"                                  , "Nuclei_Granularity_16_DNA"                                  , "Nuclei_Granularity_16_ER"                                   ,
 "Nuclei_Granularity_16_Mito"                                 , "Nuclei_Granularity_16_RNA"                                  , "Nuclei_Granularity_1_AGP"                                   , "Nuclei_Granularity_1_DNA"                                   ,
 "Nuclei_Granularity_1_ER"                                    , "Nuclei_Granularity_1_Mito"                                  , "Nuclei_Granularity_2_AGP"                                   , "Nuclei_Granularity_2_DNA"                                   ,
 "Nuclei_Granularity_2_ER"                                    , "Nuclei_Granularity_2_Mito"                                  , "Nuclei_Granularity_2_RNA"                                   , "Nuclei_Granularity_3_AGP"                                   ,
 "Nuclei_Granularity_3_DNA"                                   , "Nuclei_Granularity_3_ER"                                    , "Nuclei_Granularity_3_Mito"                                  , "Nuclei_Granularity_3_RNA"                                   ,
 "Nuclei_Granularity_4_AGP"                                   , "Nuclei_Granularity_4_DNA"                                   , "Nuclei_Granularity_4_ER"                                    , "Nuclei_Granularity_4_Mito"                                  ,
 "Nuclei_Granularity_4_RNA"                                   , "Nuclei_Granularity_5_AGP"                                   , "Nuclei_Granularity_5_DNA"                                   , "Nuclei_Granularity_5_ER"                                    ,
 "Nuclei_Granularity_5_Mito"                                  , "Nuclei_Granularity_5_RNA"                                   , "Nuclei_Granularity_6_AGP"                                   , "Nuclei_Granularity_6_DNA"                                   ,
 "Nuclei_Granularity_6_ER"                                    , "Nuclei_Granularity_6_Mito"                                  , "Nuclei_Granularity_6_RNA"                                   , "Nuclei_Granularity_7_AGP"                                   ,
 "Nuclei_Granularity_7_DNA"                                   , "Nuclei_Granularity_7_ER"                                    , "Nuclei_Granularity_7_Mito"                                  , "Nuclei_Granularity_7_RNA"                                   ,
 "Nuclei_Granularity_8_AGP"                                   , "Nuclei_Granularity_8_DNA"                                   , "Nuclei_Granularity_8_ER"                                    , "Nuclei_Granularity_8_Mito"                                  ,
 "Nuclei_Granularity_8_RNA"                                   , "Nuclei_Granularity_9_AGP"                                   , "Nuclei_Granularity_9_DNA"                                   , "Nuclei_Granularity_9_ER"                                    ,
 "Nuclei_Granularity_9_Mito"                                  , "Nuclei_Granularity_9_RNA"                                   , "Nuclei_Intensity_IntegratedIntensityEdge_AGP"               , "Nuclei_Intensity_IntegratedIntensityEdge_DNA"               ,
 "Nuclei_Intensity_IntegratedIntensityEdge_ER"                , "Nuclei_Intensity_IntegratedIntensityEdge_RNA"               , "Nuclei_Intensity_IntegratedIntensity_AGP"                   , "Nuclei_Intensity_IntegratedIntensity_DNA"                   ,
 "Nuclei_Intensity_IntegratedIntensity_ER"                    , "Nuclei_Intensity_IntegratedIntensity_Mito"                  , "Nuclei_Intensity_IntegratedIntensity_RNA"                   , "Nuclei_Intensity_LowerQuartileIntensity_AGP"                ,
 "Nuclei_Intensity_LowerQuartileIntensity_DNA"                , "Nuclei_Intensity_LowerQuartileIntensity_ER"                 , "Nuclei_Intensity_MADIntensity_DNA"                          , "Nuclei_Intensity_MADIntensity_Mito"                         ,
 "Nuclei_Intensity_MassDisplacement_AGP"                      , "Nuclei_Intensity_MassDisplacement_DNA"                      , "Nuclei_Intensity_MassDisplacement_ER"                       , "Nuclei_Intensity_MassDisplacement_Mito"                     ,
 "Nuclei_Intensity_MassDisplacement_RNA"                      , "Nuclei_Intensity_MaxIntensityEdge_AGP"                      , "Nuclei_Intensity_MaxIntensityEdge_DNA"                      , "Nuclei_Intensity_MaxIntensityEdge_ER"                       ,
 "Nuclei_Intensity_MaxIntensityEdge_Mito"                     , "Nuclei_Intensity_MaxIntensityEdge_RNA"                      , "Nuclei_Intensity_MaxIntensity_DNA"                          , "Nuclei_Intensity_MaxIntensity_ER"                           ,
 "Nuclei_Intensity_MaxIntensity_RNA"                          , "Nuclei_Intensity_MeanIntensityEdge_AGP"                     , "Nuclei_Intensity_MeanIntensityEdge_DNA"                     , "Nuclei_Intensity_MeanIntensityEdge_ER"                      ,
 "Nuclei_Intensity_MeanIntensityEdge_RNA"                     , "Nuclei_Intensity_MedianIntensity_DNA"                       , "Nuclei_Intensity_MinIntensityEdge_DNA"                      , "Nuclei_Intensity_MinIntensity_AGP"                          ,
 "Nuclei_Intensity_MinIntensity_ER"                           , "Nuclei_Intensity_MinIntensity_Mito"                         , "Nuclei_Intensity_MinIntensity_RNA"                          , "Nuclei_Intensity_StdIntensityEdge_DNA"                      ,
 "Nuclei_Intensity_StdIntensityEdge_RNA"                      , "Nuclei_Intensity_StdIntensity_AGP"                          , "Nuclei_Intensity_StdIntensity_ER"                           , "Nuclei_Intensity_StdIntensity_RNA"                          ,
 "Nuclei_Location_CenterMassIntensity_Z_AGP"                  , "Nuclei_Location_CenterMassIntensity_Z_DNA"                  , "Nuclei_Location_CenterMassIntensity_Z_ER"                   , "Nuclei_Location_CenterMassIntensity_Z_Mito"                 ,
 "Nuclei_Location_CenterMassIntensity_Z_RNA"                  , "Nuclei_Location_Center_Z"                                   , "Nuclei_Location_MaxIntensity_X_Mito"                        , "Nuclei_Location_MaxIntensity_Z_AGP"                         ,
 "Nuclei_Location_MaxIntensity_Z_DNA"                         , "Nuclei_Location_MaxIntensity_Z_ER"                          , "Nuclei_Location_MaxIntensity_Z_Mito"                        , "Nuclei_Location_MaxIntensity_Z_RNA"                         ,
 "Nuclei_Neighbors_AngleBetweenNeighbors_1"                   , "Nuclei_Neighbors_FirstClosestDistance_1"                    , "Nuclei_Neighbors_NumberOfNeighbors_1"                       , "Nuclei_Neighbors_SecondClosestDistance_1"                   ,
 "Nuclei_Number_Object_Number"                                , "Nuclei_ObjectSkeleton_NumberBranchEnds_mito_skel"           , "Nuclei_ObjectSkeleton_NumberNonTrunkBranches_mito_skel"     , "Nuclei_ObjectSkeleton_NumberTrunks_mito_skel"               ,
 "Nuclei_RadialDistribution_FracAtD_AGP_1of4"                 , "Nuclei_RadialDistribution_FracAtD_AGP_3of4"                 , "Nuclei_RadialDistribution_FracAtD_AGP_4of4"                 , "Nuclei_RadialDistribution_FracAtD_DNA_1of4"                 ,
 "Nuclei_RadialDistribution_FracAtD_DNA_3of4"                 , "Nuclei_RadialDistribution_FracAtD_DNA_4of4"                 , "Nuclei_RadialDistribution_FracAtD_ER_1of4"                  , "Nuclei_RadialDistribution_FracAtD_ER_3of4"                  ,
 "Nuclei_RadialDistribution_FracAtD_ER_4of4"                  , "Nuclei_RadialDistribution_FracAtD_Mito_1of4"                , "Nuclei_RadialDistribution_FracAtD_Mito_3of4"                , "Nuclei_RadialDistribution_FracAtD_Mito_4of4"                ,
 "Nuclei_RadialDistribution_FracAtD_RNA_1of4"                 , "Nuclei_RadialDistribution_FracAtD_RNA_3of4"                 , "Nuclei_RadialDistribution_FracAtD_RNA_4of4"                 , "Nuclei_RadialDistribution_MeanFrac_AGP_1of4"                ,
 "Nuclei_RadialDistribution_MeanFrac_AGP_3of4"                , "Nuclei_RadialDistribution_MeanFrac_AGP_4of4"                , "Nuclei_RadialDistribution_MeanFrac_DNA_1of4"                , "Nuclei_RadialDistribution_MeanFrac_DNA_3of4"                ,
 "Nuclei_RadialDistribution_MeanFrac_DNA_4of4"                , "Nuclei_RadialDistribution_MeanFrac_ER_1of4"                 , "Nuclei_RadialDistribution_MeanFrac_ER_3of4"                 , "Nuclei_RadialDistribution_MeanFrac_ER_4of4"                 ,
 "Nuclei_RadialDistribution_MeanFrac_Mito_2of4"               , "Nuclei_RadialDistribution_MeanFrac_Mito_3of4"               , "Nuclei_RadialDistribution_MeanFrac_Mito_4of4"               , "Nuclei_RadialDistribution_MeanFrac_RNA_1of4"                ,
 "Nuclei_RadialDistribution_MeanFrac_RNA_3of4"                , "Nuclei_RadialDistribution_MeanFrac_RNA_4of4"                , "Nuclei_RadialDistribution_RadialCV_AGP_1of4"                , "Nuclei_RadialDistribution_RadialCV_AGP_2of4"                ,
 "Nuclei_RadialDistribution_RadialCV_AGP_3of4"                , "Nuclei_RadialDistribution_RadialCV_AGP_4of4"                , "Nuclei_RadialDistribution_RadialCV_DNA_1of4"                , "Nuclei_RadialDistribution_RadialCV_DNA_2of4"                ,
 "Nuclei_RadialDistribution_RadialCV_DNA_3of4"                , "Nuclei_RadialDistribution_RadialCV_DNA_4of4"                , "Nuclei_RadialDistribution_RadialCV_ER_1of4"                 , "Nuclei_RadialDistribution_RadialCV_ER_2of4"                 ,
 "Nuclei_RadialDistribution_RadialCV_ER_3of4"                 , "Nuclei_RadialDistribution_RadialCV_ER_4of4"                 , "Nuclei_RadialDistribution_RadialCV_Mito_1of4"               , "Nuclei_RadialDistribution_RadialCV_Mito_2of4"               ,
 "Nuclei_RadialDistribution_RadialCV_Mito_3of4"               , "Nuclei_RadialDistribution_RadialCV_Mito_4of4"               , "Nuclei_RadialDistribution_RadialCV_RNA_1of4"                , "Nuclei_RadialDistribution_RadialCV_RNA_2of4"                ,
 "Nuclei_RadialDistribution_RadialCV_RNA_3of4"                , "Nuclei_RadialDistribution_RadialCV_RNA_4of4"               } 

correlation_dropped_cols_5 = \
 {"Nuclei_AreaShape_Area"                                      , "Nuclei_AreaShape_BoundingBoxMaximum_X"                      , "Nuclei_AreaShape_BoundingBoxMinimum_X"                      , "Nuclei_AreaShape_BoundingBoxMinimum_Y"                      ,
 "Nuclei_AreaShape_Center_X"                                  , "Nuclei_AreaShape_Center_Y"                                  , "Nuclei_AreaShape_EquivalentDiameter"                        , "Nuclei_AreaShape_MajorAxisLength"                           ,
 "Nuclei_AreaShape_MaxFeretDiameter"                          , "Nuclei_AreaShape_MaximumRadius"                             , "Nuclei_AreaShape_MeanRadius"                                , "Nuclei_AreaShape_MedianRadius"                              ,
 "Nuclei_AreaShape_MinorAxisLength"                           , "Nuclei_AreaShape_Perimeter"                                 , "Nuclei_Correlation_Manders_AGP_DNA"                         , "Nuclei_Correlation_Manders_AGP_Mito"                        ,
 "Nuclei_Correlation_Manders_DNA_AGP"                         , "Nuclei_Correlation_Manders_DNA_Mito"                        , "Nuclei_Correlation_Manders_ER_DNA"                          , "Nuclei_Correlation_Manders_ER_Mito"                         ,
 "Nuclei_Correlation_Manders_RNA_AGP"                         , "Nuclei_Correlation_Manders_RNA_DNA"                         , "Nuclei_Correlation_RWC_RNA_ER"                              , "Nuclei_Granularity_1_RNA"                                   ,
 "Nuclei_Intensity_IntegratedIntensityEdge_Mito"              , "Nuclei_Intensity_LowerQuartileIntensity_Mito"               , "Nuclei_Intensity_LowerQuartileIntensity_RNA"                , "Nuclei_Intensity_MADIntensity_AGP"                          ,
 "Nuclei_Intensity_MADIntensity_ER"                           , "Nuclei_Intensity_MADIntensity_RNA"                          , "Nuclei_Intensity_MaxIntensity_AGP"                          , "Nuclei_Intensity_MaxIntensity_Mito"                         ,
 "Nuclei_Intensity_MeanIntensityEdge_Mito"                    , "Nuclei_Intensity_MeanIntensity_AGP"                         , "Nuclei_Intensity_MeanIntensity_DNA"                         , "Nuclei_Intensity_MeanIntensity_ER"                          ,
 "Nuclei_Intensity_MeanIntensity_Mito"                        , "Nuclei_Intensity_MeanIntensity_RNA"                         , "Nuclei_Intensity_MedianIntensity_AGP"                       , "Nuclei_Intensity_MedianIntensity_ER"                        ,
 "Nuclei_Intensity_MedianIntensity_Mito"                      , "Nuclei_Intensity_MedianIntensity_RNA"                       , "Nuclei_Intensity_MinIntensityEdge_AGP"                      , "Nuclei_Intensity_MinIntensityEdge_ER"                       ,
 "Nuclei_Intensity_MinIntensityEdge_Mito"                     , "Nuclei_Intensity_MinIntensityEdge_RNA"                      , "Nuclei_Intensity_MinIntensity_DNA"                          , "Nuclei_Intensity_StdIntensityEdge_AGP"                      ,
 "Nuclei_Intensity_StdIntensityEdge_ER"                       , "Nuclei_Intensity_StdIntensityEdge_Mito"                     , "Nuclei_Intensity_StdIntensity_DNA"                          , "Nuclei_Intensity_StdIntensity_Mito"                         ,
 "Nuclei_Intensity_UpperQuartileIntensity_AGP"                , "Nuclei_Intensity_UpperQuartileIntensity_DNA"                , "Nuclei_Intensity_UpperQuartileIntensity_ER"                 , "Nuclei_Intensity_UpperQuartileIntensity_Mito"               ,
 "Nuclei_Intensity_UpperQuartileIntensity_RNA"                , "Nuclei_Location_CenterMassIntensity_X_AGP"                  , "Nuclei_Location_CenterMassIntensity_X_DNA"                  , "Nuclei_Location_CenterMassIntensity_X_ER"                   ,
 "Nuclei_Location_CenterMassIntensity_X_Mito"                 , "Nuclei_Location_CenterMassIntensity_X_RNA"                  , "Nuclei_Location_CenterMassIntensity_Y_AGP"                  , "Nuclei_Location_CenterMassIntensity_Y_DNA"                  ,
 "Nuclei_Location_CenterMassIntensity_Y_ER"                   , "Nuclei_Location_CenterMassIntensity_Y_Mito"                 , "Nuclei_Location_CenterMassIntensity_Y_RNA"                  , "Nuclei_Location_Center_X"                                   ,
 "Nuclei_Location_Center_Y"                                   , "Nuclei_Location_MaxIntensity_X_AGP"                         , "Nuclei_Location_MaxIntensity_X_DNA"                         , "Nuclei_Location_MaxIntensity_X_ER"                          ,
 "Nuclei_Location_MaxIntensity_X_RNA"                         , "Nuclei_Location_MaxIntensity_Y_AGP"                         , "Nuclei_Location_MaxIntensity_Y_DNA"                         , "Nuclei_Location_MaxIntensity_Y_ER"                          ,
 "Nuclei_Location_MaxIntensity_Y_Mito"                        , "Nuclei_Location_MaxIntensity_Y_RNA"                         , "Nuclei_Neighbors_FirstClosestObjectNumber_1"                , "Nuclei_Neighbors_PercentTouching_1"                         ,
 "Nuclei_Neighbors_SecondClosestObjectNumber_1"               , "Nuclei_ObjectSkeleton_TotalObjectSkeletonLength_mito_skel"  , "Nuclei_Parent_NucleiIncludingEdges"                         , "Nuclei_RadialDistribution_FracAtD_AGP_2of4"                 ,
 "Nuclei_RadialDistribution_FracAtD_DNA_2of4"                 , "Nuclei_RadialDistribution_FracAtD_ER_2of4"                  , "Nuclei_RadialDistribution_FracAtD_Mito_2of4"                , "Nuclei_RadialDistribution_FracAtD_RNA_2of4"                 ,
 "Nuclei_RadialDistribution_MeanFrac_AGP_2of4"                , "Nuclei_RadialDistribution_MeanFrac_DNA_2of4"                , "Nuclei_RadialDistribution_MeanFrac_ER_2of4"                 , "Nuclei_RadialDistribution_MeanFrac_Mito_1of4"               ,
 "Nuclei_RadialDistribution_MeanFrac_RNA_2of4"               } 

variance_dropped_cols_5 = set()


In [285]:
variance_dropped_cols_5 & correlation_dropped_cols_5
print(f" Number of cols dropped - high correlation: {len(correlation_dropped_cols_5)}")
print(f" Number of cols dropped - low variance    : {len(variance_dropped_cols_5)}")
print(f" Number of selected cols                  : {len(selected_cols_5)}")
print(f" Total                                    : {len(selected_cols_5)+len(correlation_dropped_cols_5)+len(variance_dropped_cols_5)}")

set()

 Number of cols dropped - high correlation: 93
 Number of cols dropped - low variance    : 0
 Number of selected cols                  : 302
 Total                                    : 395


In [286]:
(correlation_dropped_cols_5) & set(zero_features)
(variance_dropped_cols_5) & set(zero_features)
(selected_cols_5) & set(zero_features)

set()

set()

{'Nuclei_Location_CenterMassIntensity_Z_AGP',
 'Nuclei_Location_CenterMassIntensity_Z_DNA',
 'Nuclei_Location_CenterMassIntensity_Z_ER',
 'Nuclei_Location_CenterMassIntensity_Z_Mito',
 'Nuclei_Location_CenterMassIntensity_Z_RNA',
 'Nuclei_Location_Center_Z',
 'Nuclei_Location_MaxIntensity_Z_AGP',
 'Nuclei_Location_MaxIntensity_Z_DNA',
 'Nuclei_Location_MaxIntensity_Z_ER',
 'Nuclei_Location_MaxIntensity_Z_Mito',
 'Nuclei_Location_MaxIntensity_Z_RNA'}

### Group 6 - Nuclei

In [287]:
selected_cols_6 = \
{"Nuclei_Texture_AngularSecondMoment_AGP_10_01_256"           , "Nuclei_Texture_AngularSecondMoment_AGP_3_00_256"            , "Nuclei_Texture_AngularSecondMoment_DNA_10_01_256"           , "Nuclei_Texture_AngularSecondMoment_DNA_10_02_256"           ,
 "Nuclei_Texture_AngularSecondMoment_DNA_10_03_256"           , "Nuclei_Texture_AngularSecondMoment_ER_10_01_256"            , "Nuclei_Texture_AngularSecondMoment_ER_5_03_256"             , "Nuclei_Texture_AngularSecondMoment_Mito_10_01_256"          ,
 "Nuclei_Texture_AngularSecondMoment_RNA_10_00_256"           , "Nuclei_Texture_AngularSecondMoment_RNA_10_01_256"           , "Nuclei_Texture_AngularSecondMoment_RNA_10_03_256"           , "Nuclei_Texture_Contrast_DNA_5_00_256"                       ,
 "Nuclei_Texture_Contrast_Mito_3_00_256"                      , "Nuclei_Texture_Contrast_RNA_3_02_256"                       , "Nuclei_Texture_Correlation_AGP_10_00_256"                   , "Nuclei_Texture_Correlation_AGP_10_01_256"                   ,
 "Nuclei_Texture_Correlation_AGP_10_02_256"                   , "Nuclei_Texture_Correlation_AGP_10_03_256"                   , "Nuclei_Texture_Correlation_AGP_3_00_256"                    , "Nuclei_Texture_Correlation_AGP_3_01_256"                    ,
 "Nuclei_Texture_Correlation_AGP_3_02_256"                    , "Nuclei_Texture_Correlation_AGP_3_03_256"                    , "Nuclei_Texture_Correlation_AGP_5_00_256"                    , "Nuclei_Texture_Correlation_AGP_5_01_256"                    ,
 "Nuclei_Texture_Correlation_AGP_5_02_256"                    , "Nuclei_Texture_Correlation_AGP_5_03_256"                    , "Nuclei_Texture_Correlation_DNA_10_00_256"                   , "Nuclei_Texture_Correlation_DNA_10_01_256"                   ,
 "Nuclei_Texture_Correlation_DNA_10_02_256"                   , "Nuclei_Texture_Correlation_DNA_10_03_256"                   , "Nuclei_Texture_Correlation_DNA_3_00_256"                    , "Nuclei_Texture_Correlation_DNA_3_01_256"                    ,
 "Nuclei_Texture_Correlation_DNA_3_02_256"                    , "Nuclei_Texture_Correlation_DNA_3_03_256"                    , "Nuclei_Texture_Correlation_DNA_5_00_256"                    , "Nuclei_Texture_Correlation_DNA_5_01_256"                    ,
 "Nuclei_Texture_Correlation_DNA_5_02_256"                    , "Nuclei_Texture_Correlation_DNA_5_03_256"                    , "Nuclei_Texture_Correlation_ER_10_00_256"                    , "Nuclei_Texture_Correlation_ER_10_01_256"                    ,
 "Nuclei_Texture_Correlation_ER_10_02_256"                    , "Nuclei_Texture_Correlation_ER_10_03_256"                    , "Nuclei_Texture_Correlation_ER_3_00_256"                     , "Nuclei_Texture_Correlation_ER_3_01_256"                     ,
 "Nuclei_Texture_Correlation_ER_3_02_256"                     , "Nuclei_Texture_Correlation_ER_3_03_256"                     , "Nuclei_Texture_Correlation_ER_5_00_256"                     , "Nuclei_Texture_Correlation_ER_5_01_256"                     ,
 "Nuclei_Texture_Correlation_ER_5_02_256"                     , "Nuclei_Texture_Correlation_ER_5_03_256"                     , "Nuclei_Texture_Correlation_Mito_10_00_256"                  , "Nuclei_Texture_Correlation_Mito_10_01_256"                  ,
 "Nuclei_Texture_Correlation_Mito_10_02_256"                  , "Nuclei_Texture_Correlation_Mito_10_03_256"                  , "Nuclei_Texture_Correlation_Mito_3_00_256"                   , "Nuclei_Texture_Correlation_Mito_3_01_256"                   ,
 "Nuclei_Texture_Correlation_Mito_3_02_256"                   , "Nuclei_Texture_Correlation_Mito_3_03_256"                   , "Nuclei_Texture_Correlation_Mito_5_00_256"                   , "Nuclei_Texture_Correlation_Mito_5_01_256"                   ,
 "Nuclei_Texture_Correlation_Mito_5_02_256"                   , "Nuclei_Texture_Correlation_Mito_5_03_256"                   , "Nuclei_Texture_Correlation_RNA_10_00_256"                   , "Nuclei_Texture_Correlation_RNA_10_01_256"                   ,
 "Nuclei_Texture_Correlation_RNA_10_02_256"                   , "Nuclei_Texture_Correlation_RNA_10_03_256"                   , "Nuclei_Texture_Correlation_RNA_3_00_256"                    , "Nuclei_Texture_Correlation_RNA_3_01_256"                    ,
 "Nuclei_Texture_Correlation_RNA_3_02_256"                    , "Nuclei_Texture_Correlation_RNA_3_03_256"                    , "Nuclei_Texture_Correlation_RNA_5_00_256"                    , "Nuclei_Texture_Correlation_RNA_5_01_256"                    ,
 "Nuclei_Texture_Correlation_RNA_5_02_256"                    , "Nuclei_Texture_Correlation_RNA_5_03_256"                    , "Nuclei_Texture_DifferenceEntropy_DNA_3_02_256"              , "Nuclei_Texture_DifferenceEntropy_ER_3_00_256"               ,
 "Nuclei_Texture_DifferenceEntropy_RNA_3_00_256"              , "Nuclei_Texture_DifferenceVariance_AGP_10_03_256"            , "Nuclei_Texture_DifferenceVariance_DNA_10_01_256"            , "Nuclei_Texture_DifferenceVariance_ER_10_03_256"             ,
 "Nuclei_Texture_DifferenceVariance_ER_3_00_256"              , "Nuclei_Texture_DifferenceVariance_Mito_10_03_256"           , "Nuclei_Texture_DifferenceVariance_RNA_10_03_256"            , "Nuclei_Texture_Entropy_AGP_10_03_256"                       ,
 "Nuclei_Texture_Entropy_DNA_10_01_256"                       , "Nuclei_Texture_Entropy_ER_10_01_256"                        , "Nuclei_Texture_Entropy_Mito_10_03_256"                      , "Nuclei_Texture_Entropy_RNA_10_03_256"                       ,
 "Nuclei_Texture_InfoMeas1_AGP_10_01_256"                     , "Nuclei_Texture_InfoMeas1_AGP_10_02_256"                     , "Nuclei_Texture_InfoMeas1_AGP_10_03_256"                     , "Nuclei_Texture_InfoMeas1_AGP_3_00_256"                      ,
 "Nuclei_Texture_InfoMeas1_AGP_3_01_256"                      , "Nuclei_Texture_InfoMeas1_AGP_3_02_256"                      , "Nuclei_Texture_InfoMeas1_AGP_3_03_256"                      , "Nuclei_Texture_InfoMeas1_AGP_5_00_256"                      ,
 "Nuclei_Texture_InfoMeas1_AGP_5_01_256"                      , "Nuclei_Texture_InfoMeas1_AGP_5_02_256"                      , "Nuclei_Texture_InfoMeas1_AGP_5_03_256"                      , "Nuclei_Texture_InfoMeas1_DNA_10_01_256"                     ,
 "Nuclei_Texture_InfoMeas1_DNA_10_02_256"                     , "Nuclei_Texture_InfoMeas1_DNA_10_03_256"                     , "Nuclei_Texture_InfoMeas1_DNA_3_00_256"                      , "Nuclei_Texture_InfoMeas1_DNA_3_01_256"                      ,
 "Nuclei_Texture_InfoMeas1_DNA_3_02_256"                      , "Nuclei_Texture_InfoMeas1_DNA_3_03_256"                      , "Nuclei_Texture_InfoMeas1_DNA_5_00_256"                      , "Nuclei_Texture_InfoMeas1_DNA_5_02_256"                      ,
 "Nuclei_Texture_InfoMeas1_DNA_5_03_256"                      , "Nuclei_Texture_InfoMeas1_ER_10_01_256"                      , "Nuclei_Texture_InfoMeas1_ER_10_03_256"                      , "Nuclei_Texture_InfoMeas1_ER_3_00_256"                       ,
 "Nuclei_Texture_InfoMeas1_ER_3_01_256"                       , "Nuclei_Texture_InfoMeas1_ER_3_02_256"                       , "Nuclei_Texture_InfoMeas1_ER_3_03_256"                       , "Nuclei_Texture_InfoMeas1_ER_5_03_256"                       ,
 "Nuclei_Texture_InfoMeas1_Mito_10_03_256"                    , "Nuclei_Texture_InfoMeas1_Mito_3_00_256"                     , "Nuclei_Texture_InfoMeas1_Mito_3_01_256"                     , "Nuclei_Texture_InfoMeas1_Mito_3_02_256"                     ,
 "Nuclei_Texture_InfoMeas1_Mito_5_03_256"                     , "Nuclei_Texture_InfoMeas1_RNA_10_01_256"                     , "Nuclei_Texture_InfoMeas1_RNA_10_03_256"                     , "Nuclei_Texture_InfoMeas1_RNA_3_00_256"                      ,
 "Nuclei_Texture_InfoMeas1_RNA_3_01_256"                      , "Nuclei_Texture_InfoMeas1_RNA_3_02_256"                      , "Nuclei_Texture_InfoMeas1_RNA_3_03_256"                      , "Nuclei_Texture_InfoMeas1_RNA_5_00_256"                      ,
 "Nuclei_Texture_InfoMeas1_RNA_5_01_256"                      , "Nuclei_Texture_InfoMeas1_RNA_5_02_256"                      , "Nuclei_Texture_InfoMeas1_RNA_5_03_256"                      , "Nuclei_Texture_InfoMeas2_AGP_10_01_256"                     ,
 "Nuclei_Texture_InfoMeas2_AGP_3_00_256"                      , "Nuclei_Texture_InfoMeas2_AGP_3_02_256"                      , "Nuclei_Texture_InfoMeas2_DNA_10_03_256"                     , "Nuclei_Texture_InfoMeas2_DNA_3_02_256"                      ,
 "Nuclei_Texture_InfoMeas2_ER_10_03_256"                      , "Nuclei_Texture_InfoMeas2_ER_3_00_256"                       , "Nuclei_Texture_InfoMeas2_ER_3_02_256"                       , "Nuclei_Texture_InfoMeas2_Mito_10_01_256"                    ,
 "Nuclei_Texture_InfoMeas2_Mito_3_00_256"                     , "Nuclei_Texture_InfoMeas2_RNA_10_01_256"                     , "Nuclei_Texture_InfoMeas2_RNA_3_00_256"                      , "Nuclei_Texture_InverseDifferenceMoment_AGP_3_00_256"        ,
 "Nuclei_Texture_InverseDifferenceMoment_DNA_10_01_256"       , "Nuclei_Texture_InverseDifferenceMoment_DNA_3_02_256"        , "Nuclei_Texture_InverseDifferenceMoment_ER_3_00_256"         , "Nuclei_Texture_InverseDifferenceMoment_Mito_10_01_256"      ,
 "Nuclei_Texture_InverseDifferenceMoment_RNA_5_03_256"        , "Nuclei_Texture_SumAverage_AGP_10_01_256"                    , "Nuclei_Texture_SumAverage_DNA_10_01_256"                    , "Nuclei_Texture_SumAverage_ER_10_01_256"                     ,
 "Nuclei_Texture_SumAverage_Mito_10_01_256"                   , "Nuclei_Texture_SumAverage_RNA_10_01_256"                    , "Nuclei_Texture_SumEntropy_DNA_10_03_256"                    , "Nuclei_Texture_SumVariance_AGP_10_03_256"                   ,
 "Nuclei_Texture_SumVariance_DNA_10_03_256"                   , "Nuclei_Texture_SumVariance_ER_10_01_256"                    , "Nuclei_Texture_SumVariance_Mito_10_03_256"                  , "Nuclei_Texture_SumVariance_RNA_10_01_256"                  
 }

correlation_dropped_cols_6 = \
{"Nuclei_Texture_AngularSecondMoment_AGP_10_00_256"           , "Nuclei_Texture_AngularSecondMoment_AGP_10_02_256"           , "Nuclei_Texture_AngularSecondMoment_AGP_10_03_256"           , "Nuclei_Texture_AngularSecondMoment_AGP_3_01_256"            ,
 "Nuclei_Texture_AngularSecondMoment_AGP_3_02_256"            , "Nuclei_Texture_AngularSecondMoment_AGP_3_03_256"            , "Nuclei_Texture_AngularSecondMoment_AGP_5_00_256"            , "Nuclei_Texture_AngularSecondMoment_AGP_5_01_256"            ,
 "Nuclei_Texture_AngularSecondMoment_AGP_5_02_256"            , "Nuclei_Texture_AngularSecondMoment_AGP_5_03_256"            , "Nuclei_Texture_AngularSecondMoment_DNA_10_00_256"           , "Nuclei_Texture_AngularSecondMoment_DNA_3_00_256"            ,
 "Nuclei_Texture_AngularSecondMoment_DNA_3_01_256"            , "Nuclei_Texture_AngularSecondMoment_DNA_3_02_256"            , "Nuclei_Texture_AngularSecondMoment_DNA_3_03_256"            , "Nuclei_Texture_AngularSecondMoment_DNA_5_00_256"            ,
 "Nuclei_Texture_AngularSecondMoment_DNA_5_01_256"            , "Nuclei_Texture_AngularSecondMoment_DNA_5_02_256"            , "Nuclei_Texture_AngularSecondMoment_DNA_5_03_256"            , "Nuclei_Texture_AngularSecondMoment_ER_10_00_256"            ,
 "Nuclei_Texture_AngularSecondMoment_ER_10_02_256"            , "Nuclei_Texture_AngularSecondMoment_ER_10_03_256"            , "Nuclei_Texture_AngularSecondMoment_ER_3_00_256"             , "Nuclei_Texture_AngularSecondMoment_ER_3_01_256"             ,
 "Nuclei_Texture_AngularSecondMoment_ER_3_02_256"             , "Nuclei_Texture_AngularSecondMoment_ER_3_03_256"             , "Nuclei_Texture_AngularSecondMoment_ER_5_00_256"             , "Nuclei_Texture_AngularSecondMoment_ER_5_01_256"             ,
 "Nuclei_Texture_AngularSecondMoment_ER_5_02_256"             , "Nuclei_Texture_AngularSecondMoment_Mito_10_00_256"          , "Nuclei_Texture_AngularSecondMoment_Mito_10_02_256"          , "Nuclei_Texture_AngularSecondMoment_Mito_10_03_256"          ,
 "Nuclei_Texture_AngularSecondMoment_Mito_3_00_256"           , "Nuclei_Texture_AngularSecondMoment_Mito_3_01_256"           , "Nuclei_Texture_AngularSecondMoment_Mito_3_02_256"           , "Nuclei_Texture_AngularSecondMoment_Mito_3_03_256"           ,
 "Nuclei_Texture_AngularSecondMoment_Mito_5_00_256"           , "Nuclei_Texture_AngularSecondMoment_Mito_5_01_256"           , "Nuclei_Texture_AngularSecondMoment_Mito_5_02_256"           , "Nuclei_Texture_AngularSecondMoment_Mito_5_03_256"           ,
 "Nuclei_Texture_AngularSecondMoment_RNA_10_02_256"           , "Nuclei_Texture_AngularSecondMoment_RNA_3_00_256"            , "Nuclei_Texture_AngularSecondMoment_RNA_3_01_256"            , "Nuclei_Texture_AngularSecondMoment_RNA_3_02_256"            ,
 "Nuclei_Texture_AngularSecondMoment_RNA_3_03_256"            , "Nuclei_Texture_AngularSecondMoment_RNA_5_00_256"            , "Nuclei_Texture_AngularSecondMoment_RNA_5_01_256"            , "Nuclei_Texture_AngularSecondMoment_RNA_5_02_256"            ,
 "Nuclei_Texture_AngularSecondMoment_RNA_5_03_256"            , "Nuclei_Texture_Contrast_AGP_10_00_256"                      , "Nuclei_Texture_Contrast_AGP_10_01_256"                      , "Nuclei_Texture_Contrast_AGP_10_02_256"                      ,
 "Nuclei_Texture_Contrast_AGP_10_03_256"                      , "Nuclei_Texture_Contrast_AGP_3_00_256"                       , "Nuclei_Texture_Contrast_AGP_3_01_256"                       , "Nuclei_Texture_Contrast_AGP_3_02_256"                       ,
 "Nuclei_Texture_Contrast_AGP_3_03_256"                       , "Nuclei_Texture_Contrast_AGP_5_00_256"                       , "Nuclei_Texture_Contrast_AGP_5_01_256"                       , "Nuclei_Texture_Contrast_AGP_5_02_256"                       ,
 "Nuclei_Texture_Contrast_AGP_5_03_256"                       , "Nuclei_Texture_Contrast_DNA_10_00_256"                      , "Nuclei_Texture_Contrast_DNA_10_01_256"                      , "Nuclei_Texture_Contrast_DNA_10_02_256"                      ,
 "Nuclei_Texture_Contrast_DNA_10_03_256"                      , "Nuclei_Texture_Contrast_DNA_3_00_256"                       , "Nuclei_Texture_Contrast_DNA_3_01_256"                       , "Nuclei_Texture_Contrast_DNA_3_02_256"                       ,
 "Nuclei_Texture_Contrast_DNA_3_03_256"                       , "Nuclei_Texture_Contrast_DNA_5_01_256"                       , "Nuclei_Texture_Contrast_DNA_5_02_256"                       , "Nuclei_Texture_Contrast_DNA_5_03_256"                       ,
 "Nuclei_Texture_Contrast_ER_10_00_256"                       , "Nuclei_Texture_Contrast_ER_10_01_256"                       , "Nuclei_Texture_Contrast_ER_10_02_256"                       , "Nuclei_Texture_Contrast_ER_10_03_256"                       ,
 "Nuclei_Texture_Contrast_ER_3_00_256"                        , "Nuclei_Texture_Contrast_ER_3_01_256"                        , "Nuclei_Texture_Contrast_ER_3_02_256"                        , "Nuclei_Texture_Contrast_ER_3_03_256"                        ,
 "Nuclei_Texture_Contrast_ER_5_00_256"                        , "Nuclei_Texture_Contrast_ER_5_01_256"                        , "Nuclei_Texture_Contrast_ER_5_02_256"                        , "Nuclei_Texture_Contrast_ER_5_03_256"                        ,
 "Nuclei_Texture_Contrast_Mito_10_00_256"                     , "Nuclei_Texture_Contrast_Mito_10_01_256"                     , "Nuclei_Texture_Contrast_Mito_10_02_256"                     , "Nuclei_Texture_Contrast_Mito_10_03_256"                     ,
 "Nuclei_Texture_Contrast_Mito_3_01_256"                      , "Nuclei_Texture_Contrast_Mito_3_02_256"                      , "Nuclei_Texture_Contrast_Mito_3_03_256"                      , "Nuclei_Texture_Contrast_Mito_5_00_256"                      ,
 "Nuclei_Texture_Contrast_Mito_5_01_256"                      , "Nuclei_Texture_Contrast_Mito_5_02_256"                      , "Nuclei_Texture_Contrast_Mito_5_03_256"                      , "Nuclei_Texture_Contrast_RNA_10_00_256"                      ,
 "Nuclei_Texture_Contrast_RNA_10_01_256"                      , "Nuclei_Texture_Contrast_RNA_10_02_256"                      , "Nuclei_Texture_Contrast_RNA_10_03_256"                      , "Nuclei_Texture_Contrast_RNA_3_00_256"                       ,
 "Nuclei_Texture_Contrast_RNA_3_01_256"                       , "Nuclei_Texture_Contrast_RNA_3_03_256"                       , "Nuclei_Texture_Contrast_RNA_5_00_256"                       , "Nuclei_Texture_Contrast_RNA_5_01_256"                       ,
 "Nuclei_Texture_Contrast_RNA_5_02_256"                       , "Nuclei_Texture_Contrast_RNA_5_03_256"                       , "Nuclei_Texture_DifferenceEntropy_AGP_10_00_256"             , "Nuclei_Texture_DifferenceEntropy_AGP_10_01_256"             ,
 "Nuclei_Texture_DifferenceEntropy_AGP_10_02_256"             , "Nuclei_Texture_DifferenceEntropy_AGP_10_03_256"             , "Nuclei_Texture_DifferenceEntropy_AGP_3_00_256"              , "Nuclei_Texture_DifferenceEntropy_AGP_3_01_256"              ,
 "Nuclei_Texture_DifferenceEntropy_AGP_3_02_256"              , "Nuclei_Texture_DifferenceEntropy_AGP_3_03_256"              , "Nuclei_Texture_DifferenceEntropy_AGP_5_00_256"              , "Nuclei_Texture_DifferenceEntropy_AGP_5_01_256"              ,
 "Nuclei_Texture_DifferenceEntropy_AGP_5_02_256"              , "Nuclei_Texture_DifferenceEntropy_AGP_5_03_256"              , "Nuclei_Texture_DifferenceEntropy_DNA_10_00_256"             , "Nuclei_Texture_DifferenceEntropy_DNA_10_01_256"             ,
 "Nuclei_Texture_DifferenceEntropy_DNA_10_02_256"             , "Nuclei_Texture_DifferenceEntropy_DNA_10_03_256"             , "Nuclei_Texture_DifferenceEntropy_DNA_3_00_256"              , "Nuclei_Texture_DifferenceEntropy_DNA_3_01_256"              ,
 "Nuclei_Texture_DifferenceEntropy_DNA_3_03_256"              , "Nuclei_Texture_DifferenceEntropy_DNA_5_00_256"              , "Nuclei_Texture_DifferenceEntropy_DNA_5_01_256"              , "Nuclei_Texture_DifferenceEntropy_DNA_5_02_256"              ,
 "Nuclei_Texture_DifferenceEntropy_DNA_5_03_256"              , "Nuclei_Texture_DifferenceEntropy_ER_10_00_256"              , "Nuclei_Texture_DifferenceEntropy_ER_10_01_256"              , "Nuclei_Texture_DifferenceEntropy_ER_10_02_256"              ,
 "Nuclei_Texture_DifferenceEntropy_ER_10_03_256"              , "Nuclei_Texture_DifferenceEntropy_ER_3_01_256"               , "Nuclei_Texture_DifferenceEntropy_ER_3_02_256"               , "Nuclei_Texture_DifferenceEntropy_ER_3_03_256"               ,
 "Nuclei_Texture_DifferenceEntropy_ER_5_00_256"               , "Nuclei_Texture_DifferenceEntropy_ER_5_01_256"               , "Nuclei_Texture_DifferenceEntropy_ER_5_02_256"               , "Nuclei_Texture_DifferenceEntropy_ER_5_03_256"               ,
 "Nuclei_Texture_DifferenceEntropy_Mito_10_00_256"            , "Nuclei_Texture_DifferenceEntropy_Mito_10_01_256"            , "Nuclei_Texture_DifferenceEntropy_Mito_10_02_256"            , "Nuclei_Texture_DifferenceEntropy_Mito_10_03_256"            ,
 "Nuclei_Texture_DifferenceEntropy_Mito_3_00_256"             , "Nuclei_Texture_DifferenceEntropy_Mito_3_01_256"             , "Nuclei_Texture_DifferenceEntropy_Mito_3_02_256"             , "Nuclei_Texture_DifferenceEntropy_Mito_3_03_256"             ,
 "Nuclei_Texture_DifferenceEntropy_Mito_5_00_256"             , "Nuclei_Texture_DifferenceEntropy_Mito_5_01_256"             , "Nuclei_Texture_DifferenceEntropy_Mito_5_02_256"             , "Nuclei_Texture_DifferenceEntropy_Mito_5_03_256"             ,
 "Nuclei_Texture_DifferenceEntropy_RNA_10_00_256"             , "Nuclei_Texture_DifferenceEntropy_RNA_10_01_256"             , "Nuclei_Texture_DifferenceEntropy_RNA_10_02_256"             , "Nuclei_Texture_DifferenceEntropy_RNA_10_03_256"             ,
 "Nuclei_Texture_DifferenceEntropy_RNA_3_01_256"              , "Nuclei_Texture_DifferenceEntropy_RNA_3_02_256"              , "Nuclei_Texture_DifferenceEntropy_RNA_3_03_256"              , "Nuclei_Texture_DifferenceEntropy_RNA_5_00_256"              ,
 "Nuclei_Texture_DifferenceEntropy_RNA_5_01_256"              , "Nuclei_Texture_DifferenceEntropy_RNA_5_02_256"              , "Nuclei_Texture_DifferenceEntropy_RNA_5_03_256"              , "Nuclei_Texture_DifferenceVariance_AGP_10_00_256"            ,
 "Nuclei_Texture_DifferenceVariance_AGP_10_01_256"            , "Nuclei_Texture_DifferenceVariance_AGP_10_02_256"            , "Nuclei_Texture_DifferenceVariance_AGP_3_00_256"             , "Nuclei_Texture_DifferenceVariance_AGP_3_01_256"             ,
 "Nuclei_Texture_DifferenceVariance_AGP_3_02_256"             , "Nuclei_Texture_DifferenceVariance_AGP_3_03_256"             , "Nuclei_Texture_DifferenceVariance_AGP_5_00_256"             , "Nuclei_Texture_DifferenceVariance_AGP_5_01_256"             ,
 "Nuclei_Texture_DifferenceVariance_AGP_5_02_256"             , "Nuclei_Texture_DifferenceVariance_AGP_5_03_256"             , "Nuclei_Texture_DifferenceVariance_DNA_10_00_256"            , "Nuclei_Texture_DifferenceVariance_DNA_10_02_256"            ,
 "Nuclei_Texture_DifferenceVariance_DNA_10_03_256"            , "Nuclei_Texture_DifferenceVariance_DNA_3_00_256"             , "Nuclei_Texture_DifferenceVariance_DNA_3_01_256"             , "Nuclei_Texture_DifferenceVariance_DNA_3_02_256"             ,
 "Nuclei_Texture_DifferenceVariance_DNA_3_03_256"             , "Nuclei_Texture_DifferenceVariance_DNA_5_00_256"             , "Nuclei_Texture_DifferenceVariance_DNA_5_01_256"             , "Nuclei_Texture_DifferenceVariance_DNA_5_02_256"             ,
 "Nuclei_Texture_DifferenceVariance_DNA_5_03_256"             , "Nuclei_Texture_DifferenceVariance_ER_10_00_256"             , "Nuclei_Texture_DifferenceVariance_ER_10_01_256"             , "Nuclei_Texture_DifferenceVariance_ER_10_02_256"             ,
 "Nuclei_Texture_DifferenceVariance_ER_3_01_256"              , "Nuclei_Texture_DifferenceVariance_ER_3_02_256"              , "Nuclei_Texture_DifferenceVariance_ER_3_03_256"              , "Nuclei_Texture_DifferenceVariance_ER_5_00_256"              ,
 "Nuclei_Texture_DifferenceVariance_ER_5_01_256"              , "Nuclei_Texture_DifferenceVariance_ER_5_02_256"              , "Nuclei_Texture_DifferenceVariance_ER_5_03_256"              , "Nuclei_Texture_DifferenceVariance_Mito_10_00_256"           ,
 "Nuclei_Texture_DifferenceVariance_Mito_10_01_256"           , "Nuclei_Texture_DifferenceVariance_Mito_10_02_256"           , "Nuclei_Texture_DifferenceVariance_Mito_3_00_256"            , "Nuclei_Texture_DifferenceVariance_Mito_3_01_256"            ,
 "Nuclei_Texture_DifferenceVariance_Mito_3_02_256"            , "Nuclei_Texture_DifferenceVariance_Mito_3_03_256"            , "Nuclei_Texture_DifferenceVariance_Mito_5_00_256"            , "Nuclei_Texture_DifferenceVariance_Mito_5_01_256"            ,
 "Nuclei_Texture_DifferenceVariance_Mito_5_02_256"            , "Nuclei_Texture_DifferenceVariance_Mito_5_03_256"            , "Nuclei_Texture_DifferenceVariance_RNA_10_00_256"            , "Nuclei_Texture_DifferenceVariance_RNA_10_01_256"            ,
 "Nuclei_Texture_DifferenceVariance_RNA_10_02_256"            , "Nuclei_Texture_DifferenceVariance_RNA_3_00_256"             , "Nuclei_Texture_DifferenceVariance_RNA_3_01_256"             , "Nuclei_Texture_DifferenceVariance_RNA_3_02_256"             ,
 "Nuclei_Texture_DifferenceVariance_RNA_3_03_256"             , "Nuclei_Texture_DifferenceVariance_RNA_5_00_256"             , "Nuclei_Texture_DifferenceVariance_RNA_5_01_256"             , "Nuclei_Texture_DifferenceVariance_RNA_5_02_256"             ,
 "Nuclei_Texture_DifferenceVariance_RNA_5_03_256"             , "Nuclei_Texture_Entropy_AGP_10_00_256"                       , "Nuclei_Texture_Entropy_AGP_10_01_256"                       , "Nuclei_Texture_Entropy_AGP_10_02_256"                       ,
 "Nuclei_Texture_Entropy_AGP_3_00_256"                        , "Nuclei_Texture_Entropy_AGP_3_01_256"                        , "Nuclei_Texture_Entropy_AGP_3_02_256"                        , "Nuclei_Texture_Entropy_AGP_3_03_256"                        ,
 "Nuclei_Texture_Entropy_AGP_5_00_256"                        , "Nuclei_Texture_Entropy_AGP_5_01_256"                        , "Nuclei_Texture_Entropy_AGP_5_02_256"                        , "Nuclei_Texture_Entropy_AGP_5_03_256"                        ,
 "Nuclei_Texture_Entropy_DNA_10_00_256"                       , "Nuclei_Texture_Entropy_DNA_10_02_256"                       , "Nuclei_Texture_Entropy_DNA_10_03_256"                       , "Nuclei_Texture_Entropy_DNA_3_00_256"                        ,
 "Nuclei_Texture_Entropy_DNA_3_01_256"                        , "Nuclei_Texture_Entropy_DNA_3_02_256"                        , "Nuclei_Texture_Entropy_DNA_3_03_256"                        , "Nuclei_Texture_Entropy_DNA_5_00_256"                        ,
 "Nuclei_Texture_Entropy_DNA_5_01_256"                        , "Nuclei_Texture_Entropy_DNA_5_02_256"                        , "Nuclei_Texture_Entropy_DNA_5_03_256"                        , "Nuclei_Texture_Entropy_ER_10_00_256"                        ,
 "Nuclei_Texture_Entropy_ER_10_02_256"                        , "Nuclei_Texture_Entropy_ER_10_03_256"                        , "Nuclei_Texture_Entropy_ER_3_00_256"                         , "Nuclei_Texture_Entropy_ER_3_01_256"                         ,
 "Nuclei_Texture_Entropy_ER_3_02_256"                         , "Nuclei_Texture_Entropy_ER_3_03_256"                         , "Nuclei_Texture_Entropy_ER_5_00_256"                         , "Nuclei_Texture_Entropy_ER_5_01_256"                         ,
 "Nuclei_Texture_Entropy_ER_5_02_256"                         , "Nuclei_Texture_Entropy_ER_5_03_256"                         , "Nuclei_Texture_Entropy_Mito_10_00_256"                      , "Nuclei_Texture_Entropy_Mito_10_01_256"                      ,
 "Nuclei_Texture_Entropy_Mito_10_02_256"                      , "Nuclei_Texture_Entropy_Mito_3_00_256"                       , "Nuclei_Texture_Entropy_Mito_3_01_256"                       , "Nuclei_Texture_Entropy_Mito_3_02_256"                       ,
 "Nuclei_Texture_Entropy_Mito_3_03_256"                       , "Nuclei_Texture_Entropy_Mito_5_00_256"                       , "Nuclei_Texture_Entropy_Mito_5_01_256"                       , "Nuclei_Texture_Entropy_Mito_5_02_256"                       ,
 "Nuclei_Texture_Entropy_Mito_5_03_256"                       , "Nuclei_Texture_Entropy_RNA_10_00_256"                       , "Nuclei_Texture_Entropy_RNA_10_01_256"                       , "Nuclei_Texture_Entropy_RNA_10_02_256"                       ,
 "Nuclei_Texture_Entropy_RNA_3_00_256"                        , "Nuclei_Texture_Entropy_RNA_3_01_256"                        , "Nuclei_Texture_Entropy_RNA_3_02_256"                        , "Nuclei_Texture_Entropy_RNA_3_03_256"                        ,
 "Nuclei_Texture_Entropy_RNA_5_00_256"                        , "Nuclei_Texture_Entropy_RNA_5_01_256"                        , "Nuclei_Texture_Entropy_RNA_5_02_256"                        , "Nuclei_Texture_Entropy_RNA_5_03_256"                        ,
 "Nuclei_Texture_InfoMeas1_AGP_10_00_256"                     , "Nuclei_Texture_InfoMeas1_DNA_10_00_256"                     , "Nuclei_Texture_InfoMeas1_DNA_5_01_256"                      , "Nuclei_Texture_InfoMeas1_ER_10_00_256"                      ,
 "Nuclei_Texture_InfoMeas1_ER_10_02_256"                      , "Nuclei_Texture_InfoMeas1_ER_5_00_256"                       , "Nuclei_Texture_InfoMeas1_ER_5_01_256"                       , "Nuclei_Texture_InfoMeas1_ER_5_02_256"                       ,
 "Nuclei_Texture_InfoMeas1_Mito_10_00_256"                    , "Nuclei_Texture_InfoMeas1_Mito_10_01_256"                    , "Nuclei_Texture_InfoMeas1_Mito_10_02_256"                    , "Nuclei_Texture_InfoMeas1_Mito_3_03_256"                     ,
 "Nuclei_Texture_InfoMeas1_Mito_5_00_256"                     , "Nuclei_Texture_InfoMeas1_Mito_5_01_256"                     , "Nuclei_Texture_InfoMeas1_Mito_5_02_256"                     , "Nuclei_Texture_InfoMeas1_RNA_10_00_256"                     ,
 "Nuclei_Texture_InfoMeas1_RNA_10_02_256"                     , "Nuclei_Texture_InfoMeas2_AGP_10_00_256"                     , "Nuclei_Texture_InfoMeas2_AGP_10_02_256"                     , "Nuclei_Texture_InfoMeas2_AGP_10_03_256"                     ,
 "Nuclei_Texture_InfoMeas2_AGP_3_01_256"                      , "Nuclei_Texture_InfoMeas2_AGP_3_03_256"                      , "Nuclei_Texture_InfoMeas2_AGP_5_00_256"                      , "Nuclei_Texture_InfoMeas2_AGP_5_01_256"                      ,
 "Nuclei_Texture_InfoMeas2_AGP_5_02_256"                      , "Nuclei_Texture_InfoMeas2_AGP_5_03_256"                      , "Nuclei_Texture_InfoMeas2_DNA_10_00_256"                     , "Nuclei_Texture_InfoMeas2_DNA_10_01_256"                     ,
 "Nuclei_Texture_InfoMeas2_DNA_10_02_256"                     , "Nuclei_Texture_InfoMeas2_DNA_3_00_256"                      , "Nuclei_Texture_InfoMeas2_DNA_3_01_256"                      , "Nuclei_Texture_InfoMeas2_DNA_3_03_256"                      ,
 "Nuclei_Texture_InfoMeas2_DNA_5_00_256"                      , "Nuclei_Texture_InfoMeas2_DNA_5_01_256"                      , "Nuclei_Texture_InfoMeas2_DNA_5_02_256"                      , "Nuclei_Texture_InfoMeas2_DNA_5_03_256"                      ,
 "Nuclei_Texture_InfoMeas2_ER_10_00_256"                      , "Nuclei_Texture_InfoMeas2_ER_10_01_256"                      , "Nuclei_Texture_InfoMeas2_ER_10_02_256"                      , "Nuclei_Texture_InfoMeas2_ER_3_01_256"                       ,
 "Nuclei_Texture_InfoMeas2_ER_3_03_256"                       , "Nuclei_Texture_InfoMeas2_ER_5_00_256"                       , "Nuclei_Texture_InfoMeas2_ER_5_01_256"                       , "Nuclei_Texture_InfoMeas2_ER_5_02_256"                       ,
 "Nuclei_Texture_InfoMeas2_ER_5_03_256"                       , "Nuclei_Texture_InfoMeas2_Mito_10_00_256"                    , "Nuclei_Texture_InfoMeas2_Mito_10_02_256"                    , "Nuclei_Texture_InfoMeas2_Mito_10_03_256"                    ,
 "Nuclei_Texture_InfoMeas2_Mito_3_01_256"                     , "Nuclei_Texture_InfoMeas2_Mito_3_02_256"                     , "Nuclei_Texture_InfoMeas2_Mito_3_03_256"                     , "Nuclei_Texture_InfoMeas2_Mito_5_00_256"                     ,
 "Nuclei_Texture_InfoMeas2_Mito_5_01_256"                     , "Nuclei_Texture_InfoMeas2_Mito_5_02_256"                     , "Nuclei_Texture_InfoMeas2_Mito_5_03_256"                     , "Nuclei_Texture_InfoMeas2_RNA_10_00_256"                     ,
 "Nuclei_Texture_InfoMeas2_RNA_10_02_256"                     , "Nuclei_Texture_InfoMeas2_RNA_10_03_256"                     , "Nuclei_Texture_InfoMeas2_RNA_3_01_256"                      , "Nuclei_Texture_InfoMeas2_RNA_3_02_256"                      ,
 "Nuclei_Texture_InfoMeas2_RNA_3_03_256"                      , "Nuclei_Texture_InfoMeas2_RNA_5_00_256"                      , "Nuclei_Texture_InfoMeas2_RNA_5_01_256"                      , "Nuclei_Texture_InfoMeas2_RNA_5_02_256"                      ,
 "Nuclei_Texture_InfoMeas2_RNA_5_03_256"                      , "Nuclei_Texture_InverseDifferenceMoment_AGP_10_00_256"       , "Nuclei_Texture_InverseDifferenceMoment_AGP_10_01_256"       , "Nuclei_Texture_InverseDifferenceMoment_AGP_10_02_256"       ,
 "Nuclei_Texture_InverseDifferenceMoment_AGP_10_03_256"       , "Nuclei_Texture_InverseDifferenceMoment_AGP_3_01_256"        , "Nuclei_Texture_InverseDifferenceMoment_AGP_3_02_256"        , "Nuclei_Texture_InverseDifferenceMoment_AGP_3_03_256"        ,
 "Nuclei_Texture_InverseDifferenceMoment_AGP_5_00_256"        , "Nuclei_Texture_InverseDifferenceMoment_AGP_5_01_256"        , "Nuclei_Texture_InverseDifferenceMoment_AGP_5_02_256"        , "Nuclei_Texture_InverseDifferenceMoment_AGP_5_03_256"        ,
 "Nuclei_Texture_InverseDifferenceMoment_DNA_10_00_256"       , "Nuclei_Texture_InverseDifferenceMoment_DNA_10_02_256"       , "Nuclei_Texture_InverseDifferenceMoment_DNA_10_03_256"       , "Nuclei_Texture_InverseDifferenceMoment_DNA_3_00_256"        ,
 "Nuclei_Texture_InverseDifferenceMoment_DNA_3_01_256"        , "Nuclei_Texture_InverseDifferenceMoment_DNA_3_03_256"        , "Nuclei_Texture_InverseDifferenceMoment_DNA_5_00_256"        , "Nuclei_Texture_InverseDifferenceMoment_DNA_5_01_256"        ,
 "Nuclei_Texture_InverseDifferenceMoment_DNA_5_02_256"        , "Nuclei_Texture_InverseDifferenceMoment_DNA_5_03_256"        , "Nuclei_Texture_InverseDifferenceMoment_ER_10_00_256"        , "Nuclei_Texture_InverseDifferenceMoment_ER_10_01_256"        ,
 "Nuclei_Texture_InverseDifferenceMoment_ER_10_02_256"        , "Nuclei_Texture_InverseDifferenceMoment_ER_10_03_256"        , "Nuclei_Texture_InverseDifferenceMoment_ER_3_01_256"         , "Nuclei_Texture_InverseDifferenceMoment_ER_3_02_256"         ,
 "Nuclei_Texture_InverseDifferenceMoment_ER_3_03_256"         , "Nuclei_Texture_InverseDifferenceMoment_ER_5_00_256"         , "Nuclei_Texture_InverseDifferenceMoment_ER_5_01_256"         , "Nuclei_Texture_InverseDifferenceMoment_ER_5_02_256"         ,
 "Nuclei_Texture_InverseDifferenceMoment_ER_5_03_256"         , "Nuclei_Texture_InverseDifferenceMoment_Mito_10_00_256"      , "Nuclei_Texture_InverseDifferenceMoment_Mito_10_02_256"      , "Nuclei_Texture_InverseDifferenceMoment_Mito_10_03_256"      ,
 "Nuclei_Texture_InverseDifferenceMoment_Mito_3_00_256"       , "Nuclei_Texture_InverseDifferenceMoment_Mito_3_01_256"       , "Nuclei_Texture_InverseDifferenceMoment_Mito_3_02_256"       , "Nuclei_Texture_InverseDifferenceMoment_Mito_3_03_256"       ,
 "Nuclei_Texture_InverseDifferenceMoment_Mito_5_00_256"       , "Nuclei_Texture_InverseDifferenceMoment_Mito_5_01_256"       , "Nuclei_Texture_InverseDifferenceMoment_Mito_5_02_256"       , "Nuclei_Texture_InverseDifferenceMoment_Mito_5_03_256"       ,
 "Nuclei_Texture_InverseDifferenceMoment_RNA_10_00_256"       , "Nuclei_Texture_InverseDifferenceMoment_RNA_10_01_256"       , "Nuclei_Texture_InverseDifferenceMoment_RNA_10_02_256"       , "Nuclei_Texture_InverseDifferenceMoment_RNA_10_03_256"       ,
 "Nuclei_Texture_InverseDifferenceMoment_RNA_3_00_256"        , "Nuclei_Texture_InverseDifferenceMoment_RNA_3_01_256"        , "Nuclei_Texture_InverseDifferenceMoment_RNA_3_02_256"        , "Nuclei_Texture_InverseDifferenceMoment_RNA_3_03_256"        ,
 "Nuclei_Texture_InverseDifferenceMoment_RNA_5_00_256"        , "Nuclei_Texture_InverseDifferenceMoment_RNA_5_01_256"        , "Nuclei_Texture_InverseDifferenceMoment_RNA_5_02_256"        , "Nuclei_Texture_SumAverage_AGP_10_00_256"                    ,
 "Nuclei_Texture_SumAverage_AGP_10_02_256"                    , "Nuclei_Texture_SumAverage_AGP_10_03_256"                    , "Nuclei_Texture_SumAverage_AGP_3_00_256"                     , "Nuclei_Texture_SumAverage_AGP_3_01_256"                     ,
 "Nuclei_Texture_SumAverage_AGP_3_02_256"                     , "Nuclei_Texture_SumAverage_AGP_3_03_256"                     , "Nuclei_Texture_SumAverage_AGP_5_00_256"                     , "Nuclei_Texture_SumAverage_AGP_5_01_256"                     ,
 "Nuclei_Texture_SumAverage_AGP_5_02_256"                     , "Nuclei_Texture_SumAverage_AGP_5_03_256"                     , "Nuclei_Texture_SumAverage_DNA_10_00_256"                    , "Nuclei_Texture_SumAverage_DNA_10_02_256"                    ,
 "Nuclei_Texture_SumAverage_DNA_10_03_256"                    , "Nuclei_Texture_SumAverage_DNA_3_00_256"                     , "Nuclei_Texture_SumAverage_DNA_3_01_256"                     , "Nuclei_Texture_SumAverage_DNA_3_02_256"                     ,
 "Nuclei_Texture_SumAverage_DNA_3_03_256"                     , "Nuclei_Texture_SumAverage_DNA_5_00_256"                     , "Nuclei_Texture_SumAverage_DNA_5_01_256"                     , "Nuclei_Texture_SumAverage_DNA_5_02_256"                     ,
 "Nuclei_Texture_SumAverage_DNA_5_03_256"                     , "Nuclei_Texture_SumAverage_ER_10_00_256"                     , "Nuclei_Texture_SumAverage_ER_10_02_256"                     , "Nuclei_Texture_SumAverage_ER_10_03_256"                     ,
 "Nuclei_Texture_SumAverage_ER_3_00_256"                      , "Nuclei_Texture_SumAverage_ER_3_01_256"                      , "Nuclei_Texture_SumAverage_ER_3_02_256"                      , "Nuclei_Texture_SumAverage_ER_3_03_256"                      ,
 "Nuclei_Texture_SumAverage_ER_5_00_256"                      , "Nuclei_Texture_SumAverage_ER_5_01_256"                      , "Nuclei_Texture_SumAverage_ER_5_02_256"                      , "Nuclei_Texture_SumAverage_ER_5_03_256"                      ,
 "Nuclei_Texture_SumAverage_Mito_10_00_256"                   , "Nuclei_Texture_SumAverage_Mito_10_02_256"                   , "Nuclei_Texture_SumAverage_Mito_10_03_256"                   , "Nuclei_Texture_SumAverage_Mito_3_00_256"                    ,
 "Nuclei_Texture_SumAverage_Mito_3_01_256"                    , "Nuclei_Texture_SumAverage_Mito_3_02_256"                    , "Nuclei_Texture_SumAverage_Mito_3_03_256"                    , "Nuclei_Texture_SumAverage_Mito_5_00_256"                    ,
 "Nuclei_Texture_SumAverage_Mito_5_01_256"                    , "Nuclei_Texture_SumAverage_Mito_5_02_256"                    , "Nuclei_Texture_SumAverage_Mito_5_03_256"                    , "Nuclei_Texture_SumAverage_RNA_10_00_256"                    ,
 "Nuclei_Texture_SumAverage_RNA_10_02_256"                    , "Nuclei_Texture_SumAverage_RNA_10_03_256"                    , "Nuclei_Texture_SumAverage_RNA_3_00_256"                     , "Nuclei_Texture_SumAverage_RNA_3_01_256"                     ,
 "Nuclei_Texture_SumAverage_RNA_3_02_256"                     , "Nuclei_Texture_SumAverage_RNA_3_03_256"                     , "Nuclei_Texture_SumAverage_RNA_5_00_256"                     , "Nuclei_Texture_SumAverage_RNA_5_01_256"                     ,
 "Nuclei_Texture_SumAverage_RNA_5_02_256"                     , "Nuclei_Texture_SumAverage_RNA_5_03_256"                     , "Nuclei_Texture_SumEntropy_AGP_10_00_256"                    , "Nuclei_Texture_SumEntropy_AGP_10_01_256"                    ,
 "Nuclei_Texture_SumEntropy_AGP_10_02_256"                    , "Nuclei_Texture_SumEntropy_AGP_10_03_256"                    , "Nuclei_Texture_SumEntropy_AGP_3_00_256"                     , "Nuclei_Texture_SumEntropy_AGP_3_01_256"                     ,
 "Nuclei_Texture_SumEntropy_AGP_3_02_256"                     , "Nuclei_Texture_SumEntropy_AGP_3_03_256"                     , "Nuclei_Texture_SumEntropy_AGP_5_00_256"                     , "Nuclei_Texture_SumEntropy_AGP_5_01_256"                     ,
 "Nuclei_Texture_SumEntropy_AGP_5_02_256"                     , "Nuclei_Texture_SumEntropy_AGP_5_03_256"                     , "Nuclei_Texture_SumEntropy_DNA_10_00_256"                    , "Nuclei_Texture_SumEntropy_DNA_10_01_256"                    ,
 "Nuclei_Texture_SumEntropy_DNA_10_02_256"                    , "Nuclei_Texture_SumEntropy_DNA_3_00_256"                     , "Nuclei_Texture_SumEntropy_DNA_3_01_256"                     , "Nuclei_Texture_SumEntropy_DNA_3_02_256"                     ,
 "Nuclei_Texture_SumEntropy_DNA_3_03_256"                     , "Nuclei_Texture_SumEntropy_DNA_5_00_256"                     , "Nuclei_Texture_SumEntropy_DNA_5_01_256"                     , "Nuclei_Texture_SumEntropy_DNA_5_02_256"                     ,
 "Nuclei_Texture_SumEntropy_DNA_5_03_256"                     , "Nuclei_Texture_SumEntropy_ER_10_00_256"                     , "Nuclei_Texture_SumEntropy_ER_10_01_256"                     , "Nuclei_Texture_SumEntropy_ER_10_02_256"                     ,
 "Nuclei_Texture_SumEntropy_ER_10_03_256"                     , "Nuclei_Texture_SumEntropy_ER_3_00_256"                      , "Nuclei_Texture_SumEntropy_ER_3_01_256"                      , "Nuclei_Texture_SumEntropy_ER_3_02_256"                      ,
 "Nuclei_Texture_SumEntropy_ER_3_03_256"                      , "Nuclei_Texture_SumEntropy_ER_5_00_256"                      , "Nuclei_Texture_SumEntropy_ER_5_01_256"                      , "Nuclei_Texture_SumEntropy_ER_5_02_256"                      ,
 "Nuclei_Texture_SumEntropy_ER_5_03_256"                      , "Nuclei_Texture_SumEntropy_Mito_10_00_256"                   , "Nuclei_Texture_SumEntropy_Mito_10_01_256"                   , "Nuclei_Texture_SumEntropy_Mito_10_02_256"                   ,
 "Nuclei_Texture_SumEntropy_Mito_10_03_256"                   , "Nuclei_Texture_SumEntropy_Mito_3_00_256"                    , "Nuclei_Texture_SumEntropy_Mito_3_01_256"                    , "Nuclei_Texture_SumEntropy_Mito_3_02_256"                    ,
 "Nuclei_Texture_SumEntropy_Mito_3_03_256"                    , "Nuclei_Texture_SumEntropy_Mito_5_00_256"                    , "Nuclei_Texture_SumEntropy_Mito_5_01_256"                    , "Nuclei_Texture_SumEntropy_Mito_5_02_256"                    ,
 "Nuclei_Texture_SumEntropy_Mito_5_03_256"                    , "Nuclei_Texture_SumEntropy_RNA_10_00_256"                    , "Nuclei_Texture_SumEntropy_RNA_10_01_256"                    , "Nuclei_Texture_SumEntropy_RNA_10_02_256"                    ,
 "Nuclei_Texture_SumEntropy_RNA_10_03_256"                    , "Nuclei_Texture_SumEntropy_RNA_3_00_256"                     , "Nuclei_Texture_SumEntropy_RNA_3_01_256"                     , "Nuclei_Texture_SumEntropy_RNA_3_02_256"                     ,
 "Nuclei_Texture_SumEntropy_RNA_3_03_256"                     , "Nuclei_Texture_SumEntropy_RNA_5_00_256"                     , "Nuclei_Texture_SumEntropy_RNA_5_01_256"                     , "Nuclei_Texture_SumEntropy_RNA_5_02_256"                     ,
 "Nuclei_Texture_SumEntropy_RNA_5_03_256"                     , "Nuclei_Texture_SumVariance_AGP_10_00_256"                   , "Nuclei_Texture_SumVariance_AGP_10_01_256"                   , "Nuclei_Texture_SumVariance_AGP_10_02_256"                   ,
 "Nuclei_Texture_SumVariance_AGP_3_00_256"                    , "Nuclei_Texture_SumVariance_AGP_3_01_256"                    , "Nuclei_Texture_SumVariance_AGP_3_02_256"                    , "Nuclei_Texture_SumVariance_AGP_3_03_256"                    ,
 "Nuclei_Texture_SumVariance_AGP_5_00_256"                    , "Nuclei_Texture_SumVariance_AGP_5_01_256"                    , "Nuclei_Texture_SumVariance_AGP_5_02_256"                    , "Nuclei_Texture_SumVariance_AGP_5_03_256"                    ,
 "Nuclei_Texture_SumVariance_DNA_10_00_256"                   , "Nuclei_Texture_SumVariance_DNA_10_01_256"                   , "Nuclei_Texture_SumVariance_DNA_10_02_256"                   , "Nuclei_Texture_SumVariance_DNA_3_00_256"                    ,
 "Nuclei_Texture_SumVariance_DNA_3_01_256"                    , "Nuclei_Texture_SumVariance_DNA_3_02_256"                    , "Nuclei_Texture_SumVariance_DNA_3_03_256"                    , "Nuclei_Texture_SumVariance_DNA_5_00_256"                    ,
 "Nuclei_Texture_SumVariance_DNA_5_01_256"                    , "Nuclei_Texture_SumVariance_DNA_5_02_256"                    , "Nuclei_Texture_SumVariance_DNA_5_03_256"                    , "Nuclei_Texture_SumVariance_ER_10_00_256"                    ,
 "Nuclei_Texture_SumVariance_ER_10_02_256"                    , "Nuclei_Texture_SumVariance_ER_10_03_256"                    , "Nuclei_Texture_SumVariance_ER_3_00_256"                     , "Nuclei_Texture_SumVariance_ER_3_01_256"                     ,
 "Nuclei_Texture_SumVariance_ER_3_02_256"                     , "Nuclei_Texture_SumVariance_ER_3_03_256"                     , "Nuclei_Texture_SumVariance_ER_5_00_256"                     , "Nuclei_Texture_SumVariance_ER_5_01_256"                     ,
 "Nuclei_Texture_SumVariance_ER_5_02_256"                     , "Nuclei_Texture_SumVariance_ER_5_03_256"                     , "Nuclei_Texture_SumVariance_Mito_10_00_256"                  , "Nuclei_Texture_SumVariance_Mito_10_01_256"                  ,
 "Nuclei_Texture_SumVariance_Mito_10_02_256"                  , "Nuclei_Texture_SumVariance_Mito_3_00_256"                   , "Nuclei_Texture_SumVariance_Mito_3_01_256"                   , "Nuclei_Texture_SumVariance_Mito_3_02_256"                   ,
 "Nuclei_Texture_SumVariance_Mito_3_03_256"                   , "Nuclei_Texture_SumVariance_Mito_5_00_256"                   , "Nuclei_Texture_SumVariance_Mito_5_01_256"                   , "Nuclei_Texture_SumVariance_Mito_5_02_256"                   ,
 "Nuclei_Texture_SumVariance_Mito_5_03_256"                   , "Nuclei_Texture_SumVariance_RNA_10_00_256"                   , "Nuclei_Texture_SumVariance_RNA_10_02_256"                   , "Nuclei_Texture_SumVariance_RNA_10_03_256"                   ,
 "Nuclei_Texture_SumVariance_RNA_3_00_256"                    , "Nuclei_Texture_SumVariance_RNA_3_01_256"                    , "Nuclei_Texture_SumVariance_RNA_3_02_256"                    , "Nuclei_Texture_SumVariance_RNA_3_03_256"                    ,
 "Nuclei_Texture_SumVariance_RNA_5_00_256"                    , "Nuclei_Texture_SumVariance_RNA_5_01_256"                    , "Nuclei_Texture_SumVariance_RNA_5_02_256"                    , "Nuclei_Texture_SumVariance_RNA_5_03_256"                    ,
 "Nuclei_Texture_Variance_AGP_10_00_256"                      , "Nuclei_Texture_Variance_AGP_10_01_256"                      , "Nuclei_Texture_Variance_AGP_10_02_256"                      , "Nuclei_Texture_Variance_AGP_10_03_256"                      ,
 "Nuclei_Texture_Variance_AGP_3_00_256"                       , "Nuclei_Texture_Variance_AGP_3_01_256"                       , "Nuclei_Texture_Variance_AGP_3_02_256"                       , "Nuclei_Texture_Variance_AGP_3_03_256"                       ,
 "Nuclei_Texture_Variance_AGP_5_00_256"                       , "Nuclei_Texture_Variance_AGP_5_01_256"                       , "Nuclei_Texture_Variance_AGP_5_02_256"                       , "Nuclei_Texture_Variance_AGP_5_03_256"                       ,
 "Nuclei_Texture_Variance_DNA_10_00_256"                      , "Nuclei_Texture_Variance_DNA_10_01_256"                      , "Nuclei_Texture_Variance_DNA_10_02_256"                      , "Nuclei_Texture_Variance_DNA_10_03_256"                      ,
 "Nuclei_Texture_Variance_DNA_3_00_256"                       , "Nuclei_Texture_Variance_DNA_3_01_256"                       , "Nuclei_Texture_Variance_DNA_3_02_256"                       , "Nuclei_Texture_Variance_DNA_3_03_256"                       ,
 "Nuclei_Texture_Variance_DNA_5_00_256"                       , "Nuclei_Texture_Variance_DNA_5_01_256"                       , "Nuclei_Texture_Variance_DNA_5_02_256"                       , "Nuclei_Texture_Variance_DNA_5_03_256"                       ,
 "Nuclei_Texture_Variance_ER_10_00_256"                       , "Nuclei_Texture_Variance_ER_10_01_256"                       , "Nuclei_Texture_Variance_ER_10_02_256"                       , "Nuclei_Texture_Variance_ER_10_03_256"                       ,
 "Nuclei_Texture_Variance_ER_3_00_256"                        , "Nuclei_Texture_Variance_ER_3_01_256"                        , "Nuclei_Texture_Variance_ER_3_02_256"                        , "Nuclei_Texture_Variance_ER_3_03_256"                        ,
 "Nuclei_Texture_Variance_ER_5_00_256"                        , "Nuclei_Texture_Variance_ER_5_01_256"                        , "Nuclei_Texture_Variance_ER_5_02_256"                        , "Nuclei_Texture_Variance_ER_5_03_256"                        ,
 "Nuclei_Texture_Variance_Mito_10_00_256"                     , "Nuclei_Texture_Variance_Mito_10_01_256"                     , "Nuclei_Texture_Variance_Mito_10_02_256"                     , "Nuclei_Texture_Variance_Mito_10_03_256"                     ,
 "Nuclei_Texture_Variance_Mito_3_00_256"                      , "Nuclei_Texture_Variance_Mito_3_01_256"                      , "Nuclei_Texture_Variance_Mito_3_02_256"                      , "Nuclei_Texture_Variance_Mito_3_03_256"                      ,
 "Nuclei_Texture_Variance_Mito_5_00_256"                      , "Nuclei_Texture_Variance_Mito_5_01_256"                      , "Nuclei_Texture_Variance_Mito_5_02_256"                      , "Nuclei_Texture_Variance_Mito_5_03_256"                      ,
 "Nuclei_Texture_Variance_RNA_10_00_256"                      , "Nuclei_Texture_Variance_RNA_10_01_256"                      , "Nuclei_Texture_Variance_RNA_10_02_256"                      , "Nuclei_Texture_Variance_RNA_10_03_256"                      ,
 "Nuclei_Texture_Variance_RNA_3_00_256"                       , "Nuclei_Texture_Variance_RNA_3_01_256"                       , "Nuclei_Texture_Variance_RNA_3_02_256"                       , "Nuclei_Texture_Variance_RNA_3_03_256"                       ,
 "Nuclei_Texture_Variance_RNA_5_00_256"                       , "Nuclei_Texture_Variance_RNA_5_01_256"                       , "Nuclei_Texture_Variance_RNA_5_02_256"                       , "Nuclei_Texture_Variance_RNA_5_03_256"                       
 }

variance_dropped_cols_6 = set()


In [288]:
variance_dropped_cols_6 & correlation_dropped_cols_6
print(f" Number of cols dropped - high correlation: {len(correlation_dropped_cols_6)}")
print(f" Number of cols dropped - low variance    : {len(variance_dropped_cols_6)}")
print(f" Number of selected cols                  : {len(selected_cols_6)}")
print(f" Total                                    : {len(selected_cols_6)+len(correlation_dropped_cols_6)+len(variance_dropped_cols_6)}")

set()

 Number of cols dropped - high correlation: 620
 Number of cols dropped - low variance    : 0
 Number of selected cols                  : 160
 Total                                    : 780


In [289]:
(correlation_dropped_cols_6) & set(zero_features)
(variance_dropped_cols_6) & set(zero_features)
(selected_cols_6) & set(zero_features)

set()

set()

set()

## Concatenate selected columns from the groups

In [290]:
ttl_selected_cols     = selected_cols_1 | selected_cols_2 | selected_cols_3 | selected_cols_4 | selected_cols_5 | selected_cols_6
ttl_corr_dropped_cols = correlation_dropped_cols_1 | correlation_dropped_cols_2 | correlation_dropped_cols_3 | correlation_dropped_cols_4 | correlation_dropped_cols_5 | correlation_dropped_cols_6
ttl_var_dropped_cols  = variance_dropped_cols_1 | variance_dropped_cols_2 | variance_dropped_cols_3 | variance_dropped_cols_4 | variance_dropped_cols_5 | variance_dropped_cols_6

print()
print(f" Total selected columns:                    {len(ttl_selected_cols)}")
print(f" Total dropped columns due to correlation:  {len(ttl_corr_dropped_cols)}")
print(f" Total dropped columns due to variance:     {len(ttl_var_dropped_cols)}")
print(f" Total selected columns:                    {len(ttl_selected_cols)}")

print(f" Total dropped and selected :               {len(ttl_corr_dropped_cols) + len(ttl_selected_cols)}")
print()
print(f" Error: columns both in dropped and selected:      {len(ttl_corr_dropped_cols & ttl_selected_cols)}")


 Total selected columns:                    1477
 Total dropped columns due to correlation:  2193
 Total dropped columns due to variance:     0
 Total selected columns:                    1477
 Total dropped and selected :               3670

 Error: columns both in dropped and selected:      0


## Write selected / dropped columns to pickle file

In [119]:
feature_selection_columns = { 'selected': ttl_selected_cols,
                              'dropped_correlation': ttl_corr_dropped_cols,
                              'dropped_variance': set()}
import pickle
with open("./metadata/feature_selection_columns.pkl", 'wb') as f: 
    pickle.dump(feature_selection_columns,f)

In [122]:
with open("./metadata/feature_selection_columns.pkl", 'rb') as f: 
    x = pickle.load(f)
for i in x:
    print(f" {i}    {len(x[i])} ")

 selected    1477 
 dropped_correlation    2193 
 dropped_variance    0 


# Write profiles to separate files based on `Metadata_Bin` 

  ### Used for XGBoost and Dask Clusters 

## Create Dask Clusters 

In [45]:
n_workers = 3
n_threads = 2
cuda_visible_devices = [0,1,2]

try:
    cluster.close()
    del cluster
except Exception as e:
    print(" *** Cluster close failed ***")
else:
    print(" *** Cluster close succeeded ***\n")
finally:
    # cluster = LocalCUDACluster( CUDA_VISIBLE_DEVICES=[0,1,2], n_workers=n_workers, threads_per_worker= n_threads, silence_logs = logging.WARNING)
    cluster = LocalCluster(n_workers=n_workers, threads_per_worker= n_threads, silence_logs = logging.WARNING)
    print(f"\n *** Cluster created - Cluster dashboard link : {cluster.dashboard_link}")

 *** Cluster close succeeded ***



NameError: name 'cluster' is not defined

In [44]:
try:
    client.close()
    del client
except Exception as e:
    print(" *** Client close failed *** ")
else:
    print(" *** Client close succeeded *** ")
finally:
    client = Client(cluster)    
    print("\n *** Client created *** ")
    client

 *** Client close succeeded *** 

 *** Client created *** 


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:45785/status,
Dashboard: http://127.0.0.1:45785/status,Workers: 3
Total threads: 6,Total memory: 64.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:36569,Workers: 3
Dashboard: http://127.0.0.1:45785/status,Total threads: 6
Started: 4 minutes ago,Total memory: 64.00 GiB
Comm: tcp://127.0.0.1:38879,Total threads: 2
Dashboard: http://127.0.0.1:39267/status,Memory: 21.33 GiB
Nanny: tcp://127.0.0.1:39665,


## Create n = 12 output files for the 12 output buckets:

In [38]:
NUMBER_OF_BINS = 12
input_files = []
output_files = []
for i in range(NUMBER_OF_BINS):
    # inputFile = os.path.join('./output_11102023/', '3_sample_profiles', f"3sample_profiles_{i:03d}.csv")
    outputFile = os.path.join('./output_11102023/', '3_sample_profiles', f"3sample_profiles_1482_bin_{i:03d}.csv")
    # print(outputFile)
    # input_files.append(inputFile)
    output_files.append(outputFile)
for i in range(NUMBER_OF_BINS):
    print(f"  -->    {output_files[i]:65s}")
# print(output_files)

  -->    ./output_11102023/3_sample_profiles/3sample_profiles_1482_bin_000.csv
  -->    ./output_11102023/3_sample_profiles/3sample_profiles_1482_bin_001.csv
  -->    ./output_11102023/3_sample_profiles/3sample_profiles_1482_bin_002.csv
  -->    ./output_11102023/3_sample_profiles/3sample_profiles_1482_bin_003.csv
  -->    ./output_11102023/3_sample_profiles/3sample_profiles_1482_bin_004.csv
  -->    ./output_11102023/3_sample_profiles/3sample_profiles_1482_bin_005.csv
  -->    ./output_11102023/3_sample_profiles/3sample_profiles_1482_bin_006.csv
  -->    ./output_11102023/3_sample_profiles/3sample_profiles_1482_bin_007.csv
  -->    ./output_11102023/3_sample_profiles/3sample_profiles_1482_bin_008.csv
  -->    ./output_11102023/3_sample_profiles/3sample_profiles_1482_bin_009.csv
  -->    ./output_11102023/3_sample_profiles/3sample_profiles_1482_bin_010.csv
  -->    ./output_11102023/3_sample_profiles/3sample_profiles_1482_bin_011.csv


## Read input files 

In [39]:
# profileInputFiles = './output_11102023/profiles/compound_profiles_2samples_001.csv'
# profileInputFiles = './output_11102023/profiles/compound_profiles_2samples_002.csv'
# MetadataColumns
profileInputFiles = './output_11102023/3_sample_profiles/3sample_profiles_1482_Unordered.csv'
category_conversions  = {"Metadata_Source":'category', "Metadata_Batch":'category' ,"Metadata_Plate":'category' , "Metadata_Well":'category'}
string_conversions    = {"Metadata_Source": 'string', "Metadata_Batch":'string' ,"Metadata_Plate":'string' , "Metadata_Well":'string'}

In [40]:
# del df_profiles
# df_profiles = read_cell_profiles(profileInputFiles)
logging.info(f" Read profiles file ...")
df_profiles = read_cell_profiles(profileInputFiles, 
                                 # names = COMPOUND_PROFILE_COLUMNS, 
                                 dtype= string_conversions,
                                )
# df_profiles = read_cell_profiles(profileInputFiles, rows = 200000, names = all_columns, usecols = Xy_columns, dtype = Xy_columns_dtype)
logging.info(f" Read profiles file ... complete")

df_profiles.head(3)

2024-04-23 21:24:40,927 - INFO: -  Read profiles file ...
2024-04-23 21:24:41,100 - INFO: -  Read profiles file ... complete


 Reading cell profiles file :  ./output_11102023/3_sample_profiles/3sample_profiles_1482_Unordered.csv    {}
 Reading ALL  rows - Number of partitions:  156   



/home/kevin/miniforge3/envs/cp311/lib/python3.11/site-packages/dask/dataframe/_pyarrow_compat.py:17: FutureWarning: Minimal version of pyarrow will soon be increased to 14.0.1. You are using 11.0.0. Please consider upgrading.
  warnings.warn(


,Metadata_Source,Metadata_Batch,Metadata_Plate,Metadata_Well,Metadata_JCP2022,Metadata_Hash,Metadata_Bin,Metadata_TPSA,Metadata_lnTPSA,Metadata_log10TPSA,...,Nuclei_Texture_SumAverage_DNA_10_01_256,Nuclei_Texture_SumAverage_ER_10_01_256,Nuclei_Texture_SumAverage_Mito_10_01_256,Nuclei_Texture_SumAverage_RNA_10_01_256,Nuclei_Texture_SumEntropy_DNA_10_03_256,Nuclei_Texture_SumVariance_AGP_10_03_256,Nuclei_Texture_SumVariance_DNA_10_03_256,Nuclei_Texture_SumVariance_ER_10_01_256,Nuclei_Texture_SumVariance_Mito_10_03_256,Nuclei_Texture_SumVariance_RNA_10_01_256
0,source_1,Batch1_20221004,UL001641,AA40,JCP2022_000082,-1920040449369071690,6,64.89,4.172694,1.812178,...,0.438577,-0.051575,0.279409,0.767113,0.143125,0.498506,-0.147239,-0.072483,0.059082,0.547492
1,source_1,Batch1_20221004,UL001641,Z22,JCP2022_000137,-9070688483616626497,15,60.28,4.099000,1.780173,...,0.522905,0.206919,-0.737207,0.050593,0.889717,-0.436981,0.109206,-0.024600,-0.490239,-0.262859
2,source_1,Batch1_20221004,UL001641,Z32,JCP2022_000428,4317235005617559002,10,94.53,4.548917,1.975570,...,0.258570,0.235806,0.430072,0.565591,0.588245,-0.338327,-0.014979,0.087962,-0.005913,0.229194


In [41]:
df_profiles.shape
df_profiles.Metadata_Bin.value_counts()
# df_profiles.info(verbose = True)
# df_profiles.memory_usage(deep= True).compute()

(Delayed('int-9068b106-62f5-44b7-b64a-65d0267846c9'), 1482)

Dask Series Structure:
npartitions=1
    int64
      ...
Name: count, dtype: int64
Dask Name: value-counts-agg, 4 graph layers

In [ ]:
# df_profiles.memory_usage(deep=True)[:10]
#  #   Column               Non-Null Count  Dtype
# ---  ------               --------------  -----
#  0   Metadata_Source      231017 non-null      string
#  1   Metadata_Batch       231017 non-null      string
#  2   Metadata_Plate       231017 non-null      string
#  3   Metadata_Well        231017 non-null      string
#  4   Metadata_JCP2022     231017 non-null      object
#  5   Metadata_Hash        231017 non-null      int64
#  6   Metadata_Bin         231017 non-null      int64
#  7   Metadata_TPSA        231017 non-null      float64
#  8   Metadata_lnTPSA      231017 non-null      float64
#  9   Metadata_log10TPSA   231017 non-null      float64
# 10   Metadata_Permiation  231017 non-null      int64
# dtypes: object(1), float64(3), int64(3), string(4)


#  #   Column               Dtype          Size(Categorical)    Size(string)
# ---  ------               -------         ---------------- ----------------
#      Index                                       33,396           33,396
#  0   Metadata_Source      category/string       249,067       15,057,278
#  1   Metadata_Batch       category/string       271,226       17,233,446
#  2   Metadata_Plate       category/string       429,214       15,503,673
#  3   Metadata_Well        category/string     9,562,119       13,872,452
#  4   Metadata_JCP2022     object             16,402,207       16,402,207
#  5   Metadata_Hash        int64               1,848,136        1,848,136
#  6   Metadata_Bin         int64               1,848,136        1,848,136
#  7   Metadata_TPSA        float64             1,848,136        1,848,136
#  8   Metadata_lnTPSA      float64             1,848,136        1,848,136
#  9   Metadata_log10TPSA   float64             1,848,136        1,848,136
# 10   Metadata_Permiation  int64               1,848,136        1,848,136
 

**Memory Usage:**
 |No |Field name| Type | categorical | string[python] | 
 |---|-----------|-----|-----------------:|------------------:
 |    | Index               |                 |       33,396  |         33,396 
 | 0  | Metadata_Source     | category/string |      249,067  |     15,057,278 
 | 1  | Metadata_Batch      | category/string |      271,226  |     17,233,446 
 | 2  | Metadata_Plate      | category/string |      429,214  |     15,503,673 
 | 3  | Metadata_Well       | category/string |    9,562,119  |     13,872,452 
 | 4  | Metadata_JCP2022    | object          |   16,402,207  |     16,402,207 
 | 5  | Metadata_Hash       | int64           |    1,848,136  |      1,848,136 
 | 6  | Metadata_Bin        | int64           |    1,848,136  |      1,848,136 
 | 7  | Metadata_TPSA       | float64         |    1,848,136  |      1,848,136 
 | 8  | Metadata_lnTPSA     | float64         |    1,848,136  |      1,848,136 
 | 9  | Metadata_log10TPSA  | float64         |    1,848,136  |      1,848,136 
| 10  | Metadata_Permiation | int64           |    1,848,136  |      1,848,136 

In [ ]:
# df_profiles.Metadata_Source.dtype
# df_profiles.Metadata_Source.memory_usage(deep=True).compute()

## Group by Bin Number

In [42]:
df_grouped_profiles = df_profiles.groupby('Metadata_Bin')

### Count num of rows in each bin

In [43]:

df_profiles_bin = df_profiles['Metadata_Bin']
type(df_profiles_bin)
df_profiles_bin.npartitions

s1 = df_profiles_bin.value_counts().compute()
s1.sort_index()
s1.sum()

# Metadata_Bin
# 0     19279
# 1     18815
# 2     19048
# 3     19421
# 4     19368
# 5     19332
# 6     19480
# 7     19298
# 8     19246
# 9     19189
# 10    19183
# 11    19358
# Name: count, dtype: int64


dask.dataframe.core.Series

156

/home/kevin/miniforge3/envs/cp311/lib/python3.11/site-packages/dask/dataframe/_pyarrow_compat.py:17: FutureWarning: Minimal version of pyarrow will soon be increased to 14.0.1. You are using 11.0.0. Please consider upgrading.
  warnings.warn(
/home/kevin/miniforge3/envs/cp311/lib/python3.11/site-packages/dask/dataframe/_pyarrow_compat.py:17: FutureWarning: Minimal version of pyarrow will soon be increased to 14.0.1. You are using 11.0.0. Please consider upgrading.
  warnings.warn(


Metadata_Bin
0     21780
1     21435
2     21804
3     21714
4     21756
5     21315
6     21351
7     21834
8     21291
9     21606
10    21783
11    21645
12    22014
13    21654
14    21633
15    21927
Name: count, dtype: int64

346542

## Read Each bin group ansd write to separte file

In [35]:
# BIN_NO = 2

for BIN_NO in range(12): ## [0,1,2,3,4,5,7, 8, 9 , 10, 11]:
    
    logging.info(f" Get bin group: {BIN_NO} from df_grouped_profiles")    
    
    grp = df_grouped_profiles.get_group((BIN_NO,))
    logging.info(f" Number of partitions :  {grp.npartitions}")
    grp.head(3)
    
    # ss = grp.map_partitions(len).compute()
    # ss
    # ss.sum()
    
    logging.info(f" Write bin {BIN_NO} to  {output_files[BIN_NO]}")
    
    grp.to_csv(output_files[BIN_NO], index = False, single_file = True)
    
    logging.info(f" Writing bin {BIN_NO} profiles file ... complete")

2024-04-23 17:22:35,758 - INFO: -  Get bin group: 0 from df_grouped_profiles
2024-04-23 17:22:36,165 - INFO: -  Number of partitions :  156
2024-04-23 21:22:54,198 - ERROR: - Task exception was never retrieved
future: <Task finished name='Task-214751' coro=<Client._gather.<locals>.wait() done, defined at /home/kevin/miniforge3/envs/cp311/lib/python3.11/site-packages/distributed/client.py:2209> exception=AllExit()>
Traceback (most recent call last):
  File "/home/kevin/miniforge3/envs/cp311/lib/python3.11/site-packages/distributed/client.py", line 2218, in wait
    raise AllExit()
distributed.client.AllExit


KeyboardInterrupt: 

### Read Each bin and verify count

In [ ]:
# BIN_READ = 2

for BIN_READ in  range(12) : ## [7, 8, 9 , 10, 11]:
    logging.info(f" Read from  {output_files[BIN_READ]}  for bin {BIN_READ} ")
    grp =  dd.read_csv(output_files[BIN_READ], usecols = ['Metadata_JCP2022', 'Metadata_Bin'],)
    grp.npartitions  
    grp.head()
    ss = grp.map_partitions(len).compute()
    logging.info(f" Total rows in bin {BIN_READ} :   {ss.sum()}")


# Build Profiles Metadata file `profiles_metadata.pkl`

In [ ]:
# print(f" Features select file:  {profilesMetadataFile,}")

# with open(profilesMetadataFile,, 'rb') as f: 
#     x = pickle.load(f)
    
# for i in x:
#     print(f" {i:20s}    {len(x[i])} ")

metadata_columns = set(["Metadata_Source", "Metadata_Batch", "Metadata_Plate", "Metadata_Well", "Metadata_JCP2022"])
metadata_columns_dtype = {x: pd.StringDtype()  for x in metadata_columns} 

label_columns =  set(["Metadata_TPSA", "Metadata_lnTPSA", "Metadata_log10TPSA"])
label_columns_dtype = {x: np.dtype('float32') for x in label_columns}
label_columns_dtype["Metadata_permiation"] = np.dtype('int8')

X_columns = x['selected']
X_columns_conversion = {x: np.dtype('float32') for x in X_columns}

y_columns = set(["Metadata_permiation"])
y_columns_dtype = {x: np.dtype('int8') for x in y_columns} ## "Metadata_log10TPSA":np.dtype('float64')}

# y_columns = set(["Metadata_log10TPSA"])
# y_columns_dtype = {x: np.dtype('float32') for x in y_columns} ## "Metadata_log10TPSA":np.dtype('float64')}

Xy_columns = y_columns | X_columns
Xy_columns_dtype = y_columns_dtype | X_columns_dtype

all_columns = pd.read_csv( './output_11102023/profiles/compound_profiles_2samples_001.csv', nrows = 0).columns.tolist()
# all_columns = metadata_columns | label_columns | X_columns
# all_columns_dtype = metadata_columns_dtype | label_columns_dtype | X_columns_dtype

print(f" len(metadata_columms)        : {len(metadata_columns)} ")
print(f" len(label_columms)           : {len(label_columns)} ")
print(f" len(X_columms)               : {len(X_columns)}")
print(f" len(y_columms)               : {len(y_columns)}")
print(f" len(Xy_columms)              : {len(Xy_columns)}")
print(f" len(all_columms)             : {len(all_columns)} ")

# print(f" len(metadata_columms_dtype)  : {len(metadata_columns_dtype)} ")
# print(f" len(label_columms_dtype)     : {len(label_columns_dtype)} ")
# print(f" len(X_columms_dtype)         : {len(X_columns_dtype)}")
# print(f" len(y_columms_dtype)         : {len(y_columns_dtype)}")
# print(f" len(Xy_columms_dtype)        : {len(Xy_columns_dtype)}")
# print(f" len(all_columms_dtype)       : {len(all_columns_dtype)} ")
# del all_columns_dtype
# for k  in sorted(all_columns_dtype.keys()):
#     print(f" {k:60s}  {all_columns_dtype[k]}")

In [ ]:
feature_selection_columns = { 'selected': ttl_selected_cols,
                              'dropped_correlation': ttl_corr_dropped_cols,
                              'dropped_variance': set()
                              'dropped_nans': set()}
# with open("./metadata/feature_selection_columns.pkl", 'wb') as f: 
#     pickle.dump(feature_selection_columns,f)

In [ ]:
 
# all_profile_columns = df_profiles.columns.to_list()

In [ ]:
MetadataOriginalNames      = ["Metadata_Source", "Metadata_Batch" ,"Metadata_Plate" , "Metadata_Well", "Metadata_JCP2022", "Metadata_Hash", 
                              "TPSA"  , "lnTPSA"  , "log10TPSA", "permiation"]
                        
MetadataNames              = ["Metadata_Source", "Metadata_Batch" , "Metadata_Plate" , "Metadata_Well"     , "Metadata_JCP2022", "Metadata_Hash", 
                              "Metadata_Bin"   , "Metadata_TPSA"  , "Metadata_lnTPSA", "Metadata_log10TPSA", "Metadata_Permiation"]

MetadataCategoricalColumns = {"Metadata_Source":'category', "Metadata_Batch":'category' ,"Metadata_Plate":'category' , "Metadata_Well":'category'}

MetadataFloatColumns       = {"Metadata_TPSA":np.float32, "Metadata_lnTPSA":np.float32, "Metadata_log10TPSA":np.float32}

MetadataIntegerColumns     = {"Metadata_Hash": np.int64, "Metadata_Bin":np.int8, "Metadata_Permiation":np.int8}

MetadataNumericColumns     =  MetadataFloatColumns | MetadataIntegerColumns

metadata_columns = {"MetadataOriginalNames"      : MetadataOriginalNames,
                    "MetadataNames"              : MetadataNames,
                    "MetadataCategoricalColumns" : MetadataCategoricalColumns,
                    "MetadataNumericColumns"     : MetadataNumericColumns,
                    "MetadataFloatColumns"       : MetadataFloatColumns,
                    "MetadataIntegerColumns"     : MetadataIntegerColumns,
                   }


pickle_data['metadata_columns'] = metadata_columns

## Read Profile Metadata file

In [ ]:
print(f" Features selection file:  {profileMetadataFile}")

with open(profileMetadataFile, 'rb') as f: 
     pickle_data = pickle.load(f)
pickle_data.keys()

In [ ]:
disp_metadata_file(pickle_data)
 

In [ ]:
# pickle_data['selected_columns'].keys()
# pickle_data['selected_columns']['dropped_nans']  = {'Cells_AreaShape_FormFactor',
#                                                      'Cells_Neighbors_AngleBetweenNeighbors_5',
#                                                      'Cells_Neighbors_AngleBetweenNeighbors_Adjacent',
#                                                      'Cells_Neighbors_FirstClosestDistance_5',
#                                                      'Cells_Neighbors_FirstClosestDistance_Adjacent',
#                                                      'Cells_Neighbors_SecondClosestDistance_5',
#                                                      'Cells_Neighbors_SecondClosestDistance_Adjacent',
#                                                      'Cytoplasm_AreaShape_FormFactor',
#                                                      'Nuclei_AreaShape_FormFactor',
#                                                      'Nuclei_Neighbors_AngleBetweenNeighbors_1',
#                                                      'Nuclei_Neighbors_FirstClosestDistance_1',
#                                                      'Nuclei_Neighbors_SecondClosestDistance_1'}


# pickle_data['selected_columns']['dropped_image_features'] = set(list(ParquetColumns['Image'].keys()))
# len(pickle_data['selected_columns']['dropped_image_features'])
# pickle_data['selected_columns']['dropped_metadata'] = set(list(ParquetColumns['Metadata'].keys()))
# len(pickle_data['selected_columns']['dropped_metadata'])

In [ ]:
# for i in ['Cells_AreaShape_FormFactor', 'Cytoplasm_AreaShape_FormFactor', 'Nuclei_AreaShape_FormFactor'
#           'Cells_Neighbors_AngleBetweenNeighbors_5',
#           'Cells_Neighbors_AngleBetweenNeighbors_Adjacent',
#           'Cells_Neighbors_FirstClosestDistance_5',
#           'Cells_Neighbors_FirstClosestDistance_Adjacent',
#           'Cells_Neighbors_SecondClosestDistance_5',
#           'Cells_Neighbors_SecondClosestDistance_Adjacent',
#           'Nuclei_Neighbors_AngleBetweenNeighbors_1',
#           'Nuclei_Neighbors_FirstClosestDistance_1',
#           'Nuclei_Neighbors_SecondClosestDistance_1']:

# for i in pickle_data["selected_columns"]["dropped_nans"]:
    
#     if i in pickle_data['all_profile_columns']: 
#     # if i in pickle_data['selected_columns']['dropped_correlation']: 
#         # del parquetNumericColumns [x]
#         print(f" FOUND in pickle_data['all_profile_columns'], deleted -- {i:35s}")
#     else:
#         print(f" NOT FOUND in pickle_data['all_profile_columns'], skipped -- {i:35s}")
# len(pickle_data['all_profile_columns'])
# t = set(pickle_data['all_profile_columns'])  - set(pickle_data['metadata_columns']['MetadataColumns'])
# len(t)
# # t = set(pickle_data['all_profile_columns'])  - set(pickle_data['selected_columns']['selected'])
# # len(t)
# t1 = t -set(pickle_data['selected_columns']['dropped_correlation'])
# len(t1)
# t1

In [ ]:
# ttl_bad_columns =  {'Cells_Neighbors_AngleBetweenNeighbors_Adjacent', 
#                     'Cells_Neighbors_AngleBetweenNeighbors_5', 
#                     'Cells_Neighbors_FirstClosestDistance_Adjacent', 
#                     'Cells_Neighbors_SecondClosestDistance_Adjacent',
#                     'Cells_Neighbors_FirstClosestDistance_5', 
#                     'Cells_Neighbors_SecondClosestDistance_5',
#                     'Nuclei_Neighbors_AngleBetweenNeighbors_1', 
#                     'Nuclei_Neighbors_SecondClosestDistance_1', 
#                     'Nuclei_Neighbors_FirstClosestDistance_1', 
#                    }
# len(ttl_bad_columns)
# pickle_data['selected_columns']['dropped_nans'] = ttl_bad_columns | {'Cells_AreaShape_FormFactor', 'Cytoplasm_AreaShape_FormFactor', 'Nuclei_AreaShape_FormFactor'}
# len(pickle_data['selected_columns']['dropped_nans']) 
# pickle_data['selected_columns']['dropped_nans']  

In [ ]:
# tmp_list = []
# check_in = pickle_data['all_profile_columns']
# check_in = pickle_data['selected_columns']['dropped_correlation']

# for i in ttl_bad_columns: 
#     if i in check_in:
#         print(f"{i} is in DROPPED CORRELATION" )
#         # pickle_data['selected_columns']['dropped_correlation']
#         # print(f"{i} is in 'all_profile_columns'" )
#         # pickle_data['all_profile_columns'].remove(i)
#         tmp_list.append(i)        
#     else:
#         # print(f" ***** {i} is NOT in all_profile_columns'" )
#         # print(f"{i} is NOT in DROPPED CORRELATION" )
#         # if i in pickle_data['selected_columns']['dropped_correlation']:
#             # print(f"{i} is in DROPPED CORRELATION" )
#         # else :
#         pass
# tmp_list         
# pickle_data['selected_columns']['dropped_correlation'] = pickle_data['selected_columns']['dropped_correlation'] - set(tmp_list)

In [ ]:
# dump_data = { 'all_profile_columns'   : COMPOUND_PROFILE_COLUMNS,  ## pickle_data['all_profile_columns'],
#                 'metadata_columns'    : pickle_data['metadata_columns'],
#                 'selected_columns'    : pickle_data['selected_columns'],
#                 'parquet_columns'     : pickle_data['parquet_columns'] }

In [ ]:

with open(profileMetadataFile, 'wb') as f: 
     pickle.dump(pickle_data,f)


In [ ]:

with open(profileMetadataFile, 'rb') as f: 
    pickle_data = pickle.load(f)
    

In [ ]:
disp_metadata_file(pickle_data)

# Build Metadata File of Parquet column names (if not existing)

Build dictionary of Parquet data columns - write columns to pickle

This list of column names will allow us to identify & control the columns we want to load from parquet files

In [ ]:
with open("./metadata/parquet_columns.pkl",'rb') as f:
    ParquetColumns = pickle.load(f)

for k,v in ParquetColumns.items():
    print(f" {k:20s}   items: {len(v)}")
# print(ParquetColumns['Cytoplasm'])

parquetNumericColumns  =  ParquetColumns['Cells'] | ParquetColumns['Cytoplasm'] | ParquetColumns['Nuclei']

print(f"\n\n all numeric columns (Cells + Cytoplasm + Nuclei) : {len(parquetNumericColumns )}")


In [ ]:
# # keys = ('source_1', 'Batch1_20221004', 'UL001641')
# keys = ('source_6' , 'p210920CPU2OS48hw384exp028JUMP',   '110000296345') 

# dframe = read_profiles_from_local(keys, local_formatter,  columns = None)
# cols = dframe.columns.to_list()
# print(f" rows: {len(dframe)}  columns {len(cols)}")      
# cat_cols = cat_columns(dframe)

# tt = [] 
# for k in cols:
#     if (dframe[k].dtype in [np.int64]):
#         print(f" {k:50s}   {dframe[k].dtype}")
#         tt.append(k)
# len(tt)


In [ ]:
# ParquetColumns = { 'Metadata' : {x: dframe[x].dtype for x in cat_cols['Metadata']},
#                    'Cells'    : {x: dframe[x].dtype for x in cat_cols['Cells']},
#                    'Cytoplasm': {x: dframe[x].dtype for x in cat_cols['Cytoplasm']},
#                     'Nuclei'  : {x: dframe[x].dtype for x in cat_cols['Nuclei']},
#                     'Image'   : {x: dframe[x].dtype for x in (cat_cols['Image_Granularity']+ cat_cols['Image_ImageQuality'] + 
#                                                               cat_cols['Image_Intensity']  + cat_cols['Image_Texture']      + 
#                                                               cat_cols['Image_Threshold'])},
#                  }

# print(len(floatConversion) + len(intConversion))
# import pickle
# with open("./metadata/parquet_columns.pkl", 'wb') as f: 
#     pickle.dump(ParquetColumns,f)

In [ ]:
# del ParquetColumns

# with open("./metadata/parquet_columns.pkl",'rb') as f:
#     ParquetColumns = pickle.load(f)

# for k,v in ParquetColumns.items():
#     print(f" {k:20s}   items: {len(v)}")

# for k,v in ParquetColumns['Cytoplasm'].items():
#     print(f" {k:50s}   type: {v}")


# `Nan` detection on Compound / Profiles dataset - Development

In [ ]:
check_fld_grps = {'_'.join(x.split('_')[:2]) for x in ParquetColumns['Cells'] }
# ParquetColumns['Cells']
# ParquetColumns['Nuclei']

In [ ]:
import re
pattern = re.compile("Cells_Neighbors*")
cell_neighbor_cols = list(filter(pattern.match, list(ParquetColumns['Cells'].keys())))
cell_neighbor_cols

pattern = re.compile("Nuclei_Neighbors*")
nuclei_neighbor_cols = list(filter(pattern.match, list(ParquetColumns['Nuclei'].keys())))
nuclei_neighbor_cols

check_cols = cell_neighbor_cols + nuclei_neighbor_cols
check_cols

## Load Extended metadata

The following files contain the metadata information for the entire dataset.
The schema is [here](metadata/README.md).

In [13]:
CompoundExtendedMetadata2SampleFile
CompoundExtendedMetadata3SampleFile

INPUT_FILE = CompoundExtendedMetadata3SampleFile
print(f" Input file: {INPUT_FILE}")
# MetadataNames
MetadataCategoricalColumns

'./output_11102023/compound_extended_metadata_2samples.csv'

'./output_11102023/compound_extended_metadata_3samples.csv'

 Input file: ./output_11102023/compound_extended_metadata_3samples.csv


NameError: name 'MetadataCategoricalColumns' is not defined

In [ ]:
ext_metadata = pd.read_csv(INPUT_FILE ,
                           # usecols = [0,1,2,3,4,5,  9,10,11,12], 
                           # usecols = MetadataNames, 
                           # names=extMetadataNames,  
                           header=0,
                          )
ext_metadata.sort_values(by=list(MetadataCategoricalColumns), inplace=True)

ext_metadata.info()
# ext_metadata.head(5)
# ext_metadata.tail(5)
ext_metadata.reset_index(drop=True, inplace = True)

# print('---------------------------------------------------------------')
# print(' After Index Reset')
# print('---------------------------------------------------------------')
# ext_metadata.info()
# ext_metadata.head(3)
# ext_metadata.tail(3)

ext_metadata.iloc[199995:200005,:6]
ext_metadata.memory_usage(deep=True) 


## Group  Metadata by Source, Batch, and Plate Id

In [ ]:
grouped_metadata = ext_metadata.groupby(by = ['Metadata_Source','Metadata_Batch','Metadata_Plate'])
type(grouped_metadata)
len(grouped_metadata)

for ctr, (keys, group_data) in enumerate(grouped_metadata):
    print(f"{ctr:5d}     {keys}") 
    if ctr > 10:
        break

## For each Metadata group (Plate) read matching parquet profile

In [ ]:
# keys =  ('source_1', 'Batch6_20221102', 'UL000597')
# keys =  ('source_10', '2021_06_08_U2OS_48_hr_run4', 'Dest210608-152921') 
# keys =  ('source_10', '2021_08_23_U2OS_48_hr_run18', 'Dest210823-174556')
# keys =  ('source_11', 'Batch3', 'EC000136')
keys = ('source_11',	'Batch3',	'EC000157'	)


In [ ]:
group_data = ext_metadata.query('Metadata_Source == @keys[0] & Metadata_Batch == @keys[1] & Metadata_Plate == @keys[2]')
type(group_data)
group_data.columns
group_data
# group_data[[check_cols].isna().any()
indices_to_change = group_data.index
indices_to_change

In [ ]:
# ext_metadata.loc[indices_to_change, 'Metadata_Plate']  = 'Kevin'
# ext_metadata.loc[indices_to_change] 

In [ ]:
 # 4  key: ('source_1', 'Batch6_20221102', 'UL000595')   1
 # 5  key: ('source_1', 'Batch6_20221102', 'UL000597')   388
 # 6  key: ('source_10', '2021_06_08_U2OS_48_hr_run4', 'Dest210608-152921')   1
 # 7  key: ('source_10', '2021_08_23_U2OS_48_hr_run18', 'Dest210823-174556')   1
 # 8  key: ('source_11', 'Batch3', 'EC000136')   1
 # 9  key: ('source_11', 'Batch3', 'EC000157')   129

In [ ]:
# keys =  ('source_1', 'Batch6_20221102', 'UL000595')
# keys =  ('source_1', 'Batch6_20221102', 'UL000597')
# keys =  ('source_11', 'Batch3', 'EC000136')
# keys = ('source_11',	'Batch3',	'EC000157'	)
# keys

# source_path = local_formatter.format(root_folder, keys[0], keys[1], keys[2])
# source_path 

# dframe = read_profiles_from_local(root_folder, keys, local_formatter,  READ_PARQUET_COLUMNS)
# dframe

# indices_to_change = dframe.index
# indices_to_change

# dframe['Metadata_Plate'] = 'EC000157'
# dframe

In [ ]:
# dframe.to_parquet(source_path)

In [ ]:
# df_join = pd.merge(group_data, dframe, how='left', on=['Metadata_Source', 'Metadata_Plate', 'Metadata_Well'])
# df_join

## Run Nan detection routine against Metadata/Profile join

In [ ]:
# del na_rows, rows_read
df_profiles_list = []
group_id = 0
ttl_meta_rows = 0
ttl_rows_read = 0
ttl_na_rows = 0 
ttl_bad_columns = set()
df_Nans = pd.DataFrame()

In [ ]:
START_GROUP = 200
END_GROUP  = 9999

for group_id, (keys, group_data) in enumerate(grouped_metadata,1):
    if group_id <= START_GROUP: 
        continue
        
    df_profile = read_profiles_from_local(root_folder, keys, local_formatter,  READ_PARQUET_COLUMNS)
    df_join = pd.merge(group_data, df_profile, how='left', on=['Metadata_Source', 'Metadata_Plate', 'Metadata_Well'])
    
    print(f" group: {group_id}  key: {keys}  metadata size: {len(group_data)}    profile size: {len(df_profile )}    join_size: {len(df_join)}")
    ttl_rows_read, ttl_na_rows, ttl_bad_columns, df_Nans = check_df_for_nans(df_join, group_id, ttl_rows_read , ttl_na_rows, ttl_bad_columns, df_Nans)    

    ttl_meta_rows  += len(group_data)
 
    if group_id >= END_GROUP:
        break

print("\n")
print(f" # of groups: {group_id} \n")
print(f" ttl_meta_rows - total number of metadata rows: {ttl_meta_rows}")
print(f" ttl_rows_read - total join rows read   : {ttl_rows_read}")  
print(f" ttl_na_rows   - total invalid rows     : {ttl_na_rows}")
print(f"\n Bad columns: {ttl_bad_columns}")
print("\n")


In [ ]:
df_Nans.info()
# list(df_Nans.index)
# df_Nans.columns

df_Nans_T = df_Nans.transpose()

In [ ]:
df_Nans_T.info()
df_Nans_T

In [ ]:
for group_id, (keys, group_data) in enumerate(df_Nans_T.groupby(by = ['Metadata_Source','Metadata_Batch','Metadata_Plate'])):
    print(f" {group_id}  key: {keys}   {len(group_data)}")

In [ ]:
check_cols

In [ ]:
for i in check_cols: 
    df_Nans_T[i].isna().value_counts()

In [ ]:
c = 100
df_Nans_T.iloc[195:201][['Metadata_JCP2022']+check_cols]

In [ ]:
df_join.head(4)
# df_profiles['Metadata_Plate'].unique()

row_num, na_rows, bad_columns = check_df_for_nans(df_join)
print(f" Rows read  : {row_num}   invalid rows : {na_rows}")
print(f" Bad columns: {bad_columns}")

In [ ]:
# COMPOUND_PROFILE_COLUMNS = df_profiles.columns.to_list()
# len(COMPOUND_PROFILE_COLUMNS)
len(READ_PARQUET_COLUMNS)
# col_dict = cat_columns(df_profiles)
# col_dict['Nuclei'][1000:]
JOIN_COLUMNS[:12]

# Normalize by Plate|

**`normalize Parameters()`**

- **profiles :** pandas.core.frame.DataFrame or path\
    Either a pandas DataFrame or a file that stores profile data
- **features :** list\
    A list of strings corresponding to feature measurement column names in the `profiles` DataFrame. All features listed must be found in `profiles`.\
    Defaults to `"infer"`. If `"infer"`, then assume cell painting features are those prefixed with `"Cells"`, `"Nuclei"`, or `"Cytoplasm"`.
- **image_features:** bool, default False\
    Whether the profiles contain image features.
- **meta_features :** list\
    A list of strings corresponding to metadata column names in the `profiles`DataFrame. All features listed must be found in `profiles`. Defaults to "infer".
    If `"infer"`, then assume metadata features are those prefixed with `"Metadata"`
- **samples :** tsr, default `allr\
    The metadata column values to use as a normalization reference. We often us econtrol samples. The function uses a `pd.query()` function, so you should\
    structure samples in this fashion. An example is"Metadata_treatment == 'control'" (include all quotes). Defaults to "all".
- **method :** str  `'standardize'` | `'robustize'` | `'mad_robustize'` | `'spherize'` \
    How to normalize the dataframe. Defaults to "standardize". Check avail_method sfor available normalization methods.
- **output_file :** str, optional\
    If provided, will write normalized profiles to file. If not specified, will return the normalized profiles as outpu.t\
    We recommend that this output file b  suffixed with "_normalized.csv".
- **output_type :** str, optional\
    If provided, will write normalized profiles as a specified file type (either CSV or parquet).
    If not specified and output_file is provided, then the file will be outputed as CSV as default.
- **compression_options :** str or dict, optional\
    Contains compression options as input to pd.DataFrame.to_csv(compression=compression_options). pandas version >= 1.2.
- **float_format :** str, optional\
    Decimal precision to use in writing output file as input to
    pd.DataFrame.to_csv(float_format=float_format). For example, use "%.3g" for 3
    decimal precision.
- **mad_robustize_epsilon:** float, optional\
    The mad_robustize fudge factor parameter. The function only uses
    this variable if method = "mad_robustize". Set this to 0 if
    mad_robustize generates features with large values.
- **spherize_center :** bool\
    If the function should center data before sphering (aka whitening). The
    function only uses this variable if method = "spherize". Defaults to True.
- **spherize_method :** str\
    The sphering (aka whitening) normalization selection. The function only uses
    this variable if method = "spherize". Defaults to "ZCA-corr". See
    :py:func:`pycytominer.operations.transform` for available spherize methods.
- **spherize_epsilon :** float, default 1e-6.\
    The sphering (aka whitening) fudge factor parameter. The function only uses
    this variable if method = "spherize".
  
**Returns**
 
- **normalized :** pandas.core.frame.DataFrame, optional\
    The normalized profile DataFrame. If output_file=None, then return the
    DataFrame. If you specify output_file, then write to file and do not return
    data.
, then write to file and do not return
        data
.

In [ ]:
try:
    df_normalized = normalize(
        profiles=df_join,
        features="infer",
        meta_features="infer",
        samples="all",
        method="standardize",
        # method="robustize",
        # method="mad_robustize",
        # method="spherize",
        # output_file=None,
        # mad_robustize_epsilon = 0 , 
        # compression_options=compression_options,
    )    
except Exception as e:
    print(f" exception:  {e} occurred at {keys}")

In [ ]:
df_normalized.head(5)

In [ ]:
# df_normalized[:10]
# df_normalized.isna().any(axis=1)

In [ ]:
disp_stats(df_normalized, col_dict['Cells'][:20])

In [ ]:
disp_stats(df_normalized, ['Cells_AreaShape_Area', 'Nuclei_Texture_Variance_RNA_10_03_256'])

## Display and plot normalized columns

In [ ]:
# disp_stats(df_normalized_robust, ['Cells_AreaShape_Area', 'Nuclei_Texture_Variance_RNA_10_03_256'])
# disp_stats(df_normalized_mad, ['Cells_AreaShape_Area', 'Nuclei_Texture_Variance_RNA_10_03_256'])
# len(col_dict['Cells'])
# col_dict['Cells'][:20]

In [ ]:
# fig = px.box(df_normalized, y=cols_subset[:34], notched=True)
import plotly.graph_objects as go
from plotly.colors import n_colors
# px.violin(df_normalized, y = col_list,  width=1600, height=800  )
 

col_list = col_dict['Cells'][:20]
col_list = col_dict['Cells'][:35]

colors = n_colors('rgb(5, 200, 200)', 'rgb(200, 10, 10)', len(col_list), colortype='rgb')
# del fig
fig = go.Figure();
for data_line, color in zip(col_list , colors):
    col_name = '_'.join(data_line.split('_')[2:])
    fig.add_trace(go.Violin(y=df_normalized[data_line], line_color=color, name = col_name,box_visible=True,));
fig.update_layout(xaxis_showgrid=False, 
                    xaxis_zeroline=True,
                    autosize=False,
                    width=2500,
                    height=750,
                    margin=dict(
                        l=50,
                        r=50,
                        b=20,
                        t=20,
                        pad=4
                    ),
                    paper_bgcolor="LightSteelBlue",
);

In [ ]:
fig.show()

# Read Binned Profile files

In [ ]:
# profilesFile = CompoundProfiles2SampleFile + '_001' + CSV
# oldOutputFile ='./output_11102023/compound_profiles_2samples_old2.csv.bz2'
x=17
x = 1
x = 4
profilesFile =  "./output_11102023/binned_profiles/binned_2sample_profiles_{0:03d}.csv"
print(f" Read from {profilesFile.format(x)}")

In [ ]:
df_profiles =  pd.read_csv(profilesFile, 
                           # nrows = 20,
                           # skiprows = 0, 
                           # header = 0, 
                           names = COMPOUND_PROFILE_COLUMNS, 
                           usecols = MetadataNames, 
                           # usecols = ['Metadata_JCP2022', 'Metadata_permiation', 'Cells_AreaShape_Area']
                           dtype= string_conversions,
                           # dtype = conversionDict,
                          )

In [ ]:
df_profiles.shape
df_profiles.columns
# df_profiles.loc[0]
# df_profiles.memory_usage(deep=True).sum()
# df_profiles.memory_usage(deep=True)[:20]
df_profiles.head(5)
# df_profiles.info(verbose=True)
# HEADER_LIST = df_profiles.columns.to_list()

In [ ]:
# files = [f for f in pathlib.Path().glob(f"{output_path}/profiles/*.csv")]
# files.sort()
# files
# for profilesFile in files:

# for x in range(1,12):
#     profilesFile = CompoundProfiles2SampleFile + f"_{x:03d}" + CSV
#     profilesFile_a = CompoundProfiles2SampleFile + f"_{x:03d}_a" + CSV
    
#     df_iter =  pd.read_csv(profilesFile, 
#                            low_memory = True,
#                            # skiprows = 0,
#                            # nrows = 5,
#                            header = 0,
#                            # chunksize= 10000,  
#                            # iterator = True, 
#                            # dtype = conversionDict,
#                            # names=JOIN_COLUMNS,
#                            # usecols = JOIN_COLUMNS
#                           )
#     # for i, df_profiles in enumerate(df_iter):
#     # print(f"*** Chunk {i}   Starting index : {df_profiles.index[0]}   Ending Index {df_profiles.index[-1]}")
#     # print(f"\t Number of rows {df_profiles.shape[0]}   Size of chunk read: {df_profiles.memory_usage(deep=True).sum():,d} bytes")
#     # inv_cols = check_values(df_profiles,  numeric_columns)
#     print("-"*150)
#     print(f" {profilesFile}   {df_iter.shape}") 
#     print("-"*150)
#     df_iter.head(3)
#     print("\n\n")
    
#     # df_profiles.tail(5)
#     # if x > 13 :
#     break

In [ ]:
# files = [f for f in pathlib.Path().glob(f"{output_path}/profiles/*.csv")]
# files.sort()
# files
# for profilesFile in files:

for x in range(1,6):
    
    # profilesFile = CompoundProfiles2SampleFile + f"_{x:03d}" + CSV
    # profilesFile_a = CompoundProfiles2SampleFile + f"_{x:03d}_a" + CSV
    
    df_iter =  pd.read_csv(profilesFile.format(x), 
                           low_memory = True,
                           # skiprows = 0,
                           # nrows = 5,
                           header = 0,
                           names = COMPOUND_PROFILE_COLUMNS, 
                           # names=JOIN_COLUMNS,
                           usecols = MetadataNames, 
                           # usecols = ['Metadata_JCP2022', 'Metadata_permiation', 'Cells_AreaShape_Area']
                           dtype= string_conversions,
                           # dtype = conversionDict,
                           # chunksize= 10000,  
                           # iterator = True, 
                          )
    # for i, df_profiles in enumerate(df_iter):
    # print(f"*** Chunk {i}   Starting index : {df_profiles.index[0]}   Ending Index {df_profiles.index[-1]}")
    # print(f"\t Number of rows {df_profiles.shape[0]}   Size of chunk read: {df_profiles.memory_usage(deep=True).sum():,d} bytes")
    # inv_cols = check_values(df_profiles,  numeric_columns)
    print("-"*150)
    print(f" {profilesFile.format(x)}   {df_iter.shape}") 
    print("-"*150)
    df_iter.head(4)
    df_iter.tail(4)
    print("\n\n")
    
    # df_profiles.tail(5)
    # if x > 13 :
    # break

# Experiments on reducing the memory foot print 

- converting the Metadata fields from `object` to `category`

In [ ]:
try: 
    del df_profiles2
except Exception as e:
    pass

In [ ]:
outputFile = CompoundProfiles2SampleFile
type = '.bz2'

compoundProfiles = outputFile+type
# del df_profiles_list 
print(f" Read from {compoundProfiles}")

In [ ]:
print(compoundProfiles)

In [ ]:
df_profiles2 =  pd.read_csv(compoundProfiles, usecols = ['Metadata_Source'], nrows=30000)

In [ ]:
df_profiles2.info()

df_profiles2.memory_usage(deep=True)[:6]

In [ ]:
## experiment with converting columns to "category" type to reduce memory footprint

# df_profiles2 = df_profiles.copy()
# del df_profiles2
# df_profiles2["Metadata_Source"] = df_profiles2["Metadata_Source"].astype("category")
# df_profiles2["Metadata_Batch"] = df_profiles2["Metadata_Batch"].astype("category")
# df_profiles2["Metadata_Plate"] = df_profiles2["Metadata_Plate"].astype("category")
# df_profiles2["Metadata_Well"] = df_profiles2["Metadata_Well"].astype("category")

# df_profiles2["Metadata_JCP2022"] = df_profiles2["Metadata_JCP2022"].astype("object")

In [ ]:
df_profiles2.memory_usage(deep=True)[:6]

In [ ]:
del df_profiles2

In [ ]:
df_profiles2.memory_usage(deep=True)[:6]
df_profiles2.head(10)
df_profiles2.tail(10)

**Each row in `dframes` is well-level profile, containing thousands of features (n=4762) averaged over (typically) a couple of thousand cells per well.**

In [ ]:
df_profiles.columns[-5:]

In [ ]:
# df_profiles.drop(columns='_merge', inplace = True)
df_profiles2.columns[:20]
df_profiles2.columns[-5:]
df_profiles2.head(2)
# df_profiles.index.value_counts()
print('---------------------------------------------------------------')
print(' Before Index Reset')
print('---------------------------------------------------------------')
df_profiles2[1552:1557]
print('---------------------------------------------------------------')
d

In [ ]:
df_profiles2.reset_index(drop=True, inplace = True)
df_profiles2.info()
print('---------------------------------------------------------------')
print(' After Index Reset')
print('---------------------------------------------------------------')
df_profiles2.head(5)
df_profiles2.tail(5)
print('---------------------------------------------------------------')

In [ ]:
df_info = df_profiles.info(max_cols = 10,verbose = True, memory_usage = True, show_counts = True)

In [ ]:
st_col, end_col = 0,10
col_list = list(df_profiles.columns)
col_list[st_col:end_col]
df_profiles[col_list[st_col:end_col]].shape
df_profiles[col_list[st_col:end_col]]

In [ ]:
# dframes.drop(index=384, inplace = True)

# Get Plate Level Stats

In [ ]:
def get_stats(df_stats, df_data):
    df_data_cols = df_data.columns.tolist()
    print(df_data_cols[:10])
    if len(df_stats.columns) == 0 :
        print(f" length of df_stats.columns: {len(df_stats.columns)}")
        df_stats = pd.DataFrame(columns = ["Stat"]+(df_data.columns.tolist()) ) 
        df_stats_cols = df_stats.columns.tolist()
        print(df_stats_cols[:10])
    col_data = [df_data.iloc[0,0], df_data.iloc[0,1], "All wells"]
    print(col_data)
    df_stats.loc[len(df_stats)] = ["Minimum"] + col_data + df_data.min(0).tolist()[5:] 
    df_stats.loc[len(df_stats)] = ["Maximum"] + col_data + df_data.max(0).tolist()[5:] 
    # df_stats.loc[len(df_stats)] = ["Mean"]    + col_data + df_data.mean(0, numeric_only=True).tolist()[5:] 
    # df_stats.loc[len(df_stats)] = ["Median"]  + col_data + df_data.median(0, numeric_only=True).tolist()[5:] 
    print(type(df_stats), len(df_stats))
    return df_stats



In [ ]:
del stats
stats = pd.DataFrame()
stats = get_stats(stats, df_profiles)

In [ ]:
print(type(stats), len(stats))
stats

In [ ]:
# for i in dframes.columns[:500]:
#     print(f' {i}')
# stats.columns.to_series().to_csv('./output/measurement_column_names.csv')
stats.to_csv('./output/sample_statistics.csv')

## Join features with Compound and Well metadata

The profiles are annotated with only three columns of metadata (source, plate, well).

Let's add more metadata!

In [ ]:
# wells[(wells.Metadata_Source == "source_3") & (wells.Metadata_Plate == "BR5874c3") & (wells.Metadata_JCP2022 == "JCP2022_999999")][20:50]

In [ ]:
wells.head(3)
plates.head(3)

In [ ]:
metadata = wells.merge(plates, on=["Metadata_Source", "Metadata_Plate"])

In [ ]:
metadata.info()
metadata.head()
metadata.Metadata_PlateType.value_counts()
metadata2.Metadata_JCP2022.value_counts()

In [ ]:
compound.head(3)

In [ ]:
metadata2 = metadata.merge(compound, on="Metadata_JCP2022")
metadata2.info()
metadata2.head()
metadata2.Metadata_PlateType.value_counts()
metadata2.Metadata_JCP2022.value_counts()

In [ ]:
jcp_999999_idxs = metadata2[ metadata2.Metadata_JCP2022 == "JCP2022_999999"].index
len(jcp_999999_idxs)

In [ ]:
jcp_033924_idxs = metadata2[ metadata2.Metadata_JCP2022 == "JCP2022_033924"].index
len(jcp_033924_idxs)

In [ ]:
# jcp_all_idxs = jcp_999999_idxs + jcp_033924_idx
jcp_bad_idxs = pd.Index.append(jcp_999999_idxs,jcp_033924_idxs)
len(jcp_bad_idxs)

In [ ]:
metadata3 = metadata2.drop(jcp_bad_idxs)

In [ ]:

# metadata2 = compound.merge(wells, on="Metadata_JCP2022")
# metadata2.info()
# metadata2.head()
# metadata2.Metadata_JCP2022.value_counts()

# metadata3 = metadata2.merge(plates, on=["Metadata_Source", "Metadata_Plate"])
metadata3.info()
metadata3.head()
metadata3.Metadata_JCP2022.value_counts()[:20]

### Save as `compound_well_plate.csv`

In [ ]:
compoundWellPlateFile         = f"./output/cwp_metadata_clean.csv"
metadata3.to_csv(compoundWellPlateFile)

In [ ]:
ann_dframe = metadata3.merge(    
    dframes, on=["Metadata_Source", "Metadata_Plate", "Metadata_Well"]
)

We now know a little bit more about each profile:

In [ ]:
ann_dframe.info()
ann_dframe.head()
ann_dframe.Metadata_PlateType.value_counts()
ann_dframe.Metadata_JCP2022.value_counts()[:20]

In [ ]:
# ann_dframe.sample(2, random_state=42)
ann_dframe.sort_values("Metadata_JCP2022")

More metadata information will be added in the future. 

# Plot features

The scatterplot below contains every well in the sampled dataset.

In the interactive plot (see settings for `pio.renderers.default` above), you can hover over the points to see the JCP ID and the InChiKey for a given compound.

<div class="alert alert-warning">
NOTE: Because these are raw, unnormalized features, you will notice discernable clusters corresponding to each source due to batch effects.
Upcoming data releases will included normalized features, where these effects are mitigated to some extent. 
</div>

In [ ]:
 # pio.renderers.default = 'jupyterl`ab'
# pio.renderers.default = 'notebook_connected'
pio.renderers.default = "browser"

In [ ]:
from pickle import FALSE, TRUE
del fig
fig = px.scatter(
    df_profiles,
    x="Cells_AreaShape_Eccentricity",
    y="Cells_AreaShape_EulerNumber",
    color="Metadata_Source",
    hover_name="Metadata_JCP2022",
    hover_data=["Metadata_Source"],
)

fig.show()

### **Use DASH**

In [ ]:
from dash import Dash, dcc, html, Input, Output
import plotly.express as px

app = Dash(__name__)

app.layout = html.Div([
    html.H4('Analysis of Iris data using scatter matrix'),
    dcc.Dropdown(
        id="dropdown",
        options=useColumns,
        value=[useColumns[0], "Cells_AreaShape_Eccentricity"],
        multi=True
    ),
    dcc.Graph(id="graph"),
])


@app.callback(
    Output("graph", "figure"), 
    Input("dropdown", "value"))
def update_bar_chart(dims):
    # df = px.data.iris() # replace with your own data source
    # fig = px.scatter_matrix(
    #     df_profiles, dimensions=dims, color="species")
    print(value)
    fig = px.scatter(
            df_profiles,
            x="Cells_AreaShape_Eccentricity",
            y="Cells_AreaShape_EulerNumber",
            color="Metadata_Source",
            hover_name="Metadata_JCP2022",
            hover_data=["Metadata_Source"],
        )
    return fig


app.run_server(debug=True,port=8051)

So that's just a couple of (raw) measurements from the sentinel plates for 10/13 of the sources, for the principal dataset alone. 